DC, for the Employment data analysis Please do the Data preparatin as following:

Get the following fields from the CICdata

  customer_id, TIN, run_date, occupationdesc, occupationstatusdesc, TradeName, employment_duration_in_days (DateHiredTo or run_date - DateHiredFrom), PSICdesc, AnnualMonthlyIndicator, Currency, GrossIncome, max_sanctioned_amt_24m, max_creditcard_limit_24m, max_financed_amt_24m, tot_granted_contracts_cnt_24m, tot_granted_contracts_amt_12m, max_sanctioned_amt_12m, Max_CreditCard_limit_12m, max_financed_amt_12m, tot_granted_contracts_cnt_12m, tot_granted_contracts_amt_12m
  
  Get the self declared employment details, application_date,  and MaxSalary information from our internal Salary scaling table using the CustomerID or TIN and application_date = run_date
  
  Get the gender, age, region, province of the customer from demographic table using customer_id or TIN and application_date = run_date
  
  Get the following fields from the Credolab tables using the customer_id or mobile number and application_date = run date:
  Device Price
  Device Manufacturer (Apple/ Samsung/ Oppo etc.)
  Device Model
  Device Age (usually derived by the application_date - earliest apps installation date)
  number of job search apps installed (linkedin, jobstreet, glint etc.)
  number of professional apps installed (office, gdrive etc)
  number of ecommerce apps installed (lazada, shopee, shein, zalora, temu etc.)
  number of ride hailing apps installed (grab, moveit etc.)
  number of finance apps installed (Banking apps, FinTech apps, eWallet apps)
  number of Risky apps (gambling apps, crypto apps, payday loan apps)
  number of OTT apps (Netflix, PrimeVideo etc.)
  Sowndarya: may be you can share your earlier apps categorisation work with DC for this. To keep things simple, just give him your final app categorisation mapping.

DC, next steps on the Income data:<br>
Tradename column from CICdata is missing, pls add it<br>
Calculate the employment duration (if no end date then run date - start date, otherwise end date - start date, if no start date then null)<br>
Create two additional flags from CICdata:<br>
--> Has Credit Loan (credit_loan_flag)<br>
--> Has Installment loan flag (inst_loan_flag)<br>
Add the other Credolab fields:<br>
--> App category wise count (just use Credolab app categories)<br>
-->  Device Brand (from credolab_raw.android_credolab_datasets_struct_columns` select Hardware.device_brand)<br>
--> Device Model (from credolab_raw.android_credolab_datasets_struct_columns` select Hardware.device_model)<br>
--> Network Provider (from credolab_raw.android_credolab_datasets_struct_columns` select GeneralData.sim_operator_name)<br>
--> Number of BlueTooth Device (from credolab_raw.android_credolab_datasets_struct_columns` select Hardware.bluetooth__bonded_devices_count)<br>

# <div align="center" style="color: #ff5733;">Income Estimation Data Preparation For Model</div>

Dated: 20-08-2024

# Declare Libraries

In [ ]:
# %% [markdown]
# # Jupyter Notebook Loading Header
#
# This is a custom loading header for Jupyter Notebooks in Visual Studio Code.
# It includes common imports and settings to get you started quickly.

# %% [markdown]
## Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
import os
path = r'C:\Users\DwaipayanChakroborti\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path
client = bigquery.Client(project='prj-prod-dataplatform')
from pandasql import sqldf

# %% [markdown]
## Configure Settings
# Set options or configurations as needed
# Example: pd.set_option('display.max_columns', None)

import time

# Query

In [2]:
sq = """
with 
ed
as 
( 
SELECT distinct
  digitalLoanAccountId,
  crifApplicationId,
  customerId,
  OccupationDesc,
  OccupationStatusDesc,
  PSICDesc,
  TradeName,
  AnnualMonthlyIndicator,
  GrossIncome,
  CAST(
    CASE
      WHEN COALESCE(AnnualMonthlyIndicator, 'NA') LIKE 'M' THEN ROUND(CAST(COALESCE(GrossIncome, '0') AS NUMERIC)*12, 0)
      WHEN COALESCE(AnnualMonthlyIndicator, 'NA') LIKE 'Y' THEN CAST(COALESCE(GrossIncome, '0') AS NUMERIC)
      ELSE 0
    END AS INT64
  ) AS AnnualIncomecalculated,
  LastUpdateDate,
  row_number() over (partition by digitalLoanAccountId order by LastUpdateDate desc) rnk
from 
prj-prod-dataplatform.risk_credit_cic_data.employment_data
)
-- select customerId, count(customerId) from ed group by 1 having count(customerId) > 1 order by 2 desc
-- select * from ed where rnk = 1 order by LastUpdateDate desc
,
ed1 
as 
(select * from ed where rnk = 1)
-- select * from ed1
,
educate as 
(select distinct edu.digitalLoanAccountId, edu.education_id, edu1.description
from `prj-prod-dataplatform.dl_loans_db_raw.tdbk_loan_purpose` edu
inner join (select id, description from dl_loans_db_raw.tdbk_loan_lov_mtb where module = 'Education') edu1 on edu.education_id = edu1.id
),
educate2 as 
(select *, row_number() over(partition by digitalLoanAccountId order by education_id desc) rnk from educate),
educate3 as 
(select * from educate2 where rnk = 1),
reg as 
(select 
distinct
hm_province,
CASE
  WHEN hm_province = 'KALINGA' THEN 'CAR – Cordillera Administrative Region'
  WHEN hm_province = 'OCCIDENTAL MINDORO' THEN 'MIMAROPA Region'
  WHEN hm_province = 'ORIENTAL MINDORO' THEN  'MIMAROPA Region'
  WHEN hm_province = 'ORODNIM' THEN 'MIMAROPA Region'
  WHEN hm_province = 'SAMAR (WESTERN SAMAR)' THEN 'Region VIII – Eastern Visayas'
  WHEN hm_province = 'ZAMBOANGA SIBUGAY' THEN 'Region IX – Zamboanga Peninsula'
  ELSE Region_name END Region_Name
FROM prj-prod-dataplatform.dl_customers_db_raw.tdbk_customer_add_mtb f 
LEFT JOIN prj-prod-dataplatform.dap_ds_poweruser_playground.region_mappings h ON LOWER(f.hm_province) = LOWER(h.province)),
lm
as   
(select distinct customerId, lmt.digitalLoanAccountId, lmt.loanAccountNumber, startApplyDateTime, natureofwork,industryDescription, incomeMultiple, employmentstatus,credolabScore,city,
       barangay, province, postalCode, age, monthlyIncome,  tinNumber, gender, dateOfBirth, email, mobileNo,firstName, middleName, lastName, custLoanJourneyZipCode,
       custLoanJourneyProvince, custLoanJourneyCity,  tsa_onboarding_time,new_loan_type, dependentsCount, deviceType,brand, osversion_v2,osVersion, appVersion,
       cast(incomeperSalaryScale as numeric) incomeperSalaryScale, maritalStatus, education_id, description as Education,
       Case when decision_date is not null then case when flagApproval = 1 and flagDisbursement = 1  then 'Disbursed'
            when flagApproval = 1 and flagDisbursement = 0 and flagRejection = 0 then 'Approved Not Disbursed'
            when flagRejection = 1 and flagApproval = 0 then 'Rejected' else 'No Decision' end else 'No Decision' end ApplicationStatus,
       case when flagDisbursement = 1 and obsFPD10 = 1 and coalesce(currentDelinquency, 0) > 0 then 'Bad' 
            when flagDisbursement = 1 and obsFPD10 = 1 and coalesce(currentDelinquency, 0) = 0 then 'Good'
            Else 'Not Eligible' end Delinquency , defFPD15, obsFPD15,
      case when c.obs_min_inst_def30>=1 then lmt.disbursedloanamount else 0 end as obs_fpd30_vol,
      case when c.min_inst_def30=1 then lmt.disbursedloanamount else 0 end as def_fpd30_vol,
      case when c.obs_min_inst_def30>=1 then 1 else 0 end as obs_fpd30_cnt,
      case when c.min_inst_def30=1 then 1 else 0 end as def_fpd30_cnt,
      case when c.obs_min_inst_def30>=2 then lmt.disbursedloanamount else 0 end as obs_fspd30_vol,
      case when c.min_inst_def30 <=2 then lmt.disbursedloanamount else 0 end as def_fspd30_vol,
      case when c.obs_min_inst_def30>=2 then 1 else 0 end as obs_fspd30_cnt,
      case when c.min_inst_def30<=2 then 1 else 0 end as def_fspd30_cnt,
      case when c.obs_min_inst_def30>=3 then lmt.disbursedloanamount else 0 end as obs_fstpd30_vol,
      case when c.min_inst_def30<=3 then lmt.disbursedloanamount else 0 end as def_fstpd30_vol,
      case when c.obs_min_inst_def30>=3 then 1 else 0 end as obs_fstpd30_cnt,
      case when c.min_inst_def30<=3 then 1 else 0 end as def_fstpd30_cnt,
      reg.Region_Name,
      row_number() over (partition by customerId order by startApplyDateTime desc) rnk
from prj-prod-dataplatform.risk_credit_mis.loan_master_table lmt
left join educate3 on educate3.digitalLoanAccountId = lmt.digitalLoanAccountId
left join `prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data` c on lmt.loanAccountNumber=c.loanAccountNumber
left join reg on reg.hm_province = coalesce(lmt.custLoanJourneyProvince, lmt.province)
),
lm1
as
(select customerId, digitalLoanAccountId,
 tinNumber, dateOfBirth, email, mobileNo,firstName, middleName, lastName, custLoanJourneyZipCode,
        tsa_onboarding_time,new_loan_type, 
natureofwork,
age,
Education,
industryDescription,
custLoanJourneyProvince, 
monthlyIncome, 
incomeperSalaryScale ,
gender, 
credolabScore,
maritalStatus,
dependentsCount,
custLoanJourneyCity,
Region_Name,
deviceType,
brand, 
coalesce(osversion_v2, osVersion) osversion, 
appVersion,
defFPD15,
obsFPD15,
obs_fpd30_vol,
def_fpd30_vol,
obs_fpd30_cnt,
def_fpd30_cnt,
obs_fspd30_vol,
def_fspd30_vol,
obs_fspd30_cnt,
def_fspd30_cnt,
obs_fstpd30_vol,
def_fstpd30_vol,
obs_fstpd30_cnt,
def_fstpd30_cnt
from lm where rnk = 1)
select lm1.customerId, 
lm1.digitalLoanAccountId ,
lm1.tinNumber,
lm1.dateOfBirth,
lm1.email,
lm1.mobileNo,
lm1.firstName,
lm1.middleName,
lm1.lastName,
lm1.custLoanJourneyZipCode,
lm1.custLoanJourneyCity,
lm1.tsa_onboarding_time,
lm1.new_loan_type,
lm1.natureofwork,
lm1.age, 
lm1.Education,
lm1.industryDescription,
lm1.custLoanJourneyProvince,
lm1.monthlyIncome,
round(cast(lm1.monthlyIncome as numeric) *12, 0) AnnualIncome,
lm1.incomeperSalaryScale,
round(incomeperSalaryScale * 12, 0) incomeperSalaryScale_Annual,
lm1.gender,
lm1.credolabScore,
lm1.maritalStatus,
lm1.dependentsCount,
lm1.Region_Name,
lm1.deviceType,
lm1.brand,
lm1.osversion,
lm1.appVersion,
lm1.defFPD15,
lm1.obsFPD15,
lm1.obs_fpd30_vol,
lm1.def_fpd30_vol,
lm1.obs_fpd30_cnt,
lm1.def_fpd30_cnt,
lm1.obs_fspd30_vol,
lm1.def_fspd30_vol,
lm1.obs_fspd30_cnt,
lm1.def_fspd30_cnt,
lm1.obs_fstpd30_vol,
lm1.def_fstpd30_vol,
lm1.obs_fstpd30_cnt,
lm1.def_fstpd30_cnt,
ed1.OccupationDesc,
ed1.OccupationStatusDesc,
ed1.PSICDesc,
ed1.TradeName,
ed1.AnnualMonthlyIndicator,
ed1.GrossIncome,
ed1.AnnualIncomecalculated,
ed1.LastUpdateDate
from lm1
left join ed1 on ed1.digitalLoanAccountId = lm1.digitalLoanAccountId
;
"""

df = client.query(sq).to_dataframe(progress_bar_type='tqdm')

Job ID b27173b8-ff43-43db-bd5f-c744a64e0017 successfully executed: |          |
Downloading: 100%|██████████|


In [3]:
print(f"The rows and columns are:\t {df.shape}")

The rows and columns are:	 (1302893, 53)


In [4]:
df.head()

,customerId,digitalLoanAccountId,tinNumber,dateOfBirth,email,mobileNo,firstName,middleName,lastName,custLoanJourneyZipCode,...,obs_fstpd30_cnt,def_fstpd30_cnt,OccupationDesc,OccupationStatusDesc,PSICDesc,TradeName,AnnualMonthlyIndicator,GrossIncome,AnnualIncomecalculated,LastUpdateDate
0,2428539,ab9f1da9-4762-4079-86e8-a148f4033ec5,None,22 Nov 1996,spormentira@gmail.com,639062981758,STEPHANIE,SAMIA,PORMENTIRA,None,...,0,0,None,None,None,None,None,None,<NA>,None
1,2459679,44660170-23ba-414e-a13f-5182da4ddbce,252031028,13 Jan 1988,tcl.jonjonmercado_j88@yahoo.com,639563712269,JON JON,FERNANDEZ,MERCADO,4027,...,0,0,None,None,None,None,None,None,<NA>,None
2,2474677,a36703e2-3cae-4222-9ce1-421ae910ea8f,None,01 Apr 1998,llasus.jksl@gmail.com,639983176216,JOANNA KRISTI,SAMSON,LLASUS,None,...,0,0,None,None,None,None,None,None,<NA>,None
3,2480762,d2f36c9e-9573-4b2d-9ba9-f85db00acc27,None,08 Jun 1984,cunananalyn8@gmail.com,639669837559,ANALYN,APORADOR,CUNAN,1428,...,0,0,None,None,None,None,None,None,0,2024-04-25
4,2483808,1ae4f606-f235-4857-a1cc-8c11b926549e,307398157,19 Mar 1988,syrianna29@yahoo.com,639559318471,MOHAMMAD WASEEM,ALANGADI,SALEM,1127,...,0,0,None,None,None,None,None,None,<NA>,None


In [5]:
counts = df['digitalLoanAccountId'].value_counts()
duplicates = counts > 1

if duplicates.any():
    print("There are duplicates in the digitalLoanAccountId column")
else:
    print("No duplicates found in the digitalLoanAccountId column")

No duplicates found in the digitalLoanAccountId column


In [6]:
df.to_csv("StepA_Education_withbadrate.csv", index = False)

# CIC Data

In [7]:
sq = """WITH
  CICBaseTable AS ----To create combine CIC raw data combining Granted and Non Granted table
  ( -- Query FOR dfgranted
  SELECT
    digitalLoanAccountId,
    crifApplicationId,
    customerId,
    processEngineGuid,
    requestGuid,
    ContractHistoryType,
    CBContractCode,
    ContractEndDate,
    ContractPhase,
    ContractPhaseDesc,
    ContractStartDate,
    ContractStatus,
    ContractStatusDesc,
    ContractType,
    ContractTypeDesc,
    Currency,
    CurrencyDesc,
    LastUpdateDate,
    OriginalCurrency,
    OriginalCurrencyDesc,
    ProviderCodeEncrypted,
    ProviderContractNo,
    ReferenceNo,
    Role,
    RoleDesc,
    BilledAmount,
    BoardResolutionFlag,
    BoardResolutionFlagDesc,
    CancellationDate,
    CardReferenceCode,
    ChargedAmount,
    CreditLimit,
    CreditPurpose,
    CreditPurposeDesc,
    FinancedAmount,
    FirstPaymentDate,
    FlagCardUsed,
    HolderLiability,
    HolderLiabilityDesc,
    InstallmentType,
    InstallmentTypeDesc,
    InstallmentsNumber,
    LastChargeDate,
    LastPaymentAmount,
    LastPaymentDate,
    MinPaymentIndicator,
    MinPaymentIndicatorDesc,
    MinPaymentPercentage,
    MonthlyPaymentAmount,
    NextPayment,
    NextPaymentDate,
    OutstandingBalance,
    OutstandingBalanceUnbilled,
    OutstandingPaymentsNumber,
    OverallCreditLimit,
    OverdueDays,
    OverdueDaysDesc,
    OverduePaymentsAmount,
    OverduePaymentsNumber,
    PaymentMethod,
    PaymentMethodDesc,
    PaymentPeriodicity,
    PaymentPeriodicityDesc,
    PremiumCard,
    PremiumCardDesc,
    ReorganizedCreditCode,
    ReorganizedCreditCodeDesc,
    ServicesLinesNo,
    TimesCardUsed,
    TransactionType,
    TransactionTypeDesc,
    Utilization,
    LinkedSubject_CBSubjectCode,
    LinkedSubject_Name,
    LinkedSubject_Role,
    LinkedSubject_RoleDesc,
    Note_TypeDesc,
    Note_Text,
    Note_Type,
    run_date,
    NULL AS ContractRequestDate,
    'granted' AS SOURCE
  FROM
    prj-prod-dataplatform.risk_credit_cic_data.granted_contracts
  UNION ALL
    -- Query FOR dfnongranted
  SELECT
    digitalLoanAccountId,
    crifApplicationId,
    customerId,
    processEngineGuid,
    requestGuid,
    NULL AS ContractHistoryType,
    CBContractCode,
    NULL AS ContractEndDate,
    ContractPhase,
    ContractPhaseDesc,
    NULL AS ContractStartDate,
    NULL AS ContractStatus,
    NULL AS ContractStatusDesc,
    ContractType,
    ContractTypeDesc,
    NULL AS Currency,
    NULL AS CurrencyDesc,
    LastUpdateDate,
    NULL AS OriginalCurrency,
    NULL AS OriginalCurrencyDesc,
    ProviderCodeEncrypted,
    ProviderContractNo,
    ReferenceNo,
    Role,
    RoleDesc,
    NULL AS BilledAmount,
    NULL AS BoardResolutionFlag,
    NULL AS BoardResolutionFlagDesc,
    NULL AS CancellationDate,
    NULL AS CardReferenceCode,
    NULL AS ChargedAmount,
    CreditLimit,
    NULL AS CreditPurpose,
    NULL AS CreditPurposeDesc,
    FinancedAmount,
    NULL AS FirstPaymentDate,
    NULL AS FlagCardUsed,
    NULL AS HolderLiability,
    NULL AS HolderLiabilityDesc,
    NULL AS InstallmentType,
    NULL AS InstallmentTypeDesc,
    InstallmentsNumber,
    NULL AS LastChargeDate,
    NULL AS LastPaymentAmount,
    NULL AS LastPaymentDate,
    NULL AS MinPaymentIndicator,
    NULL AS MinPaymentIndicatorDesc,
    NULL AS MinPaymentPercentage,
    MonthlyPaymentAmount,
    NULL AS NextPayment,
    NULL AS NextPaymentDate,
    NULL AS OutstandingBalance,
    NULL AS OutstandingBalanceUnbilled,
    NULL AS OutstandingPaymentsNumber,
    NULL AS OverallCreditLimit,
    NULL AS OverdueDays,
    NULL AS OverdueDaysDesc,
    NULL AS OverduePaymentsAmount,
    NULL AS OverduePaymentsNumber,
    NULL AS PaymentMethod,
    NULL AS PaymentMethodDesc,
    PaymentPeriodicity,
    PaymentPeriodicityDesc,
    NULL AS PremiumCard,
    NULL AS PremiumCardDesc,
    NULL AS ReorganizedCreditCode,
    NULL AS ReorganizedCreditCodeDesc,
    NULL AS ServicesLinesNo,
    NULL AS TimesCardUsed,
    NULL AS TransactionType,
    NULL AS TransactionTypeDesc,
    NULL AS Utilization,
    LinkedSubject_CBSubjectCode,
    LinkedSubject_Name,
    LinkedSubject_Role,
    LinkedSubject_RoleDesc,
    Note_TypeDesc,
    Note_Text,
    Note_Type,
    run_date,
    ContractRequestDate,
    'nongranted' AS SOURCE
  FROM
    prj-prod-dataplatform.risk_credit_cic_data.notgranted_contracts )
    -- select * from CICBaseTable where digitalLoanAccountId = 'c7948327-e6a6-46ee-96fc-66c1b3b56f93'
-- SELECT  ContractHistoryType, RoleDesc, count(digitalLoanAccountId) cnt FROM  CICBaseTable where COALESCE(ContractHistoryType, 'NA') in ('Installments', 'CreditCards', 'NA') group by 1,2 order by 3 desc;
,
employementdata as
(SELECT distinct
  digitalLoanAccountId,
  crifApplicationId,
  customerId,
  AnnualMonthlyIndicator,
  Currency,
  DateHiredFrom,
  DateHiredTo,
  GrossIncome,
  CAST(
    CASE
      WHEN COALESCE(AnnualMonthlyIndicator, 'NA') LIKE 'M' THEN CAST(COALESCE(GrossIncome, '0') AS NUMERIC)
      WHEN COALESCE(AnnualMonthlyIndicator, 'NA') LIKE 'Y' THEN ROUND(CAST(COALESCE(GrossIncome, '0') AS NUMERIC)/12, 0)
      ELSE 0
    END AS INT64
  ) AS MonthlyIncome,
  CAST(
    CASE
      WHEN COALESCE(AnnualMonthlyIndicator, 'NA') LIKE 'M' THEN ROUND(CAST(COALESCE(GrossIncome, '0') AS NUMERIC)*12, 0)
      WHEN COALESCE(AnnualMonthlyIndicator, 'NA') LIKE 'Y' THEN CAST(COALESCE(GrossIncome, '0') AS NUMERIC)
      ELSE 0
    END AS INT64
  ) AS AnnualIncome,
  OccupationDesc,
  OccupationStatusDesc,
  PSIC,
  REGEXP_REPLACE(PSICDesc, r'^\d+\s*-\s*', '') AS PSICDesc ,
  row_number() over (partition by digitalLoanAccountId order by digitalLoanAccountId ) as rnk
FROM prj-prod-dataplatform.risk_credit_cic_data.employment_data),
CICBase2Table as
(SELECT digitalLoanAccountId, crifApplicationId, customerId,
       processEngineGuid, requestGuid, ContractHistoryType,
       CBContractCode, ContractEndDate, ContractPhase,
       ContractPhaseDesc, ContractStartDate, ContractStatus,
       ContractStatusDesc, ContractType, ContractTypeDesc,
       Currency, CurrencyDesc, LastUpdateDate, OriginalCurrency,
       OriginalCurrencyDesc, ProviderCodeEncrypted,
       ProviderContractNo, ReferenceNo, Role, RoleDesc,
       BilledAmount, BoardResolutionFlag, BoardResolutionFlagDesc,
       CancellationDate, CardReferenceCode, ChargedAmount,
       CreditLimit, CreditPurpose, CreditPurposeDesc,
       FinancedAmount, FirstPaymentDate, FlagCardUsed,
       HolderLiability, HolderLiabilityDesc, InstallmentType,
       InstallmentTypeDesc, InstallmentsNumber, LastChargeDate,
       LastPaymentAmount, LastPaymentDate, MinPaymentIndicator,
       MinPaymentIndicatorDesc, MinPaymentPercentage,
       MonthlyPaymentAmount, NextPayment, NextPaymentDate,
       OutstandingBalance, OutstandingBalanceUnbilled,
       OutstandingPaymentsNumber, OverallCreditLimit, OverdueDays,
       OverdueDaysDesc, OverduePaymentsAmount,
       OverduePaymentsNumber, PaymentMethod, PaymentMethodDesc,
       PaymentPeriodicity, PaymentPeriodicityDesc, PremiumCard,
       PremiumCardDesc, ReorganizedCreditCode,
       ReorganizedCreditCodeDesc, ServicesLinesNo, TimesCardUsed,
       TransactionType, TransactionTypeDesc, Utilization,
       LinkedSubject_CBSubjectCode, LinkedSubject_Name,
       LinkedSubject_Role, LinkedSubject_RoleDesc, Note_TypeDesc,
       Note_Text, Note_Type, run_date, ContractRequestDate,  SOURCE
,
  CASE
    WHEN ContractPhaseDesc = 'Active' AND ContractStatusDesc = '' THEN 'Neutral'
    WHEN ContractPhaseDesc = 'Active' AND ContractStatusDesc is null THEN 'Neutral'
    WHEN ContractPhaseDesc = 'Active' AND ContractStatusDesc = 'Pre-Activated' THEN 'Good'
    WHEN ContractPhaseDesc = 'Active' AND ContractStatusDesc = 'Foreclosure' THEN 'Good'
    WHEN ContractPhaseDesc = 'Closed' AND ContractStatusDesc = '' THEN 'Good'
    WHEN ContractPhaseDesc = 'Closed' AND ContractStatusDesc is null THEN 'Good'
    WHEN ContractPhaseDesc = 'Closed in advance' AND ContractStatusDesc = '' THEN 'Good'
    WHEN ContractPhaseDesc = 'Closed in advance' AND ContractStatusDesc is null THEN 'Good'
    WHEN ContractPhaseDesc = 'Closed in advance' AND ContractStatusDesc = 'Foreclosure' THEN 'Good'
    WHEN ContractStatusDesc IN ('Debt Assumption', 'Repossessed') THEN 'Neutral'
    WHEN ContractStatusDesc IN (
      'Write-off (BLW)', 'Past Due', 'Blocked by the Bank due to Credit Reasons',
      'Under dispute / non performing', 'Under litigation / Delinquent',
      'Blocked or Closed voluntary by the Customer', 'Blocked or Closed due to Restructuring',
      'There are unpaid amounts, Negotiated Settlement', 'Previous delinquency settled',
      'Write-off and Credit transferred to third party / Collection',
      'Write-off and Fully Settled', 'Blocked by the Bank due to card lost/stolen',
      'Blocked by the Bank due to fraud', 'Dispute / Litigation contested'
    ) THEN 'Bad'
    ELSE 'Unknown'
  END AS Repaymentcategory,
CASE
    WHEN ContractTypeDesc IN ('Salary loan', 'Personal Loan', 'Unsecured loan', 'Vehicle Loan', 'Mortgage/Real Estate', 'Time Loan', 'Short Term Loan', 'Benefit Loan', 'Home equity loan', 'Agricultural Loan', 'Student Loan', 'Vehicle leasing', 'Credit Card', 'Credit Card - Shared Limit', 'Credit Card - MultiCurrency', 'Revolving Credit', 'Trust Loan', 'Credit Line')
      OR (ContractTypeDesc = 'Term Loan' AND CreditPurposeDesc NOT LIKE 'Small and Medium Enterprise Loans%')
      OR (ContractTypeDesc = 'Loan Line' AND CreditPurposeDesc NOT LIKE 'Small and Medium Enterprise Loans%')
      OR (CreditPurposeDesc LIKE 'Loans to Individual%' AND ContractTypeDesc != 'Business Loan')
      OR (CreditPurposeDesc LIKE 'Microfinance Loans' AND ContractTypeDesc != 'Business Loan')
      OR (CreditPurposeDesc LIKE 'Other Agricultural Credit' AND ContractTypeDesc != 'Business Loan')
      OR (ContractHistoryType LIKE 'Installments' AND ContractTypeDesc = 'Term Loan' and CreditPurposeDesc is null)
      OR (ContractHistoryType is null AND ContractTypeDesc = 'Term Loan' and CreditPurposeDesc is null)
      OR CreditPurposeDesc IN ('Agrarian Reform', 'Development Loan Incentives - Socialized Low Cost Housing (Loans to individuals for housing purposes )')
      OR ContractHistoryType = 'CreditCards'
    THEN 'B2C'

    WHEN ContractTypeDesc IN ('Business Loan', 'Real estate leasing', 'Equipment leasing')
      OR CreditPurposeDesc IN ('Development Loan Incentives - Cooperatives', 'Development Loan Incentives - Educational Inst.', 'Loan to Government - GOCCs (Other Financial)', 'Loan to Government - GOCCs (Social Security Institutions)', 'Loan to Government - LGUs', 'Loan to Government - National Government', 'Loans to Private Corporation (Financial)', 'Loans to Private Corporation (Non-Financial)', 'Small and Medium Enterprise Loans (Medium Scale Enterprise)', 'Small and Medium Enterprise Loans (Small Scale Enterprise)')
      OR (ContractTypeDesc = 'Vehicle Loan' AND CreditPurposeDesc NOT LIKE 'Loans to Individual%')
      OR (ContractTypeDesc = 'Loan Line' AND CreditPurposeDesc LIKE 'Small and Medium Enterprise Loans%')
      OR (ContractTypeDesc = 'Term Loan' AND CreditPurposeDesc LIKE 'Small and Medium Enterprise Loans%')
    THEN 'B2B'

    ELSE 'Unknown'
  END AS BusinessType,
 CASE
    WHEN ContractTypeDesc = 'Time Loan' THEN 'Time Loans'
    WHEN ContractTypeDesc IN ('Short Term Loan', 'Term Loan') THEN 'Short and Term Loans'
    WHEN ContractTypeDesc = 'Home equity loan' THEN 'Home Equity Loans'
    WHEN ContractTypeDesc IN ('Credit Card', 'Credit Card - MultiCurrency', 'Credit Card - Shared Limit') THEN 'Credit Cards'
    WHEN ContractTypeDesc IN ('Loan Line', 'Credit Line') THEN 'Credit Lines'
    WHEN ContractTypeDesc IN ('Mortgage/Real Estate', 'Real estate leasing') THEN 'Real Estate Loans'
    WHEN ContractTypeDesc = 'Trust Loan' THEN 'Trust Loans'
    WHEN ContractTypeDesc = 'Personal Loan' THEN 'Personal Loans'
    ELSE 'Other Loans'
  END AS loan_segment
from CICBaseTable
  where COALESCE(ContractHistoryType, 'NA') in ('Installments', 'CreditCards', 'NA')
  and COALESCE(RoleDesc, 'NA') in ('Borrower', 'Co-Borrower', 'NA')
),
CICBase3Table as
(select distinct * FROM  CICBase2Table where BusinessType in ('B2C', 'Unknown')
)
-- select distinct * from CICBase3Table where (digitalLoanAccountid||crifApplicationId||run_date||CBContractCode) = 'c7948327-e6a6-46ee-96fc-66c1b3b56f93taran-2629742-80860572024-07-05302970230';
,
custtname as (SELECT distinct  cast(custId as numeric) custid, firstName, middleName, LastName FROM `prj-prod-dataplatform.dl_loans_db_raw.tdbk_loan_customer_details`
),
stepAtablebase as
(
select
(a.digitalLoanAccountid||b.crifApplicationId||b.run_date||b.CBContractCode) uniquekey,
a.digitalLoanAccountId,
a.customerId, cn.Firstname, cn.middleName, cn.LastName,
a.loanAccountNumber,
a.flagDisbursement,
a.startApplyDateTime,
a.disbursementDateTime,
a.termsAndConditionsSubmitDateTime,
a.natureofwork,
a.subIndustryDescription,
a.industryDescription,
case when a.reloan_flag = 1 and a.loantype not like 'FLEXUP'then 'Reloan'
      when a.loantype = 'FLEXUP' and a.new_loan_type = 'Flex-up' and a.reloan_flag = 0 and a.flagDisbursement = 1 then 'Flex-up'
              else a.new_loan_type end as LoanProduct, b.crifApplicationId,
       processEngineGuid, requestGuid, ContractHistoryType,
       CBContractCode, ContractEndDate, ContractPhase,
       ContractPhaseDesc, ContractStartDate, ContractStatus,
       ContractStatusDesc, ContractType, ContractTypeDesc,
       b.Currency, CurrencyDesc, LastUpdateDate, OriginalCurrency,
       OriginalCurrencyDesc, ProviderCodeEncrypted,
       ProviderContractNo, ReferenceNo, Role, RoleDesc,
       BilledAmount, BoardResolutionFlag, BoardResolutionFlagDesc,
       CancellationDate, CardReferenceCode, ChargedAmount,
       CreditLimit, CreditPurpose, CreditPurposeDesc,
       FinancedAmount, FirstPaymentDate, FlagCardUsed,
       HolderLiability, HolderLiabilityDesc, InstallmentType,
       InstallmentTypeDesc, InstallmentsNumber, LastChargeDate,
       LastPaymentAmount, LastPaymentDate, MinPaymentIndicator,
       MinPaymentIndicatorDesc, MinPaymentPercentage,
       MonthlyPaymentAmount, NextPayment, NextPaymentDate,
       b.OutstandingBalance, OutstandingBalanceUnbilled,
       OutstandingPaymentsNumber, OverallCreditLimit, OverdueDays,
       OverdueDaysDesc, OverduePaymentsAmount,
       OverduePaymentsNumber, PaymentMethod, PaymentMethodDesc,
       PaymentPeriodicity, PaymentPeriodicityDesc, PremiumCard,
       PremiumCardDesc, ReorganizedCreditCode,
       ReorganizedCreditCodeDesc, ServicesLinesNo, TimesCardUsed,
       TransactionType, TransactionTypeDesc, Utilization,
       LinkedSubject_CBSubjectCode, LinkedSubject_Name,
       LinkedSubject_Role, LinkedSubject_RoleDesc, Note_TypeDesc,
       Note_Text, Note_Type, run_date, ContractRequestDate,  SOURCE, Repaymentcategory, BusinessType, loan_segment
       , ed.AnnualMonthlyIndicator, ed.Currency, ed.DateHiredFrom, ed.DateHiredTo, ed.GrossIncome, ed.MonthlyIncome, ed.AnnualIncome, ed.OccupationDesc, ed.OccupationStatusDesc, ed.PSIC, ed.PSICDesc
FROM `risk_credit_mis.loan_master_table` a
inner join CICBase3Table b
ON a.digitalLoanAccountId = b.digitalLoanAccountId
    AND a.crifApplicationId = b.crifApplicationId
left join (select * from employementdata where rnk = 1) ed on ed.digitalLoanAccountId = a.digitalLoanAccountId
Left join custtname cn on cn.custid = a.customerId
where a.disbursementDateTime is not null
# and date_trunc(a.disbursementDateTime, day) >= '2023-01-10'
and date_trunc(a.disbursementDateTime, day) < current_date()
),
stepAtable2base as
(select *, row_number() over(partition by uniquekey order by uniquekey) rnk 
, row_number() over (partition by digitalLoanAccountId order by startApplyDateTime) loanrank
from stepAtablebase)
,
-- select * from stepAtable2base where rnk > 1;
-- select * from stepAtablebase where  (digitalLoanAccountid||crifApplicationId||run_date||CBContractCode) = 'c7948327-e6a6-46ee-96fc-66c1b3b56f93taran-2629742-80860572024-07-05302970230'
-- select (digitalLoanAccountid||crifApplicationId||run_date||CBContractCode), count((digitalLoanAccountid||crifApplicationId||run_date||CBContractCode)) from stepAtable2base group by 1 having count((digitalLoanAccountid||crifApplicationId||run_date||CBContractCode)) > 1;
base as
(select a.*, d.obsFSPD30, d.defFSPD30, d.obsFPD30, d.defFPD30
, case when date_trunc(a.disbursementDateTime, day) <= '2024-04-30' then 'Train_Validation'
         when date_trunc(a.disbursementDateTime, day) >'2024-04-30' and date_trunc(a.disbursementDateTime, day) <= '2024-06-07' then 'Test' else 'Other' end targetdataselection_SIL
, case when date_trunc(a.disbursementDateTime, day) <= '2024-03-31' then 'Train_Validation'
         when date_trunc(a.disbursementDateTime, day) >'2024-03-31' and date_trunc(a.disbursementDateTime, day) <= '2024-05-07' then 'Test' else 'Other' end targetdataselection_Quick
from stepAtable2base a
inner join
(
    SELECT
        loanAccountNumber
        , SUM(CASE WHEN obs_min_inst_def30 >= 1 THEN 1 ELSE 0 END) as obsFPD30
        , SUM(CASE WHEN min_inst_def30 = 1 THEN 1 else 0 END) as defFPD30
        , sum(case when obs_min_inst_def30>=1 then (select max(disbursedloanamount) from `risk_credit_mis.loan_master_table` where loanAccountNumber = a1.loanAccountNumber) else 0 end) as obs_fpd30_vol
        , sum(case when min_inst_def30=1 then (select max(disbursedloanamount) from `risk_credit_mis.loan_master_table` where loanAccountNumber = a1.loanAccountNumber) else 0 end) as def_fpd30_vol
        , SUM(CASE WHEN obs_min_inst_def30 >= 2 THEN 1 ELSE 0 END) as obsFSPD30
        , SUM(CASE WHEN obs_min_inst_def30 >= 2 AND (min_inst_def30 = 2 or min_inst_def30 = 1) THEN 1 else 0 END) as defFSPD30
        , SUM(CASE WHEN obs_min_inst_def30 >= 2 THEN (select max(disbursedloanamount) from `risk_credit_mis.loan_master_table` where loanAccountNumber = a1.loanAccountNumber) ELSE 0 END) as obsFSPD30_vol
        , SUM(CASE WHEN obs_min_inst_def30 >= 2 AND (min_inst_def30 = 2 or min_inst_def30 = 1) THEN (select max(disbursedloanamount) from `risk_credit_mis.loan_master_table` where loanAccountNumber = a1.loanAccountNumber) else 0 END) as defFSPD30_vol
        , SUM(CASE WHEN obs_min_inst_def30 >= 3 THEN 1 ELSE 0 END) as obsFSTPD30
        , SUM(CASE WHEN obs_min_inst_def30 >= 3 AND (min_inst_def30 = 3 or min_inst_def30 = 2 or min_inst_def30 = 1) THEN 1 else 0 END) as defFSTPD30
        , SUM(CASE WHEN obs_min_inst_def30 >= 3 THEN (select max(disbursedloanamount) from `risk_credit_mis.loan_master_table` where loanAccountNumber = a1.loanAccountNumber) ELSE 0 END) as obsFSTPD30_vol
        , SUM(CASE WHEN obs_min_inst_def30 >= 3 AND (min_inst_def30 = 3 or min_inst_def30 = 2 or min_inst_def30 = 1) THEN (select max(disbursedloanamount) from `risk_credit_mis.loan_master_table` where loanAccountNumber = a1.loanAccountNumber) else 0 END) as defFSTPD30_vol
    FROM `risk_credit_mis.loan_deliquency_data` a1
    GROUP BY 1
  ) d
ON a.loanAccountNumber = d.loanAccountNumber
where
a.rnk = 1
# and a.LoanProduct in ('Quick', 'SIL-Instore')
)
select * from base
--  where digitalLoanAccountId = '42c268b9-1fe7-445c-b459-8c66d0483884'
-- select digitalLoanAccountId, count(digitalLoanAccountId) cnt from base group by 1 having count(digitalLoanAccountId) > 1
-- select uniquekey, count(uniquekey) from base group by 1 having count(uniquekey) > 1
-- and
-- uniquekey = 'c7948327-e6a6-46ee-96fc-66c1b3b56f93taran-2629742-80860572024-07-05302970230'
;
"""

cicdf = client.query(sq).to_dataframe(progress_bar_type = 'tqdm')

Job ID 39af1eba-b89f-40ce-86fb-5c87c7143b0e successfully executed: 100%|██████████|
Downloading: 100%|██████████|


In [8]:
cicdf.shape

(300009, 117)

In [9]:
cicdf.columns.values

array(['uniquekey', 'digitalLoanAccountId', 'customerId', 'Firstname',
       'middleName', 'LastName', 'loanAccountNumber', 'flagDisbursement',
       'startApplyDateTime', 'disbursementDateTime',
       'termsAndConditionsSubmitDateTime', 'natureofwork',
       'subIndustryDescription', 'industryDescription', 'LoanProduct',
       'crifApplicationId', 'processEngineGuid', 'requestGuid',
       'ContractHistoryType', 'CBContractCode', 'ContractEndDate',
       'ContractPhase', 'ContractPhaseDesc', 'ContractStartDate',
       'ContractStatus', 'ContractStatusDesc', 'ContractType',
       'ContractTypeDesc', 'Currency', 'CurrencyDesc', 'LastUpdateDate',
       'OriginalCurrency', 'OriginalCurrencyDesc',
       'ProviderCodeEncrypted', 'ProviderContractNo', 'ReferenceNo',
       'Role', 'RoleDesc', 'BilledAmount', 'BoardResolutionFlag',
       'BoardResolutionFlagDesc', 'CancellationDate', 'CardReferenceCode',
       'ChargedAmount', 'CreditLimit', 'CreditPurpose',
       'CreditPurposeD

In [10]:
pd.set_option("Display.max_columns", None)
cicdf.head()

,uniquekey,digitalLoanAccountId,customerId,Firstname,middleName,LastName,loanAccountNumber,flagDisbursement,startApplyDateTime,disbursementDateTime,termsAndConditionsSubmitDateTime,natureofwork,subIndustryDescription,industryDescription,LoanProduct,crifApplicationId,processEngineGuid,requestGuid,ContractHistoryType,CBContractCode,ContractEndDate,ContractPhase,ContractPhaseDesc,ContractStartDate,ContractStatus,ContractStatusDesc,ContractType,ContractTypeDesc,Currency,CurrencyDesc,LastUpdateDate,OriginalCurrency,OriginalCurrencyDesc,ProviderCodeEncrypted,ProviderContractNo,ReferenceNo,Role,RoleDesc,BilledAmount,BoardResolutionFlag,BoardResolutionFlagDesc,CancellationDate,CardReferenceCode,ChargedAmount,CreditLimit,CreditPurpose,CreditPurposeDesc,FinancedAmount,FirstPaymentDate,FlagCardUsed,HolderLiability,HolderLiabilityDesc,InstallmentType,InstallmentTypeDesc,InstallmentsNumber,LastChargeDate,LastPaymentAmount,LastPaymentDate,MinPaymentIndicator,MinPaymentIndicatorDesc,MinPaymentPercentage,MonthlyPaymentAmount,NextPayment,NextPaymentDate,OutstandingBalance,OutstandingBalanceUnbilled,OutstandingPaymentsNumber,OverallCreditLimit,OverdueDays,OverdueDaysDesc,OverduePaymentsAmount,OverduePaymentsNumber,PaymentMethod,PaymentMethodDesc,PaymentPeriodicity,PaymentPeriodicityDesc,PremiumCard,PremiumCardDesc,ReorganizedCreditCode,ReorganizedCreditCodeDesc,ServicesLinesNo,TimesCardUsed,TransactionType,TransactionTypeDesc,Utilization,LinkedSubject_CBSubjectCode,LinkedSubject_Name,LinkedSubject_Role,LinkedSubject_RoleDesc,Note_TypeDesc,Note_Text,Note_Type,run_date,ContractRequestDate,SOURCE,Repaymentcategory,BusinessType,loan_segment,AnnualMonthlyIndicator,Currency_1,DateHiredFrom,DateHiredTo,GrossIncome,MonthlyIncome,AnnualIncome,OccupationDesc,OccupationStatusDesc,PSIC,PSICDesc,rnk,loanrank,obsFSPD30,defFSPD30,obsFPD30,defFPD30,targetdataselection_SIL,targetdataselection_Quick
0,008a74eb-d9e9-4150-94ce-39c7d1333dfdtonik-bnpl...,008a74eb-d9e9-4150-94ce-39c7d1333dfd,2237980,ERICKA NINA,AQUINO,TIONGSON,60822379800014,1,2023-09-18 08:18:14,2023-09-18 08:27:30,2023-09-18 08:21:05,Call Center Agent/Tele Marketer,Call Centre / BPO,Services,SIL-Instore,tonik-bnpl-346172,42fa6180-55b9-11ee-a41d-0242ace60015,42ce6f80-55b9-11ee-be29-0242ace60008,Installments,C04834847,2022-01-14,CL,Closed,2021-12-15,None,None,12,Personal Loan,PHP,Philippine peso,2022-01-31,PHP,Philippine peso,PF003,None,1,B,Borrower,None,None,None,None,None,None,None,31,Loans to Individual for Consumption Purposes -...,95,2022-01-14,None,None,None,None,None,1,None,99,2022-01-14,None,None,None,99,None,None,0,None,0,None,0,Paid as agreed / Current,0,0,OTH,Other,M,monthly installments-30 days,None,None,None,None,None,None,NA,NOT APPLICABLE,None,None,None,None,None,None,None,None,2023-09-18,None,granted,Good,B2C,Personal Loans,None,None,None,None,None,<NA>,<NA>,None,None,None,None,1,1,1,0,1,0,Train_Validation,Train_Validation
1,008a74eb-d9e9-4150-94ce-39c7d1333dfdtonik-bnpl...,008a74eb-d9e9-4150-94ce-39c7d1333dfd,2237980,ERICKA NINA,AQUINO,TIONGSON,60822379800014,1,2023-09-18 08:18:14,2023-09-18 08:27:30,2023-09-18 08:21:05,Call Center Agent/Tele Marketer,Call Centre / BPO,Services,SIL-Instore,tonik-bnpl-346172,42fa6180-55b9-11ee-a41d-0242ace60015,42ce6f80-55b9-11ee-be29-0242ace60008,Installments,T03778872,2022-03-15,CL,Closed,2022-01-02,None,None,12,Personal Loan,PHP,Philippine peso,2022-03-31,PHP,Philippine peso,PF003,None,1,B,Borrower,None,None,None,None,None,None,None,31,Loans to Individual for Consumption Purposes -...,426,2022-01-14,None,None,None,None,None,3,None,159,2022-03-15,None,None,None,159,None,None,0,None,0,None,0,Paid as agreed / Current,0,0,OTH,Other,M,monthly installments-30 days,None,None,None,None,None,None,NA,NOT APPLICABLE,None,None,None,None,None,None,None,None,2023-09-18,None,granted,Good,B2C,Personal Loans,None,None,None,None,None,<NA>,<NA>,None,None,None,None,1,2,1,0,1,0,Train_Validation,Train_Validation
2,008a74eb-d9e9-4150-94ce-39c7d1333dfdtonik-bnpl...,008a74

In [11]:
cicdf[cicdf['digitalLoanAccountId'] =='000a6b2d-c9db-4d5a-9ef2-0f37987301f3']

,uniquekey,digitalLoanAccountId,customerId,Firstname,middleName,LastName,loanAccountNumber,flagDisbursement,startApplyDateTime,disbursementDateTime,termsAndConditionsSubmitDateTime,natureofwork,subIndustryDescription,industryDescription,LoanProduct,crifApplicationId,processEngineGuid,requestGuid,ContractHistoryType,CBContractCode,ContractEndDate,ContractPhase,ContractPhaseDesc,ContractStartDate,ContractStatus,ContractStatusDesc,ContractType,ContractTypeDesc,Currency,CurrencyDesc,LastUpdateDate,OriginalCurrency,OriginalCurrencyDesc,ProviderCodeEncrypted,ProviderContractNo,ReferenceNo,Role,RoleDesc,BilledAmount,BoardResolutionFlag,BoardResolutionFlagDesc,CancellationDate,CardReferenceCode,ChargedAmount,CreditLimit,CreditPurpose,CreditPurposeDesc,FinancedAmount,FirstPaymentDate,FlagCardUsed,HolderLiability,HolderLiabilityDesc,InstallmentType,InstallmentTypeDesc,InstallmentsNumber,LastChargeDate,LastPaymentAmount,LastPaymentDate,MinPaymentIndicator,MinPaymentIndicatorDesc,MinPaymentPercentage,MonthlyPaymentAmount,NextPayment,NextPaymentDate,OutstandingBalance,OutstandingBalanceUnbilled,OutstandingPaymentsNumber,OverallCreditLimit,OverdueDays,OverdueDaysDesc,OverduePaymentsAmount,OverduePaymentsNumber,PaymentMethod,PaymentMethodDesc,PaymentPeriodicity,PaymentPeriodicityDesc,PremiumCard,PremiumCardDesc,ReorganizedCreditCode,ReorganizedCreditCodeDesc,ServicesLinesNo,TimesCardUsed,TransactionType,TransactionTypeDesc,Utilization,LinkedSubject_CBSubjectCode,LinkedSubject_Name,LinkedSubject_Role,LinkedSubject_RoleDesc,Note_TypeDesc,Note_Text,Note_Type,run_date,ContractRequestDate,SOURCE,Repaymentcategory,BusinessType,loan_segment,AnnualMonthlyIndicator,Currency_1,DateHiredFrom,DateHiredTo,GrossIncome,MonthlyIncome,AnnualIncome,OccupationDesc,OccupationStatusDesc,PSIC,PSICDesc,rnk,loanrank,obsFSPD30,defFSPD30,obsFPD30,defFPD30,targetdataselection_SIL,targetdataselection_Quick
275745,000a6b2d-c9db-4d5a-9ef2-0f37987301f3taran-2626...,000a6b2d-c9db-4d5a-9ef2-0f37987301f3,2626895,VANESANIN KRISKA,PAGTALUNAN,NAVAREZ,60826268950013,1,2024-07-02 09:21:24,2024-07-02 09:34:25,2024-07-02 09:30:10,Utility Personnel/Household Help,Manpower / Manning Agencies(Security Services;...,Services,SIL-Instore,taran-2626895-7913677,9be21cf75100ec9fe03b3f154c42f1fe0bfc7364,baa577e6-56bb-4cab-91b7-774e1d3f4caa,Installments,R05143851,2023-06-28,AC,Active,2023-06-28,None,None,18,Unsecured loan,PHP,Philippine peso,2023-09-30,PHP,Philippine peso,MF002,None,1,B,Borrower,None,None,None,None,None,None,None,None,None,5000,None,None,None,None,None,None,24,None,234,None,None,None,None,234,None,None,2553,None,12,None,0,Paid as agreed / Current,0,0,None,None,W,weekly installments-7 days,None,None,None,None,None,None,NA,NOT APPLICABLE,None,None,None,None,None,None,None,None,2024-07-05,None,granted,Neutral,B2C,Other Loans,None,None,None,None,None,<NA>,<NA>,None,None,None,None,1,1,0,0,0,0,Other,Other
275746,000a6b2d-c9db-4d5a-9ef2-0f37987301f3taran-2626...,000a6b2d-c9db-4d5a-9ef2-0f37987301f3,2626895,VANESANIN KRISKA,PAGTALUNAN,NAVAREZ,60826268950013,1,2024-07-02 09:21:24,2024-07-02 09:34:25,2024-07-02 09:30:10,Utility Personnel/Household Help,Manpower / Manning Agencies(Security Services;...,Services,SIL-Instore,taran-2626895-7913677,9be21cf75100ec9fe03b3f154c42f1fe0bfc7364,baa577e6-56bb-4cab-91b7-774e1d3f4caa,Installments,C06220675,2023-12-13,AC,Active,2023-06-28,None,None,18,Unsecured loan,PHP,Philippine peso,2023-11-30,PHP,Philippine peso,MF002,None,1,B,Borrower,None,None,None,None,None,None,None,None,None,5000,None,None,None,None,None,None,24,None,234,None,None,None,None,852,None,None,447,None,2,None,0,Paid as agreed / Current,0,0,None,None,W,weekly installments-7 days,None,None,None,None,None,None,NA,NOT APPLICABLE,None,None,None,None,None,None,None,None,2024-07-02,None,granted,Neutral,B2C,Other Loans,None,None,None,None,None,<NA>,<NA>,None,None,None,None,1,2,0,0,0,0,Other,Other
275747,000a6b2d-c9db-4d5a-9ef2-0f37987301f3taran-2626...,000a6b2d-c9db-4d5a-9ef2-0f37987301f3

In [12]:
# The minimum date found for the application in above dataset cicdf

cicdf['termsAndConditionsSubmitDateTime'].min()

Timestamp('2022-10-11 15:14:52')

In [13]:
# The Unique Products 
cicdf['LoanProduct'].unique()

array(['SIL-Instore', 'Quick', 'Flex', 'ACL TSA', 'Big Loan'],
      dtype=object)

In [14]:
cicdf.to_csv(r"cicdatawithoutcicsummary.csv", index = False)

In [15]:
sq = """
select * from prj-prod-dataplatform.risk_credit_cic_data.cic_summary
;"""

dfd = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The rows and columns of cicsummary are:\t {dfd.shape}")

Job ID 0f05862c-ea93-4c71-854b-7ca41304cebf successfully executed: |          | 
Downloading: 100%|██████████|
The rows and columns of cicsummary are:	 (471393, 93)


In [16]:
# Merge both the data based on digitalLoanAccountId
dfd = dfd[['digitalLoanAccountId','DescisionValue', 'exclusionRuleLabelDescription', 'ScoreRaw', 'ScoreRange',
       'Accounts30Days', 'Accounts30DaysOther', 'Accounts60Days',
       'Accounts60DaysOther', 'Accounts90Days', 'Accounts90DaysOther',
       'AgeAtLoanTermination', 'AgeAtLoanTerminationMax',
       'CreditAvgCreditLimit', 'CreditHighestCreditLimit',
       'CreditMaxOutstandingBalance', 'CreditMaxOutstandingBalanceDate',
       'CreditMaxOverdueAmount', 'CreditMaxOverdueAmountDate',
       'CreditMaxOverdueDays', 'CreditMaxOverdueDaysDate',
       'CreditNumberOfContracts', 'CreditTotalCreditLimit',
       'CreditTotalOutstandingBalanceAmount', 'CreditTotalOverdueAmount', 'InstMaxOverdueAmount', 'InstMaxOverdueAmountDate',
       'InstMaxOverdueDays', 'InstMaxOverdueDaysDate', 'InstNumberOfContracts',
       'InstTotalMonthlyPaymentsAmount', 'InstTotalOutstandingBalanceAmount',
       'InstTotalOverdueAmount', 'LoanToValue', 'MonthsOfBooks',
       'MonthsOfBooksOther', 'NonInstAvgCreditLimit',
       'NonInstHighestCreditLimit', 'NonInstNumberOfContracts',
       'NonInstTotalCreditLimit', 'NonInstTotalOverdraftAmount',
       'NonInstTotalUtilization', 'NumberOfContract', 'NumberOfContractAC',
       'NumberOfContractACOther', 'NumberOfContractCLCA',
       'NumberOfContractCLCAOther', 'NumberOfContractOther',
       'NumberOfContractRF', 'NumberOfContractRFOther', 'NumberOfContractRN',
       'NumberOfContractRNOther', 'NumberOfContractRQ',
       'NumberOfContractRQOther', 'OverlimitFlag', 'OverlimitFlagOther',
       'ReportingProvidersNumber', 'ReportingProvidersNumberOther',
       'SubjectEventDate', 'SubjectInfoTypeCode', 'SubjectInfoTypeCodeDesc',
       'TotalOutstanding', 'TotalOutstandingOther', 'UtilityMaxBilledAmount',
       'UtilityMaxBilledAmountDate', 'UtilityMaxOutstandingBalance',
       'UtilityMaxOutstandingBalanceDate', 'UtilityMaxOverdueAmount',
       'UtilityMaxOverdueAmountDate', 'UtilityMaxOverdueDays',
       'UtilityMaxOverdueDaysDate', 'UtilityNumberOfContracts',
       'UtilityTotalBilledAmount', 'UtilityTotalOutstandingBalanceAmount',
       'UtilityTotalOverdueAmount', 'UtilizationRateCreditCard',
       'UtilizationRateCreditCardOther', 'UtilizationRateNonInstallment',
       'UtilizationRateNonInstallmentOther','scoreLabelDesc']].copy()

In [17]:
# Finding duplicate rows in dfd for cic summary data
duplicate_rows = dfd[dfd.duplicated()]

# To see all duplicate rows, including the first occurrence:
all_duplicates = dfd[dfd.duplicated(keep=False)]

print(f"The number of duplicate rows are:\t{len(duplicate_rows)}")

The number of duplicate rows are:	68


In [18]:
# Remove duplicates, keeping the last occurrence
df_cleaned = dfd.drop_duplicates(keep='last')

print(f"Original dataframe shape: {dfd.shape}")
print(f"Cleaned dataframe shape: {df_cleaned.shape}")

Original dataframe shape: (471393, 80)
Cleaned dataframe shape: (471325, 80)


In [19]:
counts = df_cleaned['digitalLoanAccountId'].value_counts()
duplicates = counts > 1

if duplicates.any():
    print("There are duplicates in the digitalLoanAccountId column")
else:
    print("No duplicates found in the digitalLoanAccountId column")

There are duplicates in the digitalLoanAccountId column


In [20]:
a= counts > 1
a

digitalLoanAccountId
3857cf93-8f85-436b-94fa-f1e70502cb4d     True
61d3e500-642b-4617-9f14-6a4000477747     True
cd548f90-a0ad-41ac-8a71-11faa561c3fc     True
82f2bc3a-e292-4dbb-a9fc-f70a2b475a60     True
e2b4ca61-7bfe-4213-bd65-70b5f0022b84     True
                                        ...  
1a52a11f-1dde-46da-bcb2-d38ac54a3597    False
f13f3d19-707e-4fae-ac16-88cc6228f10d    False
9a4acfb8-453b-455e-aaf6-a69652aec2e5    False
3167e7ac-051f-430a-830d-d2e9db92c653    False
625e4ec1-e4a9-4450-b954-677c11161afa    False
Name: count, Length: 471280, dtype: bool

In [21]:
df_cleaned[df_cleaned['digitalLoanAccountId']=='d994afa4-94ff-4127-aa2e-3d2fdc0aa6b6']

,digitalLoanAccountId,DescisionValue,exclusionRuleLabelDescription,ScoreRaw,ScoreRange,Accounts30Days,Accounts30DaysOther,Accounts60Days,Accounts60DaysOther,Accounts90Days,Accounts90DaysOther,AgeAtLoanTermination,AgeAtLoanTerminationMax,CreditAvgCreditLimit,CreditHighestCreditLimit,CreditMaxOutstandingBalance,CreditMaxOutstandingBalanceDate,CreditMaxOverdueAmount,CreditMaxOverdueAmountDate,CreditMaxOverdueDays,CreditMaxOverdueDaysDate,CreditNumberOfContracts,CreditTotalCreditLimit,CreditTotalOutstandingBalanceAmount,CreditTotalOverdueAmount,InstMaxOverdueAmount,InstMaxOverdueAmountDate,InstMaxOverdueDays,InstMaxOverdueDaysDate,InstNumberOfContracts,InstTotalMonthlyPaymentsAmount,InstTotalOutstandingBalanceAmount,InstTotalOverdueAmount,LoanToValue,MonthsOfBooks,MonthsOfBooksOther,NonInstAvgCreditLimit,NonInstHighestCreditLimit,NonInstNumberOfContracts,NonInstTotalCreditLimit,NonInstTotalOverdraftAmount,NonInstTotalUtilization,NumberOfContract,NumberOfContractAC,NumberOfContractACOther,NumberOfContractCLCA,NumberOfContractCLCAOther,NumberOfContractOther,NumberOfContractRF,NumberOfContractRFOther,NumberOfContractRN,NumberOfContractRNOther,NumberOfContractRQ,NumberOfContractRQOther,OverlimitFlag,OverlimitFlagOther,ReportingProvidersNumber,ReportingProvidersNumberOther,SubjectEventDate,SubjectInfoTypeCode,SubjectInfoTypeCodeDesc,TotalOutstanding,TotalOutstandingOther,UtilityMaxBilledAmount,UtilityMaxBilledAmountDate,UtilityMaxOutstandingBalance,UtilityMaxOutstandingBalanceDate,UtilityMaxOverdueAmount,UtilityMaxOverdueAmountDate,UtilityMaxOverdueDays,UtilityMaxOverdueDaysDate,UtilityNumberOfContracts,UtilityTotalBilledAmount,UtilityTotalOutstandingBalanceAmount,UtilityTotalOverdueAmount,UtilizationRateCreditCard,UtilizationRateCreditCardOther,UtilizationRateNonInstallment,UtilizationRateNonInstallmentOther,scoreLabelDesc
20677,d994afa4-94ff-4127-aa2e-3d2fdc0aa6b6,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
362434,d994afa4-94ff-4127-aa2e-3d2fdc0aa6b6,Review,The customer has been inactive from last 18 mo...,None,None,0,0,0,0,0,0,39,39,0,0,0,,0,,,,0,0,0,0,0,202208,0,202208,2,4313,52236,0,0.0,46,46,0,0,0,0,0,0,5,2,2,3,3,5,0,0,0,0,0,0,false,false,1,1,,,,52236,52236,0,,0,,0,,,,0,0,0,0,0,0,0,0,None


In [22]:
mergeddf = pd.merge(cicdf, df_cleaned, left_on='digitalLoanAccountId', right_on='digitalLoanAccountId', how = 'left')
mergeddf.head()

,uniquekey,digitalLoanAccountId,customerId,Firstname,middleName,LastName,loanAccountNumber,flagDisbursement,startApplyDateTime,disbursementDateTime,termsAndConditionsSubmitDateTime,natureofwork,subIndustryDescription,industryDescription,LoanProduct,crifApplicationId,processEngineGuid,requestGuid,ContractHistoryType,CBContractCode,ContractEndDate,ContractPhase,ContractPhaseDesc,ContractStartDate,ContractStatus,ContractStatusDesc,ContractType,ContractTypeDesc,Currency,CurrencyDesc,LastUpdateDate,OriginalCurrency,OriginalCurrencyDesc,ProviderCodeEncrypted,ProviderContractNo,ReferenceNo,Role,RoleDesc,BilledAmount,BoardResolutionFlag,BoardResolutionFlagDesc,CancellationDate,CardReferenceCode,ChargedAmount,CreditLimit,CreditPurpose,CreditPurposeDesc,FinancedAmount,FirstPaymentDate,FlagCardUsed,HolderLiability,HolderLiabilityDesc,InstallmentType,InstallmentTypeDesc,InstallmentsNumber,LastChargeDate,LastPaymentAmount,LastPaymentDate,MinPaymentIndicator,MinPaymentIndicatorDesc,MinPaymentPercentage,MonthlyPaymentAmount,NextPayment,NextPaymentDate,OutstandingBalance,OutstandingBalanceUnbilled,OutstandingPaymentsNumber,OverallCreditLimit,OverdueDays,OverdueDaysDesc,OverduePaymentsAmount,OverduePaymentsNumber,PaymentMethod,PaymentMethodDesc,PaymentPeriodicity,PaymentPeriodicityDesc,PremiumCard,PremiumCardDesc,ReorganizedCreditCode,ReorganizedCreditCodeDesc,ServicesLinesNo,TimesCardUsed,TransactionType,TransactionTypeDesc,Utilization,LinkedSubject_CBSubjectCode,LinkedSubject_Name,LinkedSubject_Role,LinkedSubject_RoleDesc,Note_TypeDesc,Note_Text,Note_Type,run_date,ContractRequestDate,SOURCE,Repaymentcategory,BusinessType,loan_segment,AnnualMonthlyIndicator,Currency_1,DateHiredFrom,DateHiredTo,GrossIncome,MonthlyIncome,AnnualIncome,OccupationDesc,OccupationStatusDesc,PSIC,PSICDesc,rnk,loanrank,obsFSPD30,defFSPD30,obsFPD30,defFPD30,targetdataselection_SIL,targetdataselection_Quick,DescisionValue,exclusionRuleLabelDescription,ScoreRaw,ScoreRange,Accounts30Days,Accounts30DaysOther,Accounts60Days,Accounts60DaysOther,Accounts90Days,Accounts90DaysOther,AgeAtLoanTermination,AgeAtLoanTerminationMax,CreditAvgCreditLimit,CreditHighestCreditLimit,CreditMaxOutstandingBalance,CreditMaxOutstandingBalanceDate,CreditMaxOverdueAmount,CreditMaxOverdueAmountDate,CreditMaxOverdueDays,CreditMaxOverdueDaysDate,CreditNumberOfContracts,CreditTotalCreditLimit,CreditTotalOutstandingBalanceAmount,CreditTotalOverdueAmount,InstMaxOverdueAmount,InstMaxOverdueAmountDate,InstMaxOverdueDays,InstMaxOverdueDaysDate,InstNumberOfContracts,InstTotalMonthlyPaymentsAmount,InstTotalOutstandingBalanceAmount,InstTotalOverdueAmount,LoanToValue,MonthsOfBooks,MonthsOfBooksOther,NonInstAvgCreditLimit,NonInstHighestCreditLimit,NonInstNumberOfContracts,NonInstTotalCreditLimit,NonInstTotalOverdraftAmount,NonInstTotalUtilization,NumberOfContract,NumberOfContractAC,NumberOfContractACOther,NumberOfContractCLCA,NumberOfContractCLCAOther,NumberOfContractOther,NumberOfContractRF,NumberOfContractRFOther,NumberOfContractRN,NumberOfContractRNOther,NumberOfContractRQ,NumberOfContractRQOther,OverlimitFlag,OverlimitFlagOther,ReportingProvidersNumber,ReportingProvidersNumberOther,SubjectEventDate,SubjectInfoTypeCode,SubjectInfoTypeCodeDesc,TotalOutstanding,TotalOutstandingOther,UtilityMaxBilledAmount,UtilityMaxBilledAmountDate,UtilityMaxOutstandingBalance,UtilityMaxOutstandingBalanceDate,UtilityMaxOverdueAmount,UtilityMaxOverdueAmountDate,UtilityMaxOverdueDays,UtilityMaxOverdueDaysDate,UtilityNumberOfContracts,UtilityTotalBilledAmount,UtilityTotalOutstandingBalanceAmount,UtilityTotalOverdueAmount,UtilizationRateCreditCard,UtilizationRateCreditCardOther,UtilizationRateNonInstallment,UtilizationRateNonInstallmentOther,scoreLabelDesc
0,008a74eb-d9e9-4150-94ce-39c7d1333dfdtonik-bnpl...,008a74eb-d9e9-4150-94ce-39c7d1333dfd,2237980,ERICKA NINA,AQUINO,TIONGSON,60822379800014,1,2023-09-18 08:18:14,2023-09-18 08:27:30,2023-09-18 08:21:05,Call Center Agent/Tele Marketer,Call Centre / BPO,Services,SIL-Instore,tonik

In [23]:
# 1. Check for duplicate rows
duplicates = mergeddf.duplicated()

# 2. Get the number of duplicate rows
num_duplicates = duplicates.sum()

print(f"Number of duplicate rows: {num_duplicates}")

# 3. View the duplicate rows
print("\nDuplicate rows:")
print(mergeddf[duplicates])

Number of duplicate rows: 0

Duplicate rows:
Empty DataFrame
Columns: [uniquekey, digitalLoanAccountId, customerId, Firstname, middleName, LastName, loanAccountNumber, flagDisbursement, startApplyDateTime, disbursementDateTime, termsAndConditionsSubmitDateTime, natureofwork, subIndustryDescription, industryDescription, LoanProduct, crifApplicationId, processEngineGuid, requestGuid, ContractHistoryType, CBContractCode, ContractEndDate, ContractPhase, ContractPhaseDesc, ContractStartDate, ContractStatus, ContractStatusDesc, ContractType, ContractTypeDesc, Currency, CurrencyDesc, LastUpdateDate, OriginalCurrency, OriginalCurrencyDesc, ProviderCodeEncrypted, ProviderContractNo, ReferenceNo, Role, RoleDesc, BilledAmount, BoardResolutionFlag, BoardResolutionFlagDesc, CancellationDate, CardReferenceCode, ChargedAmount, CreditLimit, CreditPurpose, CreditPurposeDesc, FinancedAmount, FirstPaymentDate, FlagCardUsed, HolderLiability, HolderLiabilityDesc, InstallmentType, InstallmentTypeDesc, Insta

In [24]:
counts = mergeddf['digitalLoanAccountId'].value_counts()
duplicates = counts > 1

if duplicates.any():
    print("There are duplicates in the digitalLoanAccountId column")
else:
    print("No duplicates found in the digitalLoanAccountId column")

There are duplicates in the digitalLoanAccountId column


In [25]:
a= counts > 1
a

digitalLoanAccountId
ebfb724a-2739-4e81-8b2e-b5ddcc745d83     True
88cb5738-75e6-46ea-84eb-5927f32df0aa     True
070c5bc9-cc2c-419c-a6e7-f2e44b3c91ee     True
263a6799-a1eb-43df-95bf-9b287f1804cd     True
42c268b9-1fe7-445c-b459-8c66d0483884     True
                                        ...  
ecad73f4-f84b-42b6-a8b0-bd05b87f99a5    False
ecb32da5-9e71-435e-a235-7bc612886145    False
ef26c76c-99ad-4169-8089-d477d885b1e0    False
f22a63ad-83f8-45b1-b95e-3949ce4d2216    False
2594dc44-d871-4a93-91ff-afc2e5535126    False
Name: count, Length: 53687, dtype: bool

In [26]:
mergeddf[mergeddf['digitalLoanAccountId']=='ebfb724a-2739-4e81-8b2e-b5ddcc745d83']

,uniquekey,digitalLoanAccountId,customerId,Firstname,middleName,LastName,loanAccountNumber,flagDisbursement,startApplyDateTime,disbursementDateTime,termsAndConditionsSubmitDateTime,natureofwork,subIndustryDescription,industryDescription,LoanProduct,crifApplicationId,processEngineGuid,requestGuid,ContractHistoryType,CBContractCode,ContractEndDate,ContractPhase,ContractPhaseDesc,ContractStartDate,ContractStatus,ContractStatusDesc,ContractType,ContractTypeDesc,Currency,CurrencyDesc,LastUpdateDate,OriginalCurrency,OriginalCurrencyDesc,ProviderCodeEncrypted,ProviderContractNo,ReferenceNo,Role,RoleDesc,BilledAmount,BoardResolutionFlag,BoardResolutionFlagDesc,CancellationDate,CardReferenceCode,ChargedAmount,CreditLimit,CreditPurpose,CreditPurposeDesc,FinancedAmount,FirstPaymentDate,FlagCardUsed,HolderLiability,HolderLiabilityDesc,InstallmentType,InstallmentTypeDesc,InstallmentsNumber,LastChargeDate,LastPaymentAmount,LastPaymentDate,MinPaymentIndicator,MinPaymentIndicatorDesc,MinPaymentPercentage,MonthlyPaymentAmount,NextPayment,NextPaymentDate,OutstandingBalance,OutstandingBalanceUnbilled,OutstandingPaymentsNumber,OverallCreditLimit,OverdueDays,OverdueDaysDesc,OverduePaymentsAmount,OverduePaymentsNumber,PaymentMethod,PaymentMethodDesc,PaymentPeriodicity,PaymentPeriodicityDesc,PremiumCard,PremiumCardDesc,ReorganizedCreditCode,ReorganizedCreditCodeDesc,ServicesLinesNo,TimesCardUsed,TransactionType,TransactionTypeDesc,Utilization,LinkedSubject_CBSubjectCode,LinkedSubject_Name,LinkedSubject_Role,LinkedSubject_RoleDesc,Note_TypeDesc,Note_Text,Note_Type,run_date,ContractRequestDate,SOURCE,Repaymentcategory,BusinessType,loan_segment,AnnualMonthlyIndicator,Currency_1,DateHiredFrom,DateHiredTo,GrossIncome,MonthlyIncome,AnnualIncome,OccupationDesc,OccupationStatusDesc,PSIC,PSICDesc,rnk,loanrank,obsFSPD30,defFSPD30,obsFPD30,defFPD30,targetdataselection_SIL,targetdataselection_Quick,DescisionValue,exclusionRuleLabelDescription,ScoreRaw,ScoreRange,Accounts30Days,Accounts30DaysOther,Accounts60Days,Accounts60DaysOther,Accounts90Days,Accounts90DaysOther,AgeAtLoanTermination,AgeAtLoanTerminationMax,CreditAvgCreditLimit,CreditHighestCreditLimit,CreditMaxOutstandingBalance,CreditMaxOutstandingBalanceDate,CreditMaxOverdueAmount,CreditMaxOverdueAmountDate,CreditMaxOverdueDays,CreditMaxOverdueDaysDate,CreditNumberOfContracts,CreditTotalCreditLimit,CreditTotalOutstandingBalanceAmount,CreditTotalOverdueAmount,InstMaxOverdueAmount,InstMaxOverdueAmountDate,InstMaxOverdueDays,InstMaxOverdueDaysDate,InstNumberOfContracts,InstTotalMonthlyPaymentsAmount,InstTotalOutstandingBalanceAmount,InstTotalOverdueAmount,LoanToValue,MonthsOfBooks,MonthsOfBooksOther,NonInstAvgCreditLimit,NonInstHighestCreditLimit,NonInstNumberOfContracts,NonInstTotalCreditLimit,NonInstTotalOverdraftAmount,NonInstTotalUtilization,NumberOfContract,NumberOfContractAC,NumberOfContractACOther,NumberOfContractCLCA,NumberOfContractCLCAOther,NumberOfContractOther,NumberOfContractRF,NumberOfContractRFOther,NumberOfContractRN,NumberOfContractRNOther,NumberOfContractRQ,NumberOfContractRQOther,OverlimitFlag,OverlimitFlagOther,ReportingProvidersNumber,ReportingProvidersNumberOther,SubjectEventDate,SubjectInfoTypeCode,SubjectInfoTypeCodeDesc,TotalOutstanding,TotalOutstandingOther,UtilityMaxBilledAmount,UtilityMaxBilledAmountDate,UtilityMaxOutstandingBalance,UtilityMaxOutstandingBalanceDate,UtilityMaxOverdueAmount,UtilityMaxOverdueAmountDate,UtilityMaxOverdueDays,UtilityMaxOverdueDaysDate,UtilityNumberOfContracts,UtilityTotalBilledAmount,UtilityTotalOutstandingBalanceAmount,UtilityTotalOverdueAmount,UtilizationRateCreditCard,UtilizationRateCreditCardOther,UtilizationRateNonInstallment,UtilizationRateNonInstallmentOther,scoreLabelDesc
259715,ebfb724a-2739-4e81-8b2e-b5ddcc745d83taran-1566...,ebfb724a-2739-4e81-8b2e-b5ddcc745d83,1566824,MARGARITA,YULO,PINEDA,60815668240031,1,2024-07-13 13:32:06,2024-07-13 13:54:12,2024-07-13 13:36:07,Sales/Marketing Professional,Others,Manufacturing,Quick,taran-1566824-6124581

In [27]:
data = mergeddf.copy()

In [28]:
data.columns.values

array(['uniquekey', 'digitalLoanAccountId', 'customerId', 'Firstname',
       'middleName', 'LastName', 'loanAccountNumber', 'flagDisbursement',
       'startApplyDateTime', 'disbursementDateTime',
       'termsAndConditionsSubmitDateTime', 'natureofwork',
       'subIndustryDescription', 'industryDescription', 'LoanProduct',
       'crifApplicationId', 'processEngineGuid', 'requestGuid',
       'ContractHistoryType', 'CBContractCode', 'ContractEndDate',
       'ContractPhase', 'ContractPhaseDesc', 'ContractStartDate',
       'ContractStatus', 'ContractStatusDesc', 'ContractType',
       'ContractTypeDesc', 'Currency', 'CurrencyDesc', 'LastUpdateDate',
       'OriginalCurrency', 'OriginalCurrencyDesc',
       'ProviderCodeEncrypted', 'ProviderContractNo', 'ReferenceNo',
       'Role', 'RoleDesc', 'BilledAmount', 'BoardResolutionFlag',
       'BoardResolutionFlagDesc', 'CancellationDate', 'CardReferenceCode',
       'ChargedAmount', 'CreditLimit', 'CreditPurpose',
       'CreditPurposeD

In [29]:
mergeddf.to_csv("CIC_DataPreparation_StartingData.csv", index = False)

## Filter data

In [30]:
# preprocessing some of the columns or creating some new columns
data['SanctionedAmount'] = data['CreditLimit'].fillna(data['FinancedAmount'])

data['disbursementDateTime'] = pd.to_datetime(data['disbursementDateTime']).dt.date
data['ContractStartDate'] = pd.to_datetime(data['ContractStartDate']).dt.date
data['ContractEndDate'] = pd.to_datetime(data['ContractEndDate'], errors='coerce')
data['run_date'] = pd.to_datetime(data['run_date']).dt.date
data['ContractRequestDate'] = pd.to_datetime(data['ContractRequestDate']).dt.date

data['ContractPhaseDesc'] = np.where((data['ContractEndDate'].notnull()) & (data['ContractEndDate'] < data['run_date']),
                                    'Closed',data['ContractPhaseDesc'])


data['OverdueDaysDesc'] = np.where((data['OverduePaymentsAmount'] == 0) & (data['OverduePaymentsNumber'] == 0),'Paid as agreed / Current',
                                                            data['OverdueDaysDesc'])

data['OverdueDays'] = np.where((data['OverduePaymentsAmount'] == 0) & (data['OverduePaymentsNumber'] == 0),0,  data['OverdueDays'])

## Snapshot Features

### Active Contract cnt

In [31]:
# Reading only granted loans
granted_loans = data[data['SOURCE'] == 'granted'].copy()

granted_loans.loc[:, 'disbursementDateTime'] = pd.to_datetime(granted_loans['disbursementDateTime']).dt.date
granted_loans.loc[:, 'ContractStartDate'] = pd.to_datetime(granted_loans['ContractStartDate']).dt.date


In [32]:
snapshot_granted_loans = granted_loans[granted_loans['run_date'] >= granted_loans['ContractStartDate']]
active_contract_loans = snapshot_granted_loans[snapshot_granted_loans['ContractPhaseDesc'] =='Active']

In [33]:
active_contract_loans_count_df =active_contract_loans.groupby(['digitalLoanAccountId']).agg(
                                         cnt_active_contracts=('CBContractCode', 'nunique')).reset_index()
active_contract_loans_count_df.head(5)

,digitalLoanAccountId,cnt_active_contracts
0,0000181e-f0bc-414d-bbc1-69fd841f4bc1,4
1,00029466-5f50-433b-859d-23aa3cb6d04d,1
2,000a3bee-c57d-4647-beaa-f8d3293b4101,1
3,000d27e2-705e-47e2-886d-542a43c0889b,5
4,000d5b99-ffd3-45ad-b650-32fe43a95dc7,19


In [34]:
# Check duplicate

counts = active_contract_loans_count_df['digitalLoanAccountId'].value_counts()
duplicates = counts > 1

if duplicates.any():
    print("There are duplicates in the digitalLoanAccountId column")
else:
    print("No duplicates found in the digitalLoanAccountId column")

No duplicates found in the digitalLoanAccountId column


### Active contracts by loan segment

In [35]:
active_contract_loans_by_loansegment =active_contract_loans.groupby(['digitalLoanAccountId','loan_segment']).agg(
                                         cnt_active_contracts=('CBContractCode', 'nunique')).reset_index()

pivot_active_contract_loans_by_loansegment = active_contract_loans_by_loansegment.pivot_table(index=['digitalLoanAccountId'], columns='loan_segment', values=['cnt_active_contracts'])

pivot_active_contract_loans_by_loansegment.columns = [metric+'_'+sector for metric, sector in pivot_active_contract_loans_by_loansegment.columns]
pivot_active_contract_loans_by_loansegment = pivot_active_contract_loans_by_loansegment.reset_index()

pivot_active_contract_loans_by_loansegment.head(5)

,digitalLoanAccountId,cnt_active_contracts_Credit Cards,cnt_active_contracts_Credit Lines,cnt_active_contracts_Home Equity Loans,cnt_active_contracts_Other Loans,cnt_active_contracts_Personal Loans,cnt_active_contracts_Real Estate Loans,cnt_active_contracts_Short and Term Loans,cnt_active_contracts_Time Loans,cnt_active_contracts_Trust Loans
0,0000181e-f0bc-414d-bbc1-69fd841f4bc1,3.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
1,00029466-5f50-433b-859d-23aa3cb6d04d,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,000a3bee-c57d-4647-beaa-f8d3293b4101,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
3,000d27e2-705e-47e2-886d-542a43c0889b,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,000d5b99-ffd3-45ad-b650-32fe43a95dc7,19.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
# Check duplicate

counts = pivot_active_contract_loans_by_loansegment['digitalLoanAccountId'].value_counts()
duplicates = counts > 1

if duplicates.any():
    print("There are duplicates in the digitalLoanAccountId column")
else:
    print("No duplicates found in the digitalLoanAccountId column")

No duplicates found in the digitalLoanAccountId column


### Active contract max limit (max sanctioned amt)

In [37]:
max_creditlimit_currently_activeloans  = active_contract_loans.groupby(['digitalLoanAccountId']).agg(
                                         max_amt_active_contracts=('SanctionedAmount', 'max')).reset_index()
max_creditlimit_currently_activeloans.head(5)

,digitalLoanAccountId,max_amt_active_contracts
0,0000181e-f0bc-414d-bbc1-69fd841f4bc1,462700
1,00029466-5f50-433b-859d-23aa3cb6d04d,90000
2,000a3bee-c57d-4647-beaa-f8d3293b4101,67400
3,000d27e2-705e-47e2-886d-542a43c0889b,5000
4,000d5b99-ffd3-45ad-b650-32fe43a95dc7,98000


In [38]:
# Check duplicate

counts = max_creditlimit_currently_activeloans['digitalLoanAccountId'].value_counts()
duplicates = counts > 1

if duplicates.any():
    print("There are duplicates in the digitalLoanAccountId column")
else:
    print("No duplicates found in the digitalLoanAccountId column")

No duplicates found in the digitalLoanAccountId column


### Active contract total outstanding balance

In [39]:
total_OutstandingBal_currently_activeloans  =active_contract_loans.groupby(['digitalLoanAccountId']).agg(
                                         tot_active_contracts_os_amt=('OutstandingBalance', 'sum')).reset_index()

total_OutstandingBal_currently_activeloans.head(5)

,digitalLoanAccountId,tot_active_contracts_os_amt
0,0000181e-f0bc-414d-bbc1-69fd841f4bc1,12000378573109255115474
1,00029466-5f50-433b-859d-23aa3cb6d04d,25845
2,000a3bee-c57d-4647-beaa-f8d3293b4101,44049
3,000d27e2-705e-47e2-886d-542a43c0889b,1489400373840
4,000d5b99-ffd3-45ad-b650-32fe43a95dc7,2553117599865552905146471084574598002952424118...


In [40]:
def duplicateincolumn(df, col):
    counts = df[col].value_counts()
    duplicates = counts > 1

    if duplicates.any():
        print("There are duplicates in the digitalLoanAccountId column")
    else:
        print("No duplicates found in the digitalLoanAccountId column")
    

In [41]:
duplicateincolumn(total_OutstandingBal_currently_activeloans, 'digitalLoanAccountId')

No duplicates found in the digitalLoanAccountId column


### Overall Utilization

In [42]:
total_Utilization_currently_activeloans  =active_contract_loans.groupby(['digitalLoanAccountId']).agg(
                                         tot_outstandingbal_amt=('OutstandingBalance', 'sum'),
                                          tot_Sanctioned_amt=('SanctionedAmount', 'sum')  ).reset_index()


In [43]:
# Convert columns to numeric
total_Utilization_currently_activeloans['tot_outstandingbal_amt'] = pd.to_numeric(total_Utilization_currently_activeloans['tot_outstandingbal_amt'], errors='coerce')
total_Utilization_currently_activeloans['tot_Sanctioned_amt'] = pd.to_numeric(total_Utilization_currently_activeloans['tot_Sanctioned_amt'], errors='coerce')

# Handle NaN values if necessary
total_Utilization_currently_activeloans.fillna(0, inplace=True)

# Calculate the ratio
total_Utilization_currently_activeloans['tot_active_contracts_util'] = total_Utilization_currently_activeloans['tot_outstandingbal_amt'] / total_Utilization_currently_activeloans['tot_Sanctioned_amt']

# Replace inf values
total_Utilization_currently_activeloans.replace([np.inf, -np.inf], -1, inplace=True)

# Select relevant columns
total_Utilization_currently_activeloans = total_Utilization_currently_activeloans[['digitalLoanAccountId','tot_active_contracts_util']]

# Display the head of the DataFrame
total_Utilization_currently_activeloans.head()


,digitalLoanAccountId,tot_active_contracts_util
0,0000181e-f0bc-414d-bbc1-69fd841f4bc1,2.593553e+06
1,00029466-5f50-433b-859d-23aa3cb6d04d,2.871667e-01
2,000a3bee-c57d-4647-beaa-f8d3293b4101,6.535460e-01
3,000d27e2-705e-47e2-886d-542a43c0889b,1.220815e-10
4,000d5b99-ffd3-45ad-b650-32fe43a95dc7,2.605213e-17


In [44]:
duplicateincolumn(total_Utilization_currently_activeloans, 'digitalLoanAccountId')

No duplicates found in the digitalLoanAccountId column


In [45]:
total_Utilization_currently_activeloans[total_Utilization_currently_activeloans['tot_active_contracts_util']>=2]

,digitalLoanAccountId,tot_active_contracts_util
0,0000181e-f0bc-414d-bbc1-69fd841f4bc1,2.593553e+06
22,004e0430-0f52-4db7-a12d-86c3fa10440a,2.155334e+00
112,015d072d-3d2c-4c68-a02b-38fcb46f9d64,1.741791e+01
223,029240e4-cc67-47ab-987b-a051ba44d90b,3.752271e+09
282,03269a4f-6bf1-4e45-a8c5-839839acfa3f,2.823400e+00
...,...,...
23835,fd65b6b5-d317-4347-89e6-44992a564bc5,1.245834e+03
23840,fd6d25a0-803e-4a8f-aaed-ace272b85754,2.180122e+03
24017,ffa9ce01-14f5-480d-bc75-34d5bd56328a,2.250000e+00
24025,ffb9c73d-1db9-49e9-9dbd-604ed81a71f5,1.295646e+04


In [46]:
active_contract_loans[active_contract_loans['digitalLoanAccountId'] == '004e0430-0f52-4db7-a12d-86c3fa10440a'][['OutstandingBalance','SanctionedAmount','FinancedAmount','CreditLimit']]

,OutstandingBalance,SanctionedAmount,FinancedAmount,CreditLimit
92023,165840,76944,76944,None


In [47]:
active_contract_loans[active_contract_loans['digitalLoanAccountId'] == 'e3ef5a12-38d2-4224-8d5a-ef292d56f611'][['OutstandingBalance','SanctionedAmount','FinancedAmount','CreditLimit']]

,OutstandingBalance,SanctionedAmount,FinancedAmount,CreditLimit
291071,175787,4883,4883,None


In [48]:
active_contract_loans[active_contract_loans['digitalLoanAccountId'] == 'ae577f39-28e0-4efa-8844-15b7e2dde91c'][['OutstandingBalance','SanctionedAmount','FinancedAmount','CreditLimit']] 

,OutstandingBalance,SanctionedAmount,FinancedAmount,CreditLimit
169914,15362,50000,50000,None
169917,84429,3127,3127,None


In [49]:
active_contract_loans[active_contract_loans['digitalLoanAccountId'] == '150f1383-db34-4f7c-80ee-1d5452ab7f72'][['OutstandingBalance','SanctionedAmount','FinancedAmount','CreditLimit']] 

,OutstandingBalance,SanctionedAmount,FinancedAmount,CreditLimit
272665,94468,66500,66500,None
272670,121694,0,None,0
272680,105758,0,None,0


### Overall Utilization by loan segment

In [50]:
# Assuming 'active_contract_loans' is your original DataFrame

# Aggregate the sum of 'OutstandingBalance' and 'SanctionedAmount' for each group
total_Utilization_by_seg = active_contract_loans.groupby(['digitalLoanAccountId', 'loan_segment']).agg(
    tot_outstandingbal_amt=('OutstandingBalance', 'sum'),
    tot_Sanctioned_amt=('SanctionedAmount', 'sum')
).reset_index()

# Convert the aggregated columns to numeric types to avoid division errors
total_Utilization_by_seg['tot_outstandingbal_amt'] = pd.to_numeric(total_Utilization_by_seg['tot_outstandingbal_amt'], errors='coerce')
total_Utilization_by_seg['tot_Sanctioned_amt'] = pd.to_numeric(total_Utilization_by_seg['tot_Sanctioned_amt'], errors='coerce')

# Calculate the ratio of Outstanding Balance to Sanctioned Amount
total_Utilization_by_seg['tot_active_contracts_util'] = total_Utilization_by_seg['tot_outstandingbal_amt'] / total_Utilization_by_seg['tot_Sanctioned_amt']

# Replace infinite values resulting from division by zero with -1
total_Utilization_by_seg.replace([np.inf, -np.inf], -1, inplace=True)

# Display the resulting DataFrame
total_Utilization_by_seg.head()

,digitalLoanAccountId,loan_segment,tot_outstandingbal_amt,tot_Sanctioned_amt,tot_active_contracts_util
0,0000181e-f0bc-414d-bbc1-69fd841f4bc1,Credit Cards,1.200011e+16,3.000030e+09,3.999996e+06
1,0000181e-f0bc-414d-bbc1-69fd841f4bc1,Other Loans,3.785730e+05,4.627000e+05,8.181824e-01
2,00029466-5f50-433b-859d-23aa3cb6d04d,Credit Cards,2.584500e+04,9.000000e+04,2.871667e-01
3,000a3bee-c57d-4647-beaa-f8d3293b4101,Other Loans,4.404900e+04,6.740000e+04,6.535460e-01
4,000d27e2-705e-47e2-886d-542a43c0889b,Credit Cards,1.489400e+12,1.220005e+22,1.220815e-10


In [51]:
duplicateincolumn(total_Utilization_by_seg, 'digitalLoanAccountId')

There are duplicates in the digitalLoanAccountId column


In [52]:
total_Utilization_by_seg = total_Utilization_by_seg[['digitalLoanAccountId','loan_segment','tot_active_contracts_util']]

pivot_atotal_Utilization_by_seg = total_Utilization_by_seg.pivot_table(index=['digitalLoanAccountId'], columns='loan_segment', values=['tot_active_contracts_util'])

pivot_atotal_Utilization_by_seg.columns = [segment+'_'+'active_contracts_util' for metric, segment in pivot_atotal_Utilization_by_seg.columns]
pivot_atotal_Utilization_by_seg = pivot_atotal_Utilization_by_seg.reset_index()

pivot_atotal_Utilization_by_seg.head(5)

,digitalLoanAccountId,Credit Cards_active_contracts_util,Credit Lines_active_contracts_util,Home Equity Loans_active_contracts_util,Other Loans_active_contracts_util,Personal Loans_active_contracts_util,Real Estate Loans_active_contracts_util,Short and Term Loans_active_contracts_util,Time Loans_active_contracts_util,Trust Loans_active_contracts_util
0,0000181e-f0bc-414d-bbc1-69fd841f4bc1,3.999996e+06,NaN,NaN,0.818182,NaN,NaN,NaN,NaN,NaN
1,00029466-5f50-433b-859d-23aa3cb6d04d,2.871667e-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,000a3bee-c57d-4647-beaa-f8d3293b4101,NaN,NaN,NaN,0.653546,NaN,NaN,NaN,NaN,NaN
3,000d27e2-705e-47e2-886d-542a43c0889b,1.220815e-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,000d5b99-ffd3-45ad-b650-32fe43a95dc7,2.605213e-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
duplicateincolumn(pivot_atotal_Utilization_by_seg, 'digitalLoanAccountId')

No duplicates found in the digitalLoanAccountId column


### Last repayment amount

In [54]:
# Reading only granted loans
granted_loans = data[data['SOURCE'] == 'granted'].copy()

# Converting LastPaymentDate to datetime, handling errors, and suppressing the warning
granted_loans.loc[:, 'LastPaymentDate'] = pd.to_datetime(granted_loans['LastPaymentDate'], errors='coerce')


In [55]:
# Ensure 'run_date' is in the same format as 'LastPaymentDate'
granted_loans['run_date'] = pd.to_datetime(granted_loans['run_date']).dt.date
granted_loans['LastPaymentDate'] = pd.to_datetime(granted_loans['LastPaymentDate'], errors='coerce').dt.date

# Filter rows where LastPaymentDate is less than or equal to run_date
LastPaymentDate = granted_loans[granted_loans['LastPaymentDate'] <= granted_loans['run_date']]

# Get the latest repayment date for each digitalLoanAccountId
latest_repayment_date = LastPaymentDate.groupby('digitalLoanAccountId')['LastPaymentDate'].max().reset_index()

# Merge with the original data to get the corresponding payment details
latest_repayment = latest_repayment_date.merge(
    LastPaymentDate[['digitalLoanAccountId', 'LastPaymentDate', 'LastPaymentAmount', 'PaymentMethodDesc']],
    on=['digitalLoanAccountId', 'LastPaymentDate'],
    how='left'
)

# Calculate the last repayment amount
last_repay_amt_ = latest_repayment.groupby('digitalLoanAccountId').agg(
    last_repay_amt=('LastPaymentAmount', 'sum')
).reset_index()

# Get the row with the highest payment amount for each digitalLoanAccountId
last_repay_amt = latest_repayment.loc[latest_repayment.groupby('digitalLoanAccountId')['LastPaymentAmount'].idxmax()]
last_repay_amt.rename(columns={'PaymentMethodDesc': 'highest_latest_repay_method'}, inplace=True)

# Merge the highest repayment method with the last repayment amount
last_repay_amt = pd.merge(
    last_repay_amt_, 
    last_repay_amt[['digitalLoanAccountId', 'highest_latest_repay_method']], 
    on='digitalLoanAccountId', 
    how='left'
)

# Select and display the relevant columns
last_repay_amt_all = last_repay_amt[['digitalLoanAccountId', 'last_repay_amt', 'highest_latest_repay_method']]
last_repay_amt_all.head()


,digitalLoanAccountId,last_repay_amt,highest_latest_repay_method
0,0000181e-f0bc-414d-bbc1-69fd841f4bc1,0,Other
1,00025b22-70c8-4d93-ad59-4a58f5c11d56,10490,Current Account Debit
2,00029466-5f50-433b-859d-23aa3cb6d04d,24916,Cash
3,000a3bee-c57d-4647-beaa-f8d3293b4101,4726,None
4,000b9c72-3e95-4c0c-8e4c-b5c57a0bd0ff,253,Other


In [56]:
duplicateincolumn(last_repay_amt_all, 'digitalLoanAccountId')

No duplicates found in the digitalLoanAccountId column


In [57]:
LastPaymentDate_30d = granted_loans[(granted_loans['LastPaymentDate'] >= (granted_loans['run_date'] - pd.Timedelta(days=30))) & 
                 (granted_loans['LastPaymentDate'] <= granted_loans['run_date'])]

# Get the latest repayment date per digitalLoanAccountId
latest_repayment_date_30d = LastPaymentDate_30d.groupby('digitalLoanAccountId')['LastPaymentDate'].max().reset_index()

# Merge to get the corresponding RepaymentAmount
latest_repayment_30d = latest_repayment_date_30d.merge(
                        LastPaymentDate_30d[['digitalLoanAccountId', 'LastPaymentDate', 'LastPaymentAmount','PaymentMethodDesc']], on=['digitalLoanAccountId', 'LastPaymentDate'], how='left')


latest_repayment_30d_ = latest_repayment_30d.groupby(['digitalLoanAccountId']).agg(
                                         last_repay_amt_30d=('LastPaymentAmount', 'sum') ).reset_index()

 
last_repay_amt_30d = latest_repayment_30d.loc[latest_repayment_30d.groupby('digitalLoanAccountId')['LastPaymentAmount'].idxmax()]
last_repay_amt_30d.rename(columns={'PaymentMethodDesc': 'highest_latest_repay_method_30d'}, inplace=True)


last_repay_amt_30d = pd.merge(latest_repayment_30d_, last_repay_amt_30d[['digitalLoanAccountId','highest_latest_repay_method_30d']], on='digitalLoanAccountId', how='left')

last_repay_amt_30d = last_repay_amt_30d[['digitalLoanAccountId','last_repay_amt_30d', 'highest_latest_repay_method_30d']]
last_repay_amt_30d.head()

,digitalLoanAccountId,last_repay_amt_30d,highest_latest_repay_method_30d
0,000d5b99-ffd3-45ad-b650-32fe43a95dc7,7000,Cash
1,00b0ab12-0a1a-4414-bf80-74c51f34dc9d,1398,Current Account Debit
2,00f5147b-5591-4602-af90-cbd639ad4a1e,1000,Cash
3,01a71534-8f57-431a-a160-9cee24e60588,5245,Current Account Debit
4,01c8b702-72cf-4792-8e78-713176eb46f0,0,Other


In [58]:
LastPaymentDate_60d = granted_loans[(granted_loans['LastPaymentDate'] >= (granted_loans['run_date'] - pd.Timedelta(days=60))) & 
                 (granted_loans['LastPaymentDate'] <= granted_loans['run_date'])]

# Get the latest repayment date per digitalLoanAccountId
latest_repayment_date_60d = LastPaymentDate_60d.groupby('digitalLoanAccountId')['LastPaymentDate'].max().reset_index()

# Merge to get the corresponding RepaymentAmount
latest_repayment_60d = latest_repayment_date_60d.merge(
                        LastPaymentDate_60d[['digitalLoanAccountId', 'LastPaymentDate', 'LastPaymentAmount','PaymentMethodDesc']], on=['digitalLoanAccountId', 'LastPaymentDate'], how='left')
 
latest_repayment_60d_ = latest_repayment_60d.groupby(['digitalLoanAccountId']).agg(
                                         last_repay_amt_60d=('LastPaymentAmount', 'sum') ).reset_index()
    
last_repay_amt_60d = latest_repayment_60d.loc[latest_repayment_60d.groupby('digitalLoanAccountId')['LastPaymentAmount'].idxmax()]
last_repay_amt_60d.rename(columns={'LastPaymentAmount': 'last_repay_amt_60d','PaymentMethodDesc': 'highest_latest_repay_method_60d'}, inplace=True)


last_repay_amt_60d = pd.merge(latest_repayment_60d_, last_repay_amt_60d[['digitalLoanAccountId','highest_latest_repay_method_60d']], on='digitalLoanAccountId', how='left')


last_repay_amt_60d = last_repay_amt_60d[['digitalLoanAccountId','last_repay_amt_60d', 'highest_latest_repay_method_60d']]
last_repay_amt_60d.head()

,digitalLoanAccountId,last_repay_amt_60d,highest_latest_repay_method_60d
0,00025b22-70c8-4d93-ad59-4a58f5c11d56,10490,Current Account Debit
1,000d5b99-ffd3-45ad-b650-32fe43a95dc7,7000,Cash
2,001afde5-17dd-48c2-9693-b6df1aa8e752,450,Cash
3,001c77ba-72f9-4c44-a08d-4de9db9f0539,1985,Current Account Debit
4,0025fd24-406b-475c-bf32-36f93457efc9,2253,Current Account Debit


In [59]:
LastPaymentDate_90d = granted_loans[(granted_loans['LastPaymentDate'] >= (granted_loans['run_date'] - pd.Timedelta(days=90))) & 
                 (granted_loans['LastPaymentDate'] <= granted_loans['run_date'])]

# Get the latest repayment date per digitalLoanAccountId
latest_repayment_date_90d = LastPaymentDate_90d.groupby('digitalLoanAccountId')['LastPaymentDate'].max().reset_index()

# Merge to get the corresponding RepaymentAmount
latest_repayment_90d = latest_repayment_date_90d.merge(
                        LastPaymentDate_90d[['digitalLoanAccountId', 'LastPaymentDate', 'LastPaymentAmount','PaymentMethodDesc']], on=['digitalLoanAccountId', 'LastPaymentDate'], how='left')

latest_repayment_90d_ = latest_repayment_90d.groupby(['digitalLoanAccountId']).agg(
                                         last_repay_amt_90d=('LastPaymentAmount', 'sum') ).reset_index()


last_repay_amt_90d = latest_repayment_90d.loc[latest_repayment_90d.groupby('digitalLoanAccountId')['LastPaymentAmount'].idxmax()]
last_repay_amt_90d.rename(columns={'LastPaymentAmount': 'last_repay_amt_90d','PaymentMethodDesc': 'highest_latest_repay_method_90d'}, inplace=True)

last_repay_amt_90d = pd.merge(latest_repayment_90d_, last_repay_amt_90d[['digitalLoanAccountId','highest_latest_repay_method_90d']], on='digitalLoanAccountId', how='left')


last_repay_amt_90d = last_repay_amt_90d[['digitalLoanAccountId','last_repay_amt_90d', 'highest_latest_repay_method_90d']]
last_repay_amt_90d.head()

,digitalLoanAccountId,last_repay_amt_90d,highest_latest_repay_method_90d
0,00025b22-70c8-4d93-ad59-4a58f5c11d56,10490,Current Account Debit
1,000d5b99-ffd3-45ad-b650-32fe43a95dc7,7000,Cash
2,001afde5-17dd-48c2-9693-b6df1aa8e752,450,Cash
3,001c77ba-72f9-4c44-a08d-4de9db9f0539,1985,Current Account Debit
4,0025fd24-406b-475c-bf32-36f93457efc9,2253,Current Account Debit


In [60]:
last_repay_amt = last_repay_amt_30d.merge(last_repay_amt_60d, on='digitalLoanAccountId', how ='left')
last_repay_amt = last_repay_amt.merge(last_repay_amt_90d, on='digitalLoanAccountId', how ='left')
last_repay_amt = last_repay_amt.merge(last_repay_amt_all, on='digitalLoanAccountId', how ='left')

In [61]:
last_repay_amt.head()

,digitalLoanAccountId,last_repay_amt_30d,highest_latest_repay_method_30d,last_repay_amt_60d,highest_latest_repay_method_60d,last_repay_amt_90d,highest_latest_repay_method_90d,last_repay_amt,highest_latest_repay_method
0,000d5b99-ffd3-45ad-b650-32fe43a95dc7,7000,Cash,7000,Cash,7000,Cash,7000,Cash
1,00b0ab12-0a1a-4414-bf80-74c51f34dc9d,1398,Current Account Debit,1398,Current Account Debit,1398,Current Account Debit,1398,Current Account Debit
2,00f5147b-5591-4602-af90-cbd639ad4a1e,1000,Cash,1000,Cash,1000,Cash,1000,Cash
3,01a71534-8f57-431a-a160-9cee24e60588,5245,Current Account Debit,5245,Current Account Debit,5245,Current Account Debit,5245,Current Account Debit
4,01c8b702-72cf-4792-8e78-713176eb46f0,0,Other,0,Other,0,Other,0,Other


In [62]:
duplicateincolumn(last_repay_amt, 'digitalLoanAccountId')

No duplicates found in the digitalLoanAccountId column


### max_age_active_contracts_snapshot

In [63]:
active_contract_loans.loc[:, 'ContractStartDate'] = pd.to_datetime(
    active_contract_loans['ContractStartDate'], format='%d-%m-%Y', errors='coerce'
)
active_contract_loans.loc[:, 'run_date'] = pd.to_datetime(
    active_contract_loans['run_date'], format='%d-%m-%Y', errors='coerce'
)
active_contract_loans.loc[:, 'ContractEndDate'] = pd.to_datetime(
    active_contract_loans['ContractEndDate'], format='%d-%m-%Y', errors='coerce'
)


In [66]:
# Ensure we're working with a copy of the original DataFrame
active_contract_loans = active_contract_loans.copy()

# Convert date columns to datetime format
date_columns = ['ContractEndDate', 'ContractStartDate', 'run_date']
for col in date_columns:
    active_contract_loans.loc[:, col] = pd.to_datetime(active_contract_loans[col], errors='coerce')

# Calculate the date difference
conditions = [
    active_contract_loans['ContractEndDate'].notna() & (active_contract_loans['run_date'] >= active_contract_loans['ContractEndDate']),
    active_contract_loans['ContractEndDate'].isna() | (active_contract_loans['run_date'] < active_contract_loans['ContractEndDate'])
]
choices = [
    (active_contract_loans['ContractEndDate'] - active_contract_loans['ContractStartDate']).dt.days,
    (active_contract_loans['run_date'] - active_contract_loans['ContractStartDate']).dt.days
]
active_contract_loans.loc[:, 'date_diff'] = np.select(conditions, choices, default=np.nan)

# Find the max date difference for each Loan_id
max_age_active_loans = active_contract_loans.groupby('digitalLoanAccountId').agg(
    max_age_active_contracts_snapshot=('date_diff', 'max')
).reset_index()
max_age_active_loans.head()

,digitalLoanAccountId,max_age_active_contracts_snapshot
0,0000181e-f0bc-414d-bbc1-69fd841f4bc1,2314.0
1,00029466-5f50-433b-859d-23aa3cb6d04d,2016.0
2,000a3bee-c57d-4647-beaa-f8d3293b4101,524.0
3,000d27e2-705e-47e2-886d-542a43c0889b,3165.0
4,000d5b99-ffd3-45ad-b650-32fe43a95dc7,2710.0


In [67]:
max_age_active_loans.head()

,digitalLoanAccountId,max_age_active_contracts_snapshot
0,0000181e-f0bc-414d-bbc1-69fd841f4bc1,2314.0
1,00029466-5f50-433b-859d-23aa3cb6d04d,2016.0
2,000a3bee-c57d-4647-beaa-f8d3293b4101,524.0
3,000d27e2-705e-47e2-886d-542a43c0889b,3165.0
4,000d5b99-ffd3-45ad-b650-32fe43a95dc7,2710.0


In [68]:
duplicateincolumn(max_age_active_loans, 'digitalLoanAccountId')

No duplicates found in the digitalLoanAccountId column


### max_age_all_contracts_snapshot

In [69]:
snapshot_granted_loans.loc[:,'ContractStartDate'] = pd.to_datetime(snapshot_granted_loans['ContractStartDate'], format='%d-%m-%Y', errors='coerce')
snapshot_granted_loans.loc[:,'run_date'] = pd.to_datetime(snapshot_granted_loans['run_date'], format='%d-%m-%Y', errors='coerce')
snapshot_granted_loans.loc[:, 'ContractEndDate'] = pd.to_datetime(snapshot_granted_loans['ContractEndDate'], format='%d-%m-%Y', errors='coerce')

In [70]:
# Ensure datetime conversion for the relevant columns
snapshot_granted_loans['ContractEndDate'] = pd.to_datetime(snapshot_granted_loans['ContractEndDate'], errors='coerce')
snapshot_granted_loans['ContractStartDate'] = pd.to_datetime(snapshot_granted_loans['ContractStartDate'], errors='coerce')
snapshot_granted_loans['run_date'] = pd.to_datetime(snapshot_granted_loans['run_date'], errors='coerce')

# Calculate the date difference based on the conditions
snapshot_granted_loans['date_diff'] = np.where(
    snapshot_granted_loans['ContractEndDate'].notna() & 
    (snapshot_granted_loans['run_date'] >= snapshot_granted_loans['ContractEndDate']),
    (snapshot_granted_loans['ContractEndDate'] - snapshot_granted_loans['ContractStartDate']).dt.days,
    (snapshot_granted_loans['run_date'] - snapshot_granted_loans['ContractStartDate']).dt.days
)

# Preview the result
snapshot_granted_loans.head()


,uniquekey,digitalLoanAccountId,customerId,Firstname,middleName,LastName,loanAccountNumber,flagDisbursement,startApplyDateTime,disbursementDateTime,termsAndConditionsSubmitDateTime,natureofwork,subIndustryDescription,industryDescription,LoanProduct,crifApplicationId,processEngineGuid,requestGuid,ContractHistoryType,CBContractCode,ContractEndDate,ContractPhase,ContractPhaseDesc,ContractStartDate,ContractStatus,ContractStatusDesc,ContractType,ContractTypeDesc,Currency,CurrencyDesc,LastUpdateDate,OriginalCurrency,OriginalCurrencyDesc,ProviderCodeEncrypted,ProviderContractNo,ReferenceNo,Role,RoleDesc,BilledAmount,BoardResolutionFlag,BoardResolutionFlagDesc,CancellationDate,CardReferenceCode,ChargedAmount,CreditLimit,CreditPurpose,CreditPurposeDesc,FinancedAmount,FirstPaymentDate,FlagCardUsed,HolderLiability,HolderLiabilityDesc,InstallmentType,InstallmentTypeDesc,InstallmentsNumber,LastChargeDate,LastPaymentAmount,LastPaymentDate,MinPaymentIndicator,MinPaymentIndicatorDesc,MinPaymentPercentage,MonthlyPaymentAmount,NextPayment,NextPaymentDate,OutstandingBalance,OutstandingBalanceUnbilled,OutstandingPaymentsNumber,OverallCreditLimit,OverdueDays,OverdueDaysDesc,OverduePaymentsAmount,OverduePaymentsNumber,PaymentMethod,PaymentMethodDesc,PaymentPeriodicity,PaymentPeriodicityDesc,PremiumCard,PremiumCardDesc,ReorganizedCreditCode,ReorganizedCreditCodeDesc,ServicesLinesNo,TimesCardUsed,TransactionType,TransactionTypeDesc,Utilization,LinkedSubject_CBSubjectCode,LinkedSubject_Name,LinkedSubject_Role,LinkedSubject_RoleDesc,Note_TypeDesc,Note_Text,Note_Type,run_date,ContractRequestDate,SOURCE,Repaymentcategory,BusinessType,loan_segment,AnnualMonthlyIndicator,Currency_1,DateHiredFrom,DateHiredTo,GrossIncome,MonthlyIncome,AnnualIncome,OccupationDesc,OccupationStatusDesc,PSIC,PSICDesc,rnk,loanrank,obsFSPD30,defFSPD30,obsFPD30,defFPD30,targetdataselection_SIL,targetdataselection_Quick,DescisionValue,exclusionRuleLabelDescription,ScoreRaw,ScoreRange,Accounts30Days,Accounts30DaysOther,Accounts60Days,Accounts60DaysOther,Accounts90Days,Accounts90DaysOther,AgeAtLoanTermination,AgeAtLoanTerminationMax,CreditAvgCreditLimit,CreditHighestCreditLimit,CreditMaxOutstandingBalance,CreditMaxOutstandingBalanceDate,CreditMaxOverdueAmount,CreditMaxOverdueAmountDate,CreditMaxOverdueDays,CreditMaxOverdueDaysDate,CreditNumberOfContracts,CreditTotalCreditLimit,CreditTotalOutstandingBalanceAmount,CreditTotalOverdueAmount,InstMaxOverdueAmount,InstMaxOverdueAmountDate,InstMaxOverdueDays,InstMaxOverdueDaysDate,InstNumberOfContracts,InstTotalMonthlyPaymentsAmount,InstTotalOutstandingBalanceAmount,InstTotalOverdueAmount,LoanToValue,MonthsOfBooks,MonthsOfBooksOther,NonInstAvgCreditLimit,NonInstHighestCreditLimit,NonInstNumberOfContracts,NonInstTotalCreditLimit,NonInstTotalOverdraftAmount,NonInstTotalUtilization,NumberOfContract,NumberOfContractAC,NumberOfContractACOther,NumberOfContractCLCA,NumberOfContractCLCAOther,NumberOfContractOther,NumberOfContractRF,NumberOfContractRFOther,NumberOfContractRN,NumberOfContractRNOther,NumberOfContractRQ,NumberOfContractRQOther,OverlimitFlag,OverlimitFlagOther,ReportingProvidersNumber,ReportingProvidersNumberOther,SubjectEventDate,SubjectInfoTypeCode,SubjectInfoTypeCodeDesc,TotalOutstanding,TotalOutstandingOther,UtilityMaxBilledAmount,UtilityMaxBilledAmountDate,UtilityMaxOutstandingBalance,UtilityMaxOutstandingBalanceDate,UtilityMaxOverdueAmount,UtilityMaxOverdueAmountDate,UtilityMaxOverdueDays,UtilityMaxOverdueDaysDate,UtilityNumberOfContracts,UtilityTotalBilledAmount,UtilityTotalOutstandingBalanceAmount,UtilityTotalOverdueAmount,UtilizationRateCreditCard,UtilizationRateCreditCardOther,UtilizationRateNonInstallment,UtilizationRateNonInstallmentOther,scoreLabelDesc,SanctionedAmount,date_diff
0,008a74eb-d9e9-4150-94ce-39c7d1333dfdtonik-bnpl...,008a74eb-d9e9-4150-94ce-39c7d1333dfd,2237980,ERICKA NINA,AQUINO,TIONGSON,60822379800014,1,2023-09-18 08:18:14,2023-09-18,2023-09-18 08:21:05,Call Center Agent/Tele Marketer,Call Centre / BPO,Services

In [71]:
# Find the max date difference for each Loan_id
max_age_granted_loans = snapshot_granted_loans.groupby('digitalLoanAccountId').agg(
                                        max_age_all_contracts_snapshot=('date_diff', 'max')).reset_index()

max_age_granted_loans.head()

,digitalLoanAccountId,max_age_all_contracts_snapshot
0,0000181e-f0bc-414d-bbc1-69fd841f4bc1,2314.0
1,00025b22-70c8-4d93-ad59-4a58f5c11d56,586.0
2,00029466-5f50-433b-859d-23aa3cb6d04d,2016.0
3,00051867-ce3d-4514-a9f3-b1c2f85ae4ed,171.0
4,000a3bee-c57d-4647-beaa-f8d3293b4101,524.0


In [72]:
duplicateincolumn(max_age_granted_loans, 'digitalLoanAccountId')

No duplicates found in the digitalLoanAccountId column


### Days since last enquiry granted and non granted

In [73]:
data['run_date'] = pd.to_datetime(data['run_date'], errors='coerce')
data['ContractStartDate'] = pd.to_datetime(data['ContractStartDate'], errors='coerce')
data['ContractRequestDate'] = pd.to_datetime(data['ContractRequestDate'], errors='coerce')

data['no_of_last_enquiry_days'] = np.where(data['ContractStartDate'].isna(),
                                (data['run_date'] - data['ContractRequestDate']).dt.days,
                                (data['run_date'] - data['ContractStartDate']).dt.days)

In [74]:
last_enquiry_days = data.groupby('digitalLoanAccountId').agg(
                                        days_since_last_inquiry=('no_of_last_enquiry_days', 'min')).reset_index()
last_enquiry_days.head()

,digitalLoanAccountId,days_since_last_inquiry
0,0000181e-f0bc-414d-bbc1-69fd841f4bc1,273.0
1,00025b22-70c8-4d93-ad59-4a58f5c11d56,174.0
2,00029466-5f50-433b-859d-23aa3cb6d04d,2016.0
3,0003edd4-5a8c-4e00-bfeb-3fffc587d38b,2.0
4,00051867-ce3d-4514-a9f3-b1c2f85ae4ed,1534.0


In [75]:
duplicateincolumn(last_enquiry_days, 'digitalLoanAccountId')

No duplicates found in the digitalLoanAccountId column


### Days since last grant date

In [76]:
# Find the latest 'ContractStartDate' for each 'digitalLoanAccountId'
latest_grant_dates = snapshot_granted_loans.groupby(['digitalLoanAccountId','run_date'])['ContractStartDate'].max().reset_index()

latest_grant_dates['days_since_last_grant'] = (latest_grant_dates['run_date'] - latest_grant_dates['ContractStartDate']).dt.days
latest_grant_dates = latest_grant_dates[['digitalLoanAccountId','days_since_last_grant']]
latest_grant_dates.head()

,digitalLoanAccountId,days_since_last_grant
0,0000181e-f0bc-414d-bbc1-69fd841f4bc1,273
1,00025b22-70c8-4d93-ad59-4a58f5c11d56,201
2,00029466-5f50-433b-859d-23aa3cb6d04d,2016
3,00051867-ce3d-4514-a9f3-b1c2f85ae4ed,1534
4,000a3bee-c57d-4647-beaa-f8d3293b4101,524


In [77]:
duplicateincolumn(latest_grant_dates, 'digitalLoanAccountId')

There are duplicates in the digitalLoanAccountId column


### Days since last closed date

In [78]:
# Filter the DataFrame to include relevant rows
closed_contract_loans = snapshot_granted_loans[snapshot_granted_loans['ContractPhaseDesc'].isin(['Closed', 'Closed in advance'])]

# Find the latest 'ContractEndDate' for each 'digitalLoanAccountId'
latest_closed_dates = closed_contract_loans.groupby(['digitalLoanAccountId','run_date'])['ContractEndDate'].max().reset_index()

latest_closed_dates['days_since_last_closed'] = (latest_closed_dates['run_date'] - latest_closed_dates['ContractEndDate']).dt.days
latest_closed_dates = latest_closed_dates[['digitalLoanAccountId','days_since_last_closed']]
latest_closed_dates.head(3)

,digitalLoanAccountId,days_since_last_closed
0,0000181e-f0bc-414d-bbc1-69fd841f4bc1,93.0
1,00025b22-70c8-4d93-ad59-4a58f5c11d56,2.0
2,00029466-5f50-433b-859d-23aa3cb6d04d,686.0


### Days since last repayment date

In [79]:
snapshot_granted_loans['LastPaymentDate'] = pd.to_datetime(snapshot_granted_loans['LastPaymentDate'])

latest_repay_dates = snapshot_granted_loans.groupby(['digitalLoanAccountId','run_date'])['LastPaymentDate'].max().reset_index()
latest_repay_dates['days_since_last_repay'] = (latest_repay_dates['run_date'] - latest_repay_dates['LastPaymentDate']).dt.days
latest_repay_dates = latest_repay_dates[['digitalLoanAccountId','days_since_last_repay']]
latest_repay_dates.head(3)

,digitalLoanAccountId,days_since_last_repay
0,0000181e-f0bc-414d-bbc1-69fd841f4bc1,476.0
1,00025b22-70c8-4d93-ad59-4a58f5c11d56,33.0
2,00029466-5f50-433b-859d-23aa3cb6d04d,735.0


### Days since rejection date

In [80]:
# Filter the DataFrame for refused contract loans
refused_contract_loans = data[data['ContractPhaseDesc'] == 'Refused'].copy()

# Convert 'ContractRequestDate' and 'run_date' to datetime using .loc to avoid SettingWithCopyWarning
refused_contract_loans.loc[:, 'ContractRequestDate'] = pd.to_datetime(refused_contract_loans['ContractRequestDate'], errors='coerce')
refused_contract_loans.loc[:, 'run_date'] = pd.to_datetime(refused_contract_loans['run_date'], errors='coerce')

# Find the latest 'ContractRequestDate' for each 'digitalLoanAccountId' and 'run_date'
latest_reject_dates = refused_contract_loans.groupby(['digitalLoanAccountId', 'run_date'])['ContractRequestDate'].max().reset_index()

# Calculate the number of days since the last rejection
latest_reject_dates['days_since_last_reject'] = (latest_reject_dates['run_date'] - latest_reject_dates['ContractRequestDate']).dt.days

# Select relevant columns
latest_reject_dates = latest_reject_dates[['digitalLoanAccountId', 'days_since_last_reject']]

# Display the first few rows of the result
latest_reject_dates.head()

,digitalLoanAccountId,days_since_last_reject
0,0065031d-463d-43ec-802b-082390a75bff,NaN
1,015654c8-95a6-4e43-98ee-193d5c04c1a2,19.0
2,016d2e0c-192d-4c7c-90d8-08be55cc31a3,636.0
3,01fdd27f-2809-487e-b870-c81f4e5d796d,287.0
4,0208eeba-f264-43c5-8b43-4d1155530910,3135.0


### Days since renounce date

In [81]:
# Filter the DataFrame and convert the dates
renounced_contract_loans = data[data['ContractPhaseDesc'] == 'Renounced'].copy()

# Convert date columns to datetime format
renounced_contract_loans.loc[:, 'ContractRequestDate'] = pd.to_datetime(renounced_contract_loans['ContractRequestDate'], errors='coerce')
renounced_contract_loans.loc[:, 'run_date'] = pd.to_datetime(renounced_contract_loans['run_date'], errors='coerce')

# Find the latest 'ContractRequestDate' for each 'digitalLoanAccountId'
latest_renounced_dates = renounced_contract_loans.groupby(['digitalLoanAccountId', 'run_date'])['ContractRequestDate'].max().reset_index()

# Calculate days since the last renounce
latest_renounced_dates['days_since_last_renounce'] = (latest_renounced_dates['run_date'] - latest_renounced_dates['ContractRequestDate']).dt.days

# Select only the required columns
latest_renounced_dates = latest_renounced_dates[['digitalLoanAccountId', 'days_since_last_renounce']]

# Display the result
latest_renounced_dates.head()


,digitalLoanAccountId,days_since_last_renounce
0,0023b554-2604-4278-8f78-cf09bb8405bf,2241
1,0026991f-ce1f-431f-bf64-5c06c85c40e8,1038
2,00f71065-c692-48ec-8bb4-a9d8fb527d1f,2088
3,013f7c44-aa3e-48bb-9abe-2dd82d52c7d9,2158
4,0170778c-ad9b-4d99-8eb4-d28e95be9ef5,2558


In [82]:
snapshot_feature_df = active_contract_loans_count_df.merge(pivot_active_contract_loans_by_loansegment, on ='digitalLoanAccountId', how ='outer')
snapshot_feature_df = snapshot_feature_df.merge(max_creditlimit_currently_activeloans, on ='digitalLoanAccountId', how ='outer')
snapshot_feature_df = snapshot_feature_df.merge(total_OutstandingBal_currently_activeloans, on ='digitalLoanAccountId', how ='outer')
snapshot_feature_df = snapshot_feature_df.merge(total_Utilization_currently_activeloans, on ='digitalLoanAccountId', how ='outer')
snapshot_feature_df = snapshot_feature_df.merge(pivot_atotal_Utilization_by_seg, on ='digitalLoanAccountId', how ='outer')
snapshot_feature_df = snapshot_feature_df.merge(last_repay_amt, on ='digitalLoanAccountId', how ='outer')
snapshot_feature_df = snapshot_feature_df.merge(max_age_active_loans, on ='digitalLoanAccountId', how ='outer')
snapshot_feature_df = snapshot_feature_df.merge(max_age_granted_loans, on ='digitalLoanAccountId', how ='outer')
snapshot_feature_df = snapshot_feature_df.merge(last_enquiry_days, on ='digitalLoanAccountId', how ='outer')
snapshot_feature_df = snapshot_feature_df.merge(latest_grant_dates, on ='digitalLoanAccountId', how ='outer')
snapshot_feature_df = snapshot_feature_df.merge(latest_closed_dates, on ='digitalLoanAccountId', how ='outer')
snapshot_feature_df = snapshot_feature_df.merge(latest_repay_dates, on ='digitalLoanAccountId', how ='outer')
snapshot_feature_df = snapshot_feature_df.merge(latest_reject_dates, on ='digitalLoanAccountId', how ='outer')
snapshot_feature_df = snapshot_feature_df.merge(latest_renounced_dates, on ='digitalLoanAccountId', how ='outer')

In [83]:
snapshot_feature_df.columns

Index(['digitalLoanAccountId', 'cnt_active_contracts',
       'cnt_active_contracts_Credit Cards',
       'cnt_active_contracts_Credit Lines',
       'cnt_active_contracts_Home Equity Loans',
       'cnt_active_contracts_Other Loans',
       'cnt_active_contracts_Personal Loans',
       'cnt_active_contracts_Real Estate Loans',
       'cnt_active_contracts_Short and Term Loans',
       'cnt_active_contracts_Time Loans', 'cnt_active_contracts_Trust Loans',
       'max_amt_active_contracts', 'tot_active_contracts_os_amt',
       'tot_active_contracts_util', 'Credit Cards_active_contracts_util',
       'Credit Lines_active_contracts_util',
       'Home Equity Loans_active_contracts_util',
       'Other Loans_active_contracts_util',
       'Personal Loans_active_contracts_util',
       'Real Estate Loans_active_contracts_util',
       'Short and Term Loans_active_contracts_util',
       'Time Loans_active_contracts_util', 'Trust Loans_active_contracts_util',
       'last_repay_amt_30d', 'h

In [84]:
duplicateincolumn(snapshot_feature_df, 'digitalLoanAccountId')

There are duplicates in the digitalLoanAccountId column


In [85]:
snapshot_feature_df.head()

,digitalLoanAccountId,cnt_active_contracts,cnt_active_contracts_Credit Cards,cnt_active_contracts_Credit Lines,cnt_active_contracts_Home Equity Loans,cnt_active_contracts_Other Loans,cnt_active_contracts_Personal Loans,cnt_active_contracts_Real Estate Loans,cnt_active_contracts_Short and Term Loans,cnt_active_contracts_Time Loans,cnt_active_contracts_Trust Loans,max_amt_active_contracts,tot_active_contracts_os_amt,tot_active_contracts_util,Credit Cards_active_contracts_util,Credit Lines_active_contracts_util,Home Equity Loans_active_contracts_util,Other Loans_active_contracts_util,Personal Loans_active_contracts_util,Real Estate Loans_active_contracts_util,Short and Term Loans_active_contracts_util,Time Loans_active_contracts_util,Trust Loans_active_contracts_util,last_repay_amt_30d,highest_latest_repay_method_30d,last_repay_amt_60d,highest_latest_repay_method_60d,last_repay_amt_90d,highest_latest_repay_method_90d,last_repay_amt,highest_latest_repay_method,max_age_active_contracts_snapshot,max_age_all_contracts_snapshot,days_since_last_inquiry,days_since_last_grant,days_since_last_closed,days_since_last_repay,days_since_last_reject,days_since_last_renounce
0,0000181e-f0bc-414d-bbc1-69fd841f4bc1,4.0,3.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,462700,12000378573109255115474,2.593553e+06,3.999996e+06,NaN,NaN,0.818182,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2314.0,2314.0,273.0,273.0,93.0,476.0,NaN,NaN
1,00025b22-70c8-4d93-ad59-4a58f5c11d56,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,586.0,174.0,201.0,2.0,33.0,NaN,NaN
2,00029466-5f50-433b-859d-23aa3cb6d04d,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,90000,25845,2.871667e-01,2.871667e-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016.0,2016.0,2016.0,2016.0,686.0,735.0,NaN,NaN
3,0003edd4-5a8c-4e00-bfeb-3fffc587d38b,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN
4,00051867-ce3d-4514-a9f3-b1c2f85ae4ed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,171.0,1534.0,1534.0,1363.0,NaN,NaN,NaN


In [86]:
# # Identify duplicates based on a specific subset of columns (e.g., 'digitalLoanAccountId')
# duplicates = snapshot_feature_df[snapshot_feature_df.duplicated(subset=['digitalLoanAccountId'], keep=False)].sort_values(by=['digitalLoanAccountId'])

# # Reset the index for easier row comparison
# duplicates = duplicates.reset_index(drop=True)

# # Create a DataFrame to store which columns are different
# diff_columns = pd.DataFrame(index=duplicates.index, columns=duplicates.columns)

# # Iterate over pairs of rows with the same ID
# for i in range(0, len(duplicates), 2):
#     for col in duplicates.columns:
#         if duplicates.at[i, col] != duplicates.at[i + 1, col]:
#             diff_columns.at[i, col] = True
#             diff_columns.at[i + 1, col] = True
#         else:
#             diff_columns.at[i, col] = False
#             diff_columns.at[i + 1, col] = False

# # Only show the rows where there's a difference
# diff_columns = diff_columns[diff_columns.any(axis=1)]

# print("Differences in Duplicate Rows:")
# diff_columns


In [87]:
# Correct the column name from 'digitalLoanAccoutId' to 'digitalLoanAccountId'
snapshot_feature_df = snapshot_feature_df.drop_duplicates(subset='digitalLoanAccountId', keep='last')

In [88]:
counts= snapshot_feature_df['digitalLoanAccountId'].value_counts()
counts[counts>1].head()

Series([], Name: count, dtype: int64)

In [89]:
len(counts.unique())

1

In [90]:
snapshot_feature_df[snapshot_feature_df['digitalLoanAccountId']=='f9b5df48-6338-41ab-ad2f-b2ef5acd8164']

,digitalLoanAccountId,cnt_active_contracts,cnt_active_contracts_Credit Cards,cnt_active_contracts_Credit Lines,cnt_active_contracts_Home Equity Loans,cnt_active_contracts_Other Loans,cnt_active_contracts_Personal Loans,cnt_active_contracts_Real Estate Loans,cnt_active_contracts_Short and Term Loans,cnt_active_contracts_Time Loans,cnt_active_contracts_Trust Loans,max_amt_active_contracts,tot_active_contracts_os_amt,tot_active_contracts_util,Credit Cards_active_contracts_util,Credit Lines_active_contracts_util,Home Equity Loans_active_contracts_util,Other Loans_active_contracts_util,Personal Loans_active_contracts_util,Real Estate Loans_active_contracts_util,Short and Term Loans_active_contracts_util,Time Loans_active_contracts_util,Trust Loans_active_contracts_util,last_repay_amt_30d,highest_latest_repay_method_30d,last_repay_amt_60d,highest_latest_repay_method_60d,last_repay_amt_90d,highest_latest_repay_method_90d,last_repay_amt,highest_latest_repay_method,max_age_active_contracts_snapshot,max_age_all_contracts_snapshot,days_since_last_inquiry,days_since_last_grant,days_since_last_closed,days_since_last_repay,days_since_last_reject,days_since_last_renounce
55314,f9b5df48-6338-41ab-ad2f-b2ef5acd8164,8.0,6.0,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,655200,3410131609590010343150343151463341001463417417...,1.705057e-16,1.705057e-16,NaN,NaN,0.900504,1.357143,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1636.0,1636.0,106.0,108.0,908.0,81.0,523.0,NaN


## Historical Features For Granted Loans

In [91]:
data['threshold_90days'] = data['run_date'] - pd.Timedelta(days=90)
data['threshold_180days'] = data['run_date'] - pd.Timedelta(days=180)
data['threshold_365days'] = data['run_date'] - pd.Timedelta(days=365)
data['threshold_730days'] = data['run_date'] - pd.Timedelta(days=730)


In [92]:
granted_loans = data[data['SOURCE']=='granted']

granted_loans_90days = granted_loans[(granted_loans['ContractStartDate'] >= granted_loans['threshold_90days']) & (granted_loans['ContractStartDate'] <= granted_loans['run_date'])]
granted_loans_180days = granted_loans[(granted_loans['ContractStartDate'] >= granted_loans['threshold_180days']) & (granted_loans['ContractStartDate'] <= granted_loans['run_date'])]
granted_loans_365days = granted_loans[(granted_loans['ContractStartDate'] >= granted_loans['threshold_365days']) & (granted_loans['ContractStartDate'] <= granted_loans['run_date'])]
granted_loans_730days = granted_loans[(granted_loans['ContractStartDate'] >= granted_loans['threshold_730days']) & (granted_loans['ContractStartDate'] <= granted_loans['run_date'])]

### Count of Granted loans by loan segment

In [93]:
granted_loans_by_segment_6m =granted_loans_180days.groupby(['digitalLoanAccountId','loan_segment']).agg(
                                         granted_contracts_cnt_6M=('CBContractCode', 'nunique')).reset_index()


pivot_granted_loans_by_segment_6m = granted_loans_by_segment_6m.pivot_table(index=['digitalLoanAccountId'], columns='loan_segment', values=['granted_contracts_cnt_6M'])

pivot_granted_loans_by_segment_6m.columns = [segment+'_granted_contracts_cnt_6M' for metric, segment in pivot_granted_loans_by_segment_6m.columns]
pivot_granted_loans_by_segment_6m = pivot_granted_loans_by_segment_6m.reset_index()


pivot_granted_loans_by_segment_6m.head(5)

,digitalLoanAccountId,Credit Cards_granted_contracts_cnt_6M,Credit Lines_granted_contracts_cnt_6M,Home Equity Loans_granted_contracts_cnt_6M,Other Loans_granted_contracts_cnt_6M,Personal Loans_granted_contracts_cnt_6M,Real Estate Loans_granted_contracts_cnt_6M,Short and Term Loans_granted_contracts_cnt_6M,Time Loans_granted_contracts_cnt_6M,Trust Loans_granted_contracts_cnt_6M
0,001a8c7d-b132-4a5b-8686-ebfd945d3bad,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN
1,001afde5-17dd-48c2-9693-b6df1aa8e752,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
2,001c77ba-72f9-4c44-a08d-4de9db9f0539,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
3,0022f527-0559-43fb-9e2d-65afcffda50b,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
4,002482bc-e446-4a6f-ab2a-c95692d02a11,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN


In [94]:
duplicateincolumn(pivot_granted_loans_by_segment_6m, 'digitalLoanAccountId')

No duplicates found in the digitalLoanAccountId column


In [95]:
granted_loans_by_segment_12m =granted_loans_365days.groupby(['digitalLoanAccountId','loan_segment']).agg(
                                         granted_contracts_cnt_12M=('CBContractCode', 'nunique')).reset_index()


pivot_granted_loans_by_segment_12m = granted_loans_by_segment_12m.pivot_table(index=['digitalLoanAccountId'], columns='loan_segment', values=['granted_contracts_cnt_12M'])

pivot_granted_loans_by_segment_12m.columns = [segment+'_granted_contracts_cnt_12M' for metric, segment in pivot_granted_loans_by_segment_12m.columns]
pivot_granted_loans_by_segment_12m = pivot_granted_loans_by_segment_12m.reset_index()


pivot_granted_loans_by_segment_12m.head(5)

,digitalLoanAccountId,Credit Cards_granted_contracts_cnt_12M,Credit Lines_granted_contracts_cnt_12M,Home Equity Loans_granted_contracts_cnt_12M,Other Loans_granted_contracts_cnt_12M,Personal Loans_granted_contracts_cnt_12M,Real Estate Loans_granted_contracts_cnt_12M,Short and Term Loans_granted_contracts_cnt_12M,Time Loans_granted_contracts_cnt_12M,Trust Loans_granted_contracts_cnt_12M
0,0000181e-f0bc-414d-bbc1-69fd841f4bc1,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
1,00025b22-70c8-4d93-ad59-4a58f5c11d56,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
2,000f89ba-1e76-4259-b4d7-76737689180d,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
3,00105a59-1559-4dad-a281-6408eecc96c1,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
4,00163bb9-b1a6-40f8-9f75-64a3e7313ee1,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN


In [96]:
duplicateincolumn(pivot_granted_loans_by_segment_12m, 'digitalLoanAccountId')

No duplicates found in the digitalLoanAccountId column


In [97]:
granted_loans_by_segment_24m =granted_loans_730days.groupby(['digitalLoanAccountId','loan_segment']).agg(
                                         granted_contracts_cnt_24M=('CBContractCode', 'nunique')).reset_index()


pivot_granted_loans_by_segment_24m = granted_loans_by_segment_24m.pivot_table(index=['digitalLoanAccountId'], columns='loan_segment', values=['granted_contracts_cnt_24M'])


pivot_granted_loans_by_segment_24m.columns = [segment+'_granted_contracts_cnt_24M' for metric, segment in pivot_granted_loans_by_segment_24m.columns]
pivot_granted_loans_by_segment_24m = pivot_granted_loans_by_segment_24m.reset_index()


pivot_granted_loans_by_segment_24m.head(5)

,digitalLoanAccountId,Credit Cards_granted_contracts_cnt_24M,Credit Lines_granted_contracts_cnt_24M,Home Equity Loans_granted_contracts_cnt_24M,Other Loans_granted_contracts_cnt_24M,Personal Loans_granted_contracts_cnt_24M,Real Estate Loans_granted_contracts_cnt_24M,Short and Term Loans_granted_contracts_cnt_24M,Time Loans_granted_contracts_cnt_24M,Trust Loans_granted_contracts_cnt_24M
0,0000181e-f0bc-414d-bbc1-69fd841f4bc1,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN
1,00025b22-70c8-4d93-ad59-4a58f5c11d56,NaN,NaN,NaN,1.0,6.0,NaN,NaN,NaN,NaN
2,000a3bee-c57d-4647-beaa-f8d3293b4101,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
3,000a6b2d-c9db-4d5a-9ef2-0f37987301f3,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN
4,000b9c72-3e95-4c0c-8e4c-b5c57a0bd0ff,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN


In [98]:
duplicateincolumn(pivot_granted_loans_by_segment_24m, 'digitalLoanAccountId')

No duplicates found in the digitalLoanAccountId column


In [99]:
granted_loan_cnt_by_segment = pivot_granted_loans_by_segment_6m.merge(pivot_granted_loans_by_segment_12m, on=['digitalLoanAccountId'], how='outer')
granted_loan_cnt_by_segment = granted_loan_cnt_by_segment.merge(pivot_granted_loans_by_segment_24m, on=['digitalLoanAccountId'], how='outer')

In [100]:
granted_loan_cnt_by_segment.columns

Index(['digitalLoanAccountId', 'Credit Cards_granted_contracts_cnt_6M',
       'Credit Lines_granted_contracts_cnt_6M',
       'Home Equity Loans_granted_contracts_cnt_6M',
       'Other Loans_granted_contracts_cnt_6M',
       'Personal Loans_granted_contracts_cnt_6M',
       'Real Estate Loans_granted_contracts_cnt_6M',
       'Short and Term Loans_granted_contracts_cnt_6M',
       'Time Loans_granted_contracts_cnt_6M',
       'Trust Loans_granted_contracts_cnt_6M',
       'Credit Cards_granted_contracts_cnt_12M',
       'Credit Lines_granted_contracts_cnt_12M',
       'Home Equity Loans_granted_contracts_cnt_12M',
       'Other Loans_granted_contracts_cnt_12M',
       'Personal Loans_granted_contracts_cnt_12M',
       'Real Estate Loans_granted_contracts_cnt_12M',
       'Short and Term Loans_granted_contracts_cnt_12M',
       'Time Loans_granted_contracts_cnt_12M',
       'Trust Loans_granted_contracts_cnt_12M',
       'Credit Cards_granted_contracts_cnt_24M',
       'Credit Lines_g

In [101]:
duplicateincolumn(granted_loan_cnt_by_segment, 'digitalLoanAccountId')

No duplicates found in the digitalLoanAccountId column


### Count of Granted loans 6m,12m,24m

In [102]:
granted_loans_cnt_12m =granted_loans_365days.groupby(['digitalLoanAccountId']).agg(
                                         granted_contracts_cnt_12M=('CBContractCode', 'nunique')).reset_index()
granted_loans_cnt_12m.head()

,digitalLoanAccountId,granted_contracts_cnt_12M
0,0000181e-f0bc-414d-bbc1-69fd841f4bc1,1
1,00025b22-70c8-4d93-ad59-4a58f5c11d56,1
2,000f89ba-1e76-4259-b4d7-76737689180d,1
3,00105a59-1559-4dad-a281-6408eecc96c1,1
4,00163bb9-b1a6-40f8-9f75-64a3e7313ee1,1


In [103]:
granted_loans_cnt_6m =granted_loans_180days.groupby(['digitalLoanAccountId']).agg(
                                         granted_contracts_cnt_6M=('CBContractCode', 'nunique')).reset_index()
granted_loans_cnt_6m.head()

,digitalLoanAccountId,granted_contracts_cnt_6M
0,001a8c7d-b132-4a5b-8686-ebfd945d3bad,5
1,001afde5-17dd-48c2-9693-b6df1aa8e752,1
2,001c77ba-72f9-4c44-a08d-4de9db9f0539,1
3,0022f527-0559-43fb-9e2d-65afcffda50b,1
4,002482bc-e446-4a6f-ab2a-c95692d02a11,1


In [104]:
granted_loans_cnt_24m =granted_loans_730days.groupby(['digitalLoanAccountId']).agg(
                                         granted_contracts_cnt_24M=('CBContractCode', 'nunique')).reset_index()
granted_loans_cnt_24m.head()

,digitalLoanAccountId,granted_contracts_cnt_24M
0,0000181e-f0bc-414d-bbc1-69fd841f4bc1,3
1,00025b22-70c8-4d93-ad59-4a58f5c11d56,7
2,000a3bee-c57d-4647-beaa-f8d3293b4101,1
3,000a6b2d-c9db-4d5a-9ef2-0f37987301f3,2
4,000b9c72-3e95-4c0c-8e4c-b5c57a0bd0ff,2


In [105]:
granted_loan_cnt_df = granted_loans_cnt_6m.merge(granted_loans_cnt_12m, on=['digitalLoanAccountId'], how='outer')
granted_loan_cnt_df = granted_loan_cnt_df.merge(granted_loans_cnt_24m, on=['digitalLoanAccountId'], how='outer')
granted_loan_cnt_df.head()

,digitalLoanAccountId,granted_contracts_cnt_6M,granted_contracts_cnt_12M,granted_contracts_cnt_24M
0,0000181e-f0bc-414d-bbc1-69fd841f4bc1,NaN,1.0,3
1,00025b22-70c8-4d93-ad59-4a58f5c11d56,NaN,1.0,7
2,000a3bee-c57d-4647-beaa-f8d3293b4101,NaN,NaN,1
3,000a6b2d-c9db-4d5a-9ef2-0f37987301f3,NaN,NaN,2
4,000b9c72-3e95-4c0c-8e4c-b5c57a0bd0ff,NaN,NaN,2


In [106]:
duplicateincolumn(granted_loan_cnt_df, 'digitalLoanAccountId')

No duplicates found in the digitalLoanAccountId column


### Amt Granted loans by loan segment

In [107]:
# Explicitly create a copy to avoid the SettingWithCopyWarning
granted_loans_180days = granted_loans_180days.copy()

# Identify non-numeric values in the 'SanctionedAmount' column
non_numeric_values = granted_loans_180days[pd.to_numeric(granted_loans_180days['SanctionedAmount'], errors='coerce').isna()]
print(non_numeric_values)

# Convert 'SanctionedAmount' to numeric, coercing errors to NaN
granted_loans_180days['SanctionedAmount'] = pd.to_numeric(granted_loans_180days['SanctionedAmount'], errors='coerce')

# Drop rows where 'SanctionedAmount' is NaN
granted_loans_180days.dropna(subset=['SanctionedAmount'], inplace=True)

# Aggregate loan amounts by digitalLoanAccountId and loan_segment over 6 months
granted_loanamt_by_segment_6m = granted_loans_180days.groupby(['digitalLoanAccountId', 'loan_segment']).agg(
    granted_contracts_amt_6M=('SanctionedAmount', 'sum')).reset_index()

# Pivot the table to have loan segments as columns
pivot_granted_loanamt_by_segment_6m = granted_loanamt_by_segment_6m.pivot_table(
    index=['digitalLoanAccountId'], columns='loan_segment', values='granted_contracts_amt_6M')

# Rename columns to include '_granted_contracts_amt_6M' suffix
pivot_granted_loanamt_by_segment_6m.columns = [
    f"{segment}_granted_contracts_amt_6M" for segment in pivot_granted_loanamt_by_segment_6m.columns]

# Reset index to make 'digitalLoanAccountId' a column again
pivot_granted_loanamt_by_segment_6m = pivot_granted_loanamt_by_segment_6m.reset_index()

# Display the first 5 rows
pivot_granted_loanamt_by_segment_6m.head(5)


Empty DataFrame
Columns: [uniquekey, digitalLoanAccountId, customerId, Firstname, middleName, LastName, loanAccountNumber, flagDisbursement, startApplyDateTime, disbursementDateTime, termsAndConditionsSubmitDateTime, natureofwork, subIndustryDescription, industryDescription, LoanProduct, crifApplicationId, processEngineGuid, requestGuid, ContractHistoryType, CBContractCode, ContractEndDate, ContractPhase, ContractPhaseDesc, ContractStartDate, ContractStatus, ContractStatusDesc, ContractType, ContractTypeDesc, Currency, CurrencyDesc, LastUpdateDate, OriginalCurrency, OriginalCurrencyDesc, ProviderCodeEncrypted, ProviderContractNo, ReferenceNo, Role, RoleDesc, BilledAmount, BoardResolutionFlag, BoardResolutionFlagDesc, CancellationDate, CardReferenceCode, ChargedAmount, CreditLimit, CreditPurpose, CreditPurposeDesc, FinancedAmount, FirstPaymentDate, FlagCardUsed, HolderLiability, HolderLiabilityDesc, InstallmentType, InstallmentTypeDesc, InstallmentsNumber, LastChargeDate, LastPaymentAmo

,digitalLoanAccountId,Credit Cards_granted_contracts_amt_6M,Credit Lines_granted_contracts_amt_6M,Home Equity Loans_granted_contracts_amt_6M,Other Loans_granted_contracts_amt_6M,Personal Loans_granted_contracts_amt_6M,Real Estate Loans_granted_contracts_amt_6M,Short and Term Loans_granted_contracts_amt_6M,Time Loans_granted_contracts_amt_6M,Trust Loans_granted_contracts_amt_6M
0,001a8c7d-b132-4a5b-8686-ebfd945d3bad,NaN,NaN,NaN,NaN,20000.0,NaN,NaN,NaN,NaN
1,001afde5-17dd-48c2-9693-b6df1aa8e752,NaN,NaN,NaN,NaN,9000.0,NaN,NaN,NaN,NaN
2,001c77ba-72f9-4c44-a08d-4de9db9f0539,NaN,NaN,NaN,5458.0,NaN,NaN,NaN,NaN,NaN
3,0022f527-0559-43fb-9e2d-65afcffda50b,NaN,NaN,NaN,6562.0,NaN,NaN,NaN,NaN,NaN
4,002482bc-e446-4a6f-ab2a-c95692d02a11,NaN,NaN,NaN,NaN,13398.0,NaN,NaN,NaN,NaN


In [108]:
duplicateincolumn(pivot_granted_loanamt_by_segment_6m, 'digitalLoanAccountId')

No duplicates found in the digitalLoanAccountId column


In [109]:
# Ensure SanctionedAmount is numeric using .loc[]
granted_loans_365days.loc[:, 'SanctionedAmount'] = pd.to_numeric(granted_loans_365days['SanctionedAmount'], errors='coerce')

# Group by 'digitalLoanAccountId' and 'loan_segment', then sum the 'SanctionedAmount'
granted_loanamt_by_segment_12m = granted_loans_365days.groupby(['digitalLoanAccountId', 'loan_segment']).agg(
    granted_contracts_amt_12M=('SanctionedAmount', 'sum')
).reset_index()

# Pivot the table
pivot_granted_loanamt_by_segment_12m = granted_loanamt_by_segment_12m.pivot_table(
    index=['digitalLoanAccountId'], 
    columns='loan_segment', 
    values='granted_contracts_amt_12M'
)

# Flatten the columns
pivot_granted_loanamt_by_segment_12m.columns = [segment + '_granted_contracts_amt_12M' for segment in pivot_granted_loanamt_by_segment_12m.columns]

# Reset the index to flatten the DataFrame
pivot_granted_loanamt_by_segment_12m = pivot_granted_loanamt_by_segment_12m.reset_index()

# Display the first 5 rows
pivot_granted_loanamt_by_segment_12m.head(5)


,digitalLoanAccountId,Credit Cards_granted_contracts_amt_12M,Credit Lines_granted_contracts_amt_12M,Home Equity Loans_granted_contracts_amt_12M,Other Loans_granted_contracts_amt_12M,Personal Loans_granted_contracts_amt_12M,Real Estate Loans_granted_contracts_amt_12M,Short and Term Loans_granted_contracts_amt_12M,Time Loans_granted_contracts_amt_12M,Trust Loans_granted_contracts_amt_12M
0,0000181e-f0bc-414d-bbc1-69fd841f4bc1,NaN,NaN,NaN,30000.0,NaN,NaN,NaN,NaN,NaN
1,00025b22-70c8-4d93-ad59-4a58f5c11d56,NaN,NaN,NaN,50000.0,NaN,NaN,NaN,NaN,NaN
2,000f89ba-1e76-4259-b4d7-76737689180d,NaN,NaN,NaN,30000.0,NaN,NaN,NaN,NaN,NaN
3,00105a59-1559-4dad-a281-6408eecc96c1,NaN,NaN,NaN,NaN,7999.0,NaN,NaN,NaN,NaN
4,00163bb9-b1a6-40f8-9f75-64a3e7313ee1,NaN,NaN,NaN,NaN,16000.0,NaN,NaN,NaN,NaN


In [110]:
duplicateincolumn(pivot_granted_loanamt_by_segment_12m, 'digitalLoanAccountId')

No duplicates found in the digitalLoanAccountId column


In [111]:
# Identify non-numeric values in the 'SanctionedAmount' column
non_numeric = granted_loans_730days[~granted_loans_730days['SanctionedAmount'].apply(lambda x: isinstance(x, (int, float)))]

# Convert 'SanctionedAmount' to numeric, with coercion of errors to NaN
granted_loans_730days.loc[:, 'SanctionedAmount'] = pd.to_numeric(granted_loans_730days['SanctionedAmount'], errors='coerce')

# Group by 'digitalLoanAccountId' and 'loan_segment' to aggregate the sanctioned amounts
granted_loansamt_by_segment_24m = granted_loans_730days.groupby(['digitalLoanAccountId', 'loan_segment']).agg(
    granted_contracts_amt_24M=('SanctionedAmount', 'sum')).reset_index()

# Pivot the DataFrame to create separate columns for each loan segment
pivot_granted_loansamt_by_segment_24m = granted_loansamt_by_segment_24m.pivot_table(
    index=['digitalLoanAccountId'], columns='loan_segment', values='granted_contracts_amt_24M')

# Rename the columns to include '_granted_contracts_amt_24M' suffix
pivot_granted_loansamt_by_segment_24m.columns = [segment + '_granted_contracts_amt_24M' for segment in pivot_granted_loansamt_by_segment_24m.columns]

# Reset the index to flatten the DataFrame
pivot_granted_loansamt_by_segment_24m = pivot_granted_loansamt_by_segment_24m.reset_index()

# Display the first 5 rows of the final pivoted DataFrame
pivot_granted_loansamt_by_segment_24m.head(5)


,digitalLoanAccountId,Credit Cards_granted_contracts_amt_24M,Credit Lines_granted_contracts_amt_24M,Home Equity Loans_granted_contracts_amt_24M,Other Loans_granted_contracts_amt_24M,Personal Loans_granted_contracts_amt_24M,Real Estate Loans_granted_contracts_amt_24M,Short and Term Loans_granted_contracts_amt_24M,Time Loans_granted_contracts_amt_24M,Trust Loans_granted_contracts_amt_24M
0,0000181e-f0bc-414d-bbc1-69fd841f4bc1,NaN,NaN,NaN,90000.0,NaN,NaN,NaN,NaN,NaN
1,00025b22-70c8-4d93-ad59-4a58f5c11d56,NaN,NaN,NaN,50000.0,95342.0,NaN,NaN,NaN,NaN
2,000a3bee-c57d-4647-beaa-f8d3293b4101,NaN,NaN,NaN,67400.0,NaN,NaN,NaN,NaN,NaN
3,000a6b2d-c9db-4d5a-9ef2-0f37987301f3,NaN,NaN,NaN,20000.0,NaN,NaN,NaN,NaN,NaN
4,000b9c72-3e95-4c0c-8e4c-b5c57a0bd0ff,NaN,NaN,NaN,5000.0,NaN,NaN,5000.0,NaN,NaN


In [112]:
duplicateincolumn(pivot_granted_loansamt_by_segment_24m, 'digitalLoanAccountId')

No duplicates found in the digitalLoanAccountId column


In [113]:
granted_loan_amt_by_segment = pivot_granted_loanamt_by_segment_6m.merge(pivot_granted_loanamt_by_segment_12m, on=['digitalLoanAccountId'], how='outer')
granted_loan_amt_by_segment = granted_loan_amt_by_segment.merge(pivot_granted_loansamt_by_segment_24m, on=['digitalLoanAccountId'], how='outer')

In [114]:
granted_loan_amt_by_segment.columns

Index(['digitalLoanAccountId', 'Credit Cards_granted_contracts_amt_6M',
       'Credit Lines_granted_contracts_amt_6M',
       'Home Equity Loans_granted_contracts_amt_6M',
       'Other Loans_granted_contracts_amt_6M',
       'Personal Loans_granted_contracts_amt_6M',
       'Real Estate Loans_granted_contracts_amt_6M',
       'Short and Term Loans_granted_contracts_amt_6M',
       'Time Loans_granted_contracts_amt_6M',
       'Trust Loans_granted_contracts_amt_6M',
       'Credit Cards_granted_contracts_amt_12M',
       'Credit Lines_granted_contracts_amt_12M',
       'Home Equity Loans_granted_contracts_amt_12M',
       'Other Loans_granted_contracts_amt_12M',
       'Personal Loans_granted_contracts_amt_12M',
       'Real Estate Loans_granted_contracts_amt_12M',
       'Short and Term Loans_granted_contracts_amt_12M',
       'Time Loans_granted_contracts_amt_12M',
       'Trust Loans_granted_contracts_amt_12M',
       'Credit Cards_granted_contracts_amt_24M',
       'Credit Lines_g

In [115]:
duplicateincolumn(granted_loan_amt_by_segment, 'digitalLoanAccountId')

No duplicates found in the digitalLoanAccountId column


### Amt of Granted loans 6m,12m,24m

In [116]:
granted_loans_amt_12m =granted_loans_365days.groupby(['digitalLoanAccountId']).agg(
                                         tot_granted_contracts_amt_12M=('SanctionedAmount', 'sum')).reset_index()
granted_loans_amt_12m.head()

,digitalLoanAccountId,tot_granted_contracts_amt_12M
0,0000181e-f0bc-414d-bbc1-69fd841f4bc1,30000
1,00025b22-70c8-4d93-ad59-4a58f5c11d56,50000
2,000f89ba-1e76-4259-b4d7-76737689180d,30000
3,00105a59-1559-4dad-a281-6408eecc96c1,7999
4,00163bb9-b1a6-40f8-9f75-64a3e7313ee1,16000


In [117]:
granted_loans_amt_6m =granted_loans_180days.groupby(['digitalLoanAccountId']).agg(
                                         tot_granted_contracts_amt_6M=('SanctionedAmount', 'sum')).reset_index()
granted_loans_amt_6m.head()

,digitalLoanAccountId,tot_granted_contracts_amt_6M
0,001a8c7d-b132-4a5b-8686-ebfd945d3bad,20000
1,001afde5-17dd-48c2-9693-b6df1aa8e752,9000
2,001c77ba-72f9-4c44-a08d-4de9db9f0539,5458
3,0022f527-0559-43fb-9e2d-65afcffda50b,6562
4,002482bc-e446-4a6f-ab2a-c95692d02a11,13398


In [118]:
granted_loans_amt_24m =granted_loans_730days.groupby(['digitalLoanAccountId']).agg(
                                         tot_granted_contracts_amt_24M=('SanctionedAmount', 'sum')).reset_index()
granted_loans_amt_24m.head()

,digitalLoanAccountId,tot_granted_contracts_amt_24M
0,0000181e-f0bc-414d-bbc1-69fd841f4bc1,90000
1,00025b22-70c8-4d93-ad59-4a58f5c11d56,145342
2,000a3bee-c57d-4647-beaa-f8d3293b4101,67400
3,000a6b2d-c9db-4d5a-9ef2-0f37987301f3,20000
4,000b9c72-3e95-4c0c-8e4c-b5c57a0bd0ff,10000


In [119]:
granted_loan_amt_df = granted_loans_amt_6m.merge(granted_loans_amt_12m, on=['digitalLoanAccountId'], how='outer')
granted_loan_amt_df = granted_loan_amt_df.merge(granted_loans_amt_24m, on=['digitalLoanAccountId'], how='outer')
granted_loan_amt_df.columns

Index(['digitalLoanAccountId', 'tot_granted_contracts_amt_6M',
       'tot_granted_contracts_amt_12M', 'tot_granted_contracts_amt_24M'],
      dtype='object')

In [120]:
duplicateincolumn(granted_loan_amt_df, 'digitalLoanAccountId')

No duplicates found in the digitalLoanAccountId column


### total, min, max, avg for sanctioned amount

In [121]:
# granted_loans_365days['Sanctioned_amount'] = granted_loans_365days['CreditLimit'].fillna(granted_loans_365days['FinancedAmount'])
grannted_sanctioned_amt_12M= granted_loans_365days.groupby(['digitalLoanAccountId']).agg(
                                        avg_amt_granted_12M=('SanctionedAmount', 'mean'),
                                        min_amt_granted_12M=('SanctionedAmount', 'min'),
                                        max_amt_granted_12M=('SanctionedAmount', 'max') ,
#                                         total_amt_granted_12M=('SanctionedAmount', 'sum')
                                        ).reset_index()

grannted_sanctioned_amt_12M.head()

,digitalLoanAccountId,avg_amt_granted_12M,min_amt_granted_12M,max_amt_granted_12M
0,0000181e-f0bc-414d-bbc1-69fd841f4bc1,30000.0,30000,30000
1,00025b22-70c8-4d93-ad59-4a58f5c11d56,50000.0,50000,50000
2,000f89ba-1e76-4259-b4d7-76737689180d,30000.0,30000,30000
3,00105a59-1559-4dad-a281-6408eecc96c1,7999.0,7999,7999
4,00163bb9-b1a6-40f8-9f75-64a3e7313ee1,16000.0,16000,16000


In [122]:
grannted_sanctioned_amt_6M= granted_loans_180days.groupby(['digitalLoanAccountId']).agg(
                                        avg_amt_granted_6M=('SanctionedAmount', 'mean'),
                                        min_amt_granted_6M=('SanctionedAmount', 'min'),
                                        max_amt_granted_6M=('SanctionedAmount', 'max') ,
#                                         total_amt_granted_6M=('SanctionedAmount', 'sum')
                                            ).reset_index()

grannted_sanctioned_amt_6M.head()

,digitalLoanAccountId,avg_amt_granted_6M,min_amt_granted_6M,max_amt_granted_6M
0,001a8c7d-b132-4a5b-8686-ebfd945d3bad,4000.0,4000,4000
1,001afde5-17dd-48c2-9693-b6df1aa8e752,9000.0,9000,9000
2,001c77ba-72f9-4c44-a08d-4de9db9f0539,5458.0,5458,5458
3,0022f527-0559-43fb-9e2d-65afcffda50b,6562.0,6562,6562
4,002482bc-e446-4a6f-ab2a-c95692d02a11,13398.0,13398,13398


In [123]:
grannted_sanctioned_amt_24M= granted_loans_730days.groupby(['digitalLoanAccountId']).agg(
                                        avg_amt_granted_24M=('SanctionedAmount', 'mean'),
                                        min_amt_granted_24M=('SanctionedAmount', 'min'),
                                        max_amt_granted_24M=('SanctionedAmount', 'max') ,
#                                         total_amt_granted_24M=('SanctionedAmount', 'sum') 
                                            ).reset_index()

grannted_sanctioned_amt_24M.head()

,digitalLoanAccountId,avg_amt_granted_24M,min_amt_granted_24M,max_amt_granted_24M
0,0000181e-f0bc-414d-bbc1-69fd841f4bc1,30000.0,30000,30000
1,00025b22-70c8-4d93-ad59-4a58f5c11d56,20763.142857,8010,50000
2,000a3bee-c57d-4647-beaa-f8d3293b4101,67400.0,67400,67400
3,000a6b2d-c9db-4d5a-9ef2-0f37987301f3,5000.0,5000,5000
4,000b9c72-3e95-4c0c-8e4c-b5c57a0bd0ff,5000.0,5000,5000


### Avg and Med time difference between granted loans

In [124]:
granted_loan_datediff_12M= granted_loans_365days[['digitalLoanAccountId','ContractStartDate','CBContractCode']]
granted_loan_datediff_12M = granted_loan_datediff_12M.sort_values(by=['digitalLoanAccountId','ContractStartDate','CBContractCode'])


granted_loan_datediff_12M['ContractStartDate'] = pd.to_datetime(granted_loan_datediff_12M['ContractStartDate'])

granted_loan_datediff_12M['day_diff'] = granted_loan_datediff_12M.groupby(['digitalLoanAccountId'])['ContractStartDate'].diff().dt.days


granted_loan_datediff_12M_sorted = granted_loan_datediff_12M.dropna(subset=['day_diff'])

# Sum the differences for each mobile number
granted_loan_datediff_12M_df = granted_loan_datediff_12M_sorted.groupby(['digitalLoanAccountId']).agg(
                                                        no_of_difference=('ContractStartDate', 'count'),
                                                        Total_datediff_12M=('day_diff', 'sum'),
                                                        med_days_bw_contracts_12m=('day_diff', 'median')                                
                                                                ).reset_index()
    
    

# Repayment_cashin.columns = ['mobile_num','no_of_difference', 'Total_cashin_datediff_180days','Median_cashin_datediff_180days','Min_cashin_datediff_180days','Max_cashin_datediff_180days']
granted_loan_datediff_12M_df['avg_days_bw_contracts_12m'] =( granted_loan_datediff_12M_df['Total_datediff_12M']/granted_loan_datediff_12M_df['no_of_difference']).round(2)

granted_loan_datediff_12M_df= granted_loan_datediff_12M_df[['digitalLoanAccountId','med_days_bw_contracts_12m','avg_days_bw_contracts_12m']]

granted_loan_datediff_12M_df.head()

,digitalLoanAccountId,med_days_bw_contracts_12m,avg_days_bw_contracts_12m
0,001a8c7d-b132-4a5b-8686-ebfd945d3bad,31.5,39.50
1,003972ae-7dce-4d8c-9ab7-2ee6f0d01adc,0.0,0.00
2,004503c8-1892-490c-9838-621d41f61ec2,15.0,15.00
3,0049a771-6757-431e-a7ac-099b3fd5b677,134.0,134.00
4,004fa342-db46-4474-9725-799a841bbf89,26.0,32.33


In [125]:
duplicateincolumn(granted_loan_datediff_12M_df, 'digitalLoanAccountId')

No duplicates found in the digitalLoanAccountId column


### How many loans were foreclosed? 6m, 12m, 24m

In [126]:
granted_foreclosed_730days = granted_loans_730days[granted_loans_730days['ContractStatusDesc'] == 'Foreclosure']
foreclosure_cnt_24M = granted_foreclosed_730days.groupby(['digitalLoanAccountId']).agg(
                                                           cnt_foreclosed_contracts_24M=('CBContractCode', 'count')).reset_index()

foreclosure_cnt_24M.head()

,digitalLoanAccountId,cnt_foreclosed_contracts_24M
0,003972ae-7dce-4d8c-9ab7-2ee6f0d01adc,2
1,00aa1580-66c5-48db-a714-900b2869ec04,1
2,00b0ab12-0a1a-4414-bf80-74c51f34dc9d,1
3,00b5bd95-4301-4826-9dc1-d6b4e393de72,1
4,00b5fefc-9f6e-445b-9bf1-cd3d54e918be,2


In [127]:

granted_foreclosed_365days = granted_loans_365days[granted_loans_365days['ContractStatusDesc'] == 'Foreclosure']

foreclosure_cnt_12M = granted_foreclosed_365days.groupby(['digitalLoanAccountId']).agg(
                                                            cnt_foreclosed_contracts_12M=('CBContractCode', 'count')).reset_index()


foreclosure_cnt_12M.head()

,digitalLoanAccountId,cnt_foreclosed_contracts_12M
0,003972ae-7dce-4d8c-9ab7-2ee6f0d01adc,2
1,00aa1580-66c5-48db-a714-900b2869ec04,1
2,00b0ab12-0a1a-4414-bf80-74c51f34dc9d,1
3,00b5bd95-4301-4826-9dc1-d6b4e393de72,1
4,00b5fefc-9f6e-445b-9bf1-cd3d54e918be,2


In [128]:

granted_foreclosed_180days = granted_loans_180days[granted_loans_180days['ContractStatusDesc'] == 'Foreclosure']

foreclosure_cnt_6M = granted_foreclosed_180days.groupby(['digitalLoanAccountId']).agg(
                                                            cnt_foreclosed_contracts_6M=('CBContractCode', 'count')).reset_index()

foreclosure_cnt_6M.head()

,digitalLoanAccountId,cnt_foreclosed_contracts_6M
0,00b5fefc-9f6e-445b-9bf1-cd3d54e918be,1
1,0106eef3-6f82-42fb-b1a2-7b7002f58149,1
2,02c5d9e0-4f29-4837-ae9b-b26f903fa343,1
3,02f7f85f-12a3-484b-97d8-c1ea851441f2,1
4,034ebeb9-5e0f-4c14-943f-6349e48fd98c,1


### No of written off ever

In [129]:
data[data['ContractStatus'] =='WO']['ContractPhaseDesc'].unique()

array(['Closed', 'Active'], dtype=object)

In [130]:
cnt_writtenoff_ever = granted_loans[granted_loans['ContractStatus'].isin(['WC', 'WF', 'WO'])]


cnt_writtenoff_ever = cnt_writtenoff_ever.groupby(['digitalLoanAccountId']).agg(
                                                            cnt_writeoff_ever=('CBContractCode', 'count')).reset_index()

cnt_writtenoff_ever.head()


,digitalLoanAccountId,cnt_writeoff_ever
0,001b666a-6a7a-41f9-94e0-dc35dc7d5fda,1
1,0047123b-b775-4881-a42d-b886d126608d,1
2,00592ed1-f63f-4540-8ddf-2f439120319b,1
3,00747bbc-f7f7-4749-9759-7206c8421297,1
4,0091966e-f643-4494-84f2-40641dff8743,2


### Velocity loans granted contracts

In [131]:
# Convert 'SanctionedAmount' to numeric, using .loc to avoid SettingWithCopyWarning
granted_loans_730days.loc[:, 'SanctionedAmount'] = pd.to_numeric(granted_loans_730days['SanctionedAmount'], errors='coerce')
granted_loans_365days.loc[:, 'SanctionedAmount'] = pd.to_numeric(granted_loans_365days['SanctionedAmount'], errors='coerce')
granted_loans_180days.loc[:, 'SanctionedAmount'] = pd.to_numeric(granted_loans_180days['SanctionedAmount'], errors='coerce')
granted_loans_90days.loc[:, 'SanctionedAmount'] = pd.to_numeric(granted_loans_90days['SanctionedAmount'], errors='coerce')

# Filter out non-numeric values, using .loc to avoid SettingWithCopyWarning
granted_loans_730days = granted_loans_730days.loc[granted_loans_730days['SanctionedAmount'].apply(lambda x: str(x).replace('.', '', 1).isdigit())]
granted_loans_365days = granted_loans_365days.loc[granted_loans_365days['SanctionedAmount'].apply(lambda x: str(x).replace('.', '', 1).isdigit())]
granted_loans_180days = granted_loans_180days.loc[granted_loans_180days['SanctionedAmount'].apply(lambda x: str(x).replace('.', '', 1).isdigit())]
granted_loans_90days = granted_loans_90days.loc[granted_loans_90days['SanctionedAmount'].apply(lambda x: str(x).replace('.', '', 1).isdigit())]

# Group by 'digitalLoanAccountId' and calculate aggregates
granted_sanctioned_amt_dfvelocity_24M = granted_loans_730days.groupby(['digitalLoanAccountId']).agg(
    avg_amt_granted_24M=('SanctionedAmount', 'mean'),
    avg_cnt_granted_24M=('CBContractCode', 'nunique')
).reset_index()

granted_sanctioned_amt_dfvelocity_12M = granted_loans_365days.groupby(['digitalLoanAccountId']).agg(
    avg_amt_granted_12M=('SanctionedAmount', 'mean'),
    avg_cnt_granted_12M=('CBContractCode', 'nunique')
).reset_index()

granted_sanctioned_amt_dfvelocity_6M = granted_loans_180days.groupby(['digitalLoanAccountId']).agg(
    avg_amt_granted_6M=('SanctionedAmount', 'mean'),
    avg_cnt_granted_6M=('CBContractCode', 'nunique')
).reset_index()

granted_sanctioned_amt_dfvelocity_3M = granted_loans_90days.groupby(['digitalLoanAccountId']).agg(
    avg_amt_granted_3M=('SanctionedAmount', 'mean'),
    avg_cnt_granted_3M=('CBContractCode', 'nunique')
).reset_index()


In [132]:
granted_sanctioned_amt_dfvelocity_24M['avg_cnt_granted_24M'] =(granted_sanctioned_amt_dfvelocity_24M['avg_cnt_granted_24M']/24).round(3)
granted_sanctioned_amt_dfvelocity_12M['avg_cnt_granted_12M'] =(granted_sanctioned_amt_dfvelocity_12M['avg_cnt_granted_12M']/12).round(3)
granted_sanctioned_amt_dfvelocity_6M['avg_cnt_granted_6M'] =(granted_sanctioned_amt_dfvelocity_6M['avg_cnt_granted_6M']/6).round(3)
granted_sanctioned_amt_dfvelocity_3M['avg_cnt_granted_3M'] =(granted_sanctioned_amt_dfvelocity_3M['avg_cnt_granted_3M']/3).round(3)

granted_sanctioned_amt_dfvelocity_24M.head()

,digitalLoanAccountId,avg_amt_granted_24M,avg_cnt_granted_24M
0,0000181e-f0bc-414d-bbc1-69fd841f4bc1,30000.0,0.125
1,00025b22-70c8-4d93-ad59-4a58f5c11d56,20763.142857,0.292
2,000a3bee-c57d-4647-beaa-f8d3293b4101,67400.0,0.042
3,000a6b2d-c9db-4d5a-9ef2-0f37987301f3,5000.0,0.083
4,000b9c72-3e95-4c0c-8e4c-b5c57a0bd0ff,5000.0,0.083


In [133]:
duplicateincolumn(granted_sanctioned_amt_dfvelocity_24M, 'digitalLoanAccountId')

No duplicates found in the digitalLoanAccountId column


In [134]:
vel_contract_granted_df = pd.merge(granted_sanctioned_amt_dfvelocity_3M, granted_sanctioned_amt_dfvelocity_6M, on='digitalLoanAccountId', how='outer')
vel_contract_granted_df = pd.merge(vel_contract_granted_df, granted_sanctioned_amt_dfvelocity_12M, on='digitalLoanAccountId', how='outer')
vel_contract_granted_df = pd.merge(vel_contract_granted_df, granted_sanctioned_amt_dfvelocity_24M, on='digitalLoanAccountId', how='outer')
vel_contract_granted_df.head(3)

,digitalLoanAccountId,avg_amt_granted_3M,avg_cnt_granted_3M,avg_amt_granted_6M,avg_cnt_granted_6M,avg_amt_granted_12M,avg_cnt_granted_12M,avg_amt_granted_24M,avg_cnt_granted_24M
0,0000181e-f0bc-414d-bbc1-69fd841f4bc1,NaN,NaN,NaN,NaN,30000.0,0.083,30000.0,0.125
1,00025b22-70c8-4d93-ad59-4a58f5c11d56,NaN,NaN,NaN,NaN,50000.0,0.083,20763.142857,0.292
2,000a3bee-c57d-4647-beaa-f8d3293b4101,NaN,NaN,NaN,NaN,NaN,NaN,67400.0,0.042


In [135]:
duplicateincolumn(vel_contract_granted_df, 'digitalLoanAccountId')

No duplicates found in the digitalLoanAccountId column


In [136]:
vel_contract_granted_df['vel_contract_granted_cnt_3on12'] = vel_contract_granted_df['avg_cnt_granted_3M']/vel_contract_granted_df['avg_cnt_granted_12M']
vel_contract_granted_df['vel_contract_granted_cnt_6on12'] = vel_contract_granted_df['avg_cnt_granted_6M']/vel_contract_granted_df['avg_cnt_granted_12M']
vel_contract_granted_df['vel_contract_granted_cnt_12on24'] = vel_contract_granted_df['avg_cnt_granted_12M']/vel_contract_granted_df['avg_cnt_granted_24M']

### Velocity loan amount granted contracts

In [137]:
def safe_divide(numerator, denominator):
    """Return the result of division if denominator is not zero; otherwise, return NaN."""
    return numerator / denominator if denominator != 0 else np.nan

# Calculate ratios while avoiding division by zero
vel_contract_granted_df['vel_contract_granted_amt_3on12'] = vel_contract_granted_df.apply(
    lambda row: safe_divide(row['avg_amt_granted_3M'], row['avg_amt_granted_12M']),
    axis=1
)

vel_contract_granted_df['vel_contract_granted_amt_6on12'] = vel_contract_granted_df.apply(
    lambda row: safe_divide(row['avg_amt_granted_6M'], row['avg_amt_granted_12M']),
    axis=1
)

vel_contract_granted_df['vel_contract_granted_amt_12on24'] = vel_contract_granted_df.apply(
    lambda row: safe_divide(row['avg_amt_granted_12M'], row['avg_amt_granted_24M']),
    axis=1
)

# Print the updated DataFrame
print(vel_contract_granted_df)

                       digitalLoanAccountId avg_amt_granted_3M  \
0      0000181e-f0bc-414d-bbc1-69fd841f4bc1                NaN   
1      00025b22-70c8-4d93-ad59-4a58f5c11d56                NaN   
2      000a3bee-c57d-4647-beaa-f8d3293b4101                NaN   
3      000a6b2d-c9db-4d5a-9ef2-0f37987301f3                NaN   
4      000b9c72-3e95-4c0c-8e4c-b5c57a0bd0ff                NaN   
...                                     ...                ...   
29527  fff93cc7-530c-4641-ab71-15e7e7ac729c                NaN   
29528  fffa3d10-6829-4026-8d7b-13b76203f679                NaN   
29529  fffa9475-c041-49f2-9a1d-5bd8c5667cbf                NaN   
29530  fffaf81e-7d5d-4019-9c6d-696c86d8eef1                NaN   
29531  fffd7178-799c-40d3-8990-c43e9e54285d                NaN   

       avg_cnt_granted_3M  avg_amt_granted_6M  avg_cnt_granted_6M  \
0                     NaN                 NaN                 NaN   
1                     NaN                 NaN                 NaN   


In [138]:
vel_contract_granted_df.columns

Index(['digitalLoanAccountId', 'avg_amt_granted_3M', 'avg_cnt_granted_3M',
       'avg_amt_granted_6M', 'avg_cnt_granted_6M', 'avg_amt_granted_12M',
       'avg_cnt_granted_12M', 'avg_amt_granted_24M', 'avg_cnt_granted_24M',
       'vel_contract_granted_cnt_3on12', 'vel_contract_granted_cnt_6on12',
       'vel_contract_granted_cnt_12on24', 'vel_contract_granted_amt_3on12',
       'vel_contract_granted_amt_6on12', 'vel_contract_granted_amt_12on24'],
      dtype='object')

In [139]:
duplicateincolumn(vel_contract_granted_df, 'digitalLoanAccountId')

No duplicates found in the digitalLoanAccountId column


In [140]:
vel_contract_granted_df = vel_contract_granted_df[['digitalLoanAccountId','vel_contract_granted_cnt_3on12', 'vel_contract_granted_cnt_6on12',
       'vel_contract_granted_cnt_12on24', 'vel_contract_granted_amt_3on12',
       'vel_contract_granted_amt_6on12', 'vel_contract_granted_amt_12on24']]

In [141]:
vel_contract_granted_df.head(2)

,digitalLoanAccountId,vel_contract_granted_cnt_3on12,vel_contract_granted_cnt_6on12,vel_contract_granted_cnt_12on24,vel_contract_granted_amt_3on12,vel_contract_granted_amt_6on12,vel_contract_granted_amt_12on24
0,0000181e-f0bc-414d-bbc1-69fd841f4bc1,NaN,NaN,0.664000,NaN,NaN,1.000000
1,00025b22-70c8-4d93-ad59-4a58f5c11d56,NaN,NaN,0.284247,NaN,NaN,2.408113


In [142]:
duplicateincolumn(vel_contract_granted_df, 'digitalLoanAccountId')

No duplicates found in the digitalLoanAccountId column


### Velocity closed amount

In [143]:
closed_granted_loans = data[(data['ContractPhaseDesc']=='Closed') | (data['ContractPhaseDesc']=='Closed in advance')].copy()

closed_granted_loans.loc[:, 'ContractEndDate'] = pd.to_datetime(closed_granted_loans['ContractEndDate'], errors='coerce')
closed_granted_loans.loc[:, 'run_date'] = pd.to_datetime(closed_granted_loans['run_date'], errors='coerce')

granted_closed_loans_90days = closed_granted_loans[
    (closed_granted_loans['ContractEndDate'] >= closed_granted_loans['threshold_90days']) & 
    (closed_granted_loans['ContractEndDate'] <= closed_granted_loans['run_date'])
]

granted_closed_loans_180days = closed_granted_loans[
    (closed_granted_loans['ContractEndDate'] >= closed_granted_loans['threshold_180days']) & 
    (closed_granted_loans['ContractEndDate'] <= closed_granted_loans['run_date'])
]

granted_closed_loans_365days = closed_granted_loans[
    (closed_granted_loans['ContractEndDate'] >= closed_granted_loans['threshold_365days']) & 
    (closed_granted_loans['ContractEndDate'] <= closed_granted_loans['run_date'])
]

granted_closed_loans_730days = closed_granted_loans[
    (closed_granted_loans['ContractEndDate'] >= closed_granted_loans['threshold_730days']) & 
    (closed_granted_loans['ContractEndDate'] <= closed_granted_loans['run_date'])
]


In [144]:
# Ensure the 'SanctionedAmount' column is numeric, converting non-numeric values to NaN using .loc[]
granted_closed_loans_730days.loc[:, 'SanctionedAmount'] = pd.to_numeric(granted_closed_loans_730days['SanctionedAmount'], errors='coerce')
granted_closed_loans_365days.loc[:, 'SanctionedAmount'] = pd.to_numeric(granted_closed_loans_365days['SanctionedAmount'], errors='coerce')
granted_closed_loans_180days.loc[:, 'SanctionedAmount'] = pd.to_numeric(granted_closed_loans_180days['SanctionedAmount'], errors='coerce')
granted_closed_loans_90days.loc[:, 'SanctionedAmount'] = pd.to_numeric(granted_closed_loans_90days['SanctionedAmount'], errors='coerce')

# Now you can perform the aggregation safely
granted_closed_amt_dfvelocity_24M = granted_closed_loans_730days.groupby(['digitalLoanAccountId']).agg(
    avg_closed_amt_granted_24M=('SanctionedAmount', 'mean'),
    tot_closed_cnt_granted_24M=('CBContractCode', 'nunique')
).reset_index()

granted_closed_amt_dfvelocity_12M = granted_closed_loans_365days.groupby(['digitalLoanAccountId']).agg(
    avg_closed_amt_granted_12M=('SanctionedAmount', 'mean'),
    tot_closed_cnt_granted_12M=('CBContractCode', 'nunique')
).reset_index()

granted_closed_amt_dfvelocity_6M = granted_closed_loans_180days.groupby(['digitalLoanAccountId']).agg(
    avg_closed_amt_granted_6M=('SanctionedAmount', 'mean'),
    tot_closed_cnt_granted_6M=('CBContractCode', 'nunique')
).reset_index()

granted_closed_amt_dfvelocity_3M = granted_closed_loans_90days.groupby(['digitalLoanAccountId']).agg(
    avg_closed_amt_granted_3M=('SanctionedAmount', 'mean'),
    tot_closed_cnt_granted_3M=('CBContractCode', 'nunique')
).reset_index()


In [145]:
granted_closed_amt_dfvelocity_24M['tot_closed_cnt_granted_24M'] =(granted_closed_amt_dfvelocity_24M['tot_closed_cnt_granted_24M']/24).round(3)
granted_closed_amt_dfvelocity_12M['tot_closed_cnt_granted_12M'] =(granted_closed_amt_dfvelocity_12M['tot_closed_cnt_granted_12M']/12).round(3)
granted_closed_amt_dfvelocity_6M['tot_closed_cnt_granted_6M'] =(granted_closed_amt_dfvelocity_6M['tot_closed_cnt_granted_6M']/6).round(3)
granted_closed_amt_dfvelocity_3M['tot_closed_cnt_granted_3M'] =(granted_closed_amt_dfvelocity_3M['tot_closed_cnt_granted_3M']/3).round(3)

granted_closed_amt_dfvelocity_24M.head()

,digitalLoanAccountId,avg_closed_amt_granted_24M,tot_closed_cnt_granted_24M
0,0000181e-f0bc-414d-bbc1-69fd841f4bc1,30000.0,0.167
1,00025b22-70c8-4d93-ad59-4a58f5c11d56,19731.5,0.333
2,00029466-5f50-433b-859d-23aa3cb6d04d,12000.0,0.042
3,000a6b2d-c9db-4d5a-9ef2-0f37987301f3,5000.0,0.083
4,000b9c72-3e95-4c0c-8e4c-b5c57a0bd0ff,5000.0,0.083


In [146]:
vel_contract_granted_closed_df = pd.merge(granted_closed_amt_dfvelocity_3M, granted_closed_amt_dfvelocity_6M, on='digitalLoanAccountId', how='outer')
vel_contract_granted_closed_df = pd.merge(vel_contract_granted_closed_df, granted_closed_amt_dfvelocity_12M, on='digitalLoanAccountId', how='outer')
vel_contract_granted_closed_df = pd.merge(vel_contract_granted_closed_df, granted_closed_amt_dfvelocity_24M, on='digitalLoanAccountId', how='outer')
vel_contract_granted_closed_df.head(3)

,digitalLoanAccountId,avg_closed_amt_granted_3M,tot_closed_cnt_granted_3M,avg_closed_amt_granted_6M,tot_closed_cnt_granted_6M,avg_closed_amt_granted_12M,tot_closed_cnt_granted_12M,avg_closed_amt_granted_24M,tot_closed_cnt_granted_24M
0,0000181e-f0bc-414d-bbc1-69fd841f4bc1,NaN,NaN,30000.0,0.167,30000.0,0.167,30000.0,0.167
1,00025b22-70c8-4d93-ad59-4a58f5c11d56,50000.0,0.333,50000.0,0.167,35305.0,0.167,19731.5,0.333
2,00029466-5f50-433b-859d-23aa3cb6d04d,NaN,NaN,NaN,NaN,NaN,NaN,12000.0,0.042


In [147]:
def safe_divide(numerator, denominator):
    """Return the result of division if denominator is not zero; otherwise, return NaN."""
    return numerator / denominator if denominator != 0 else np.nan

# Calculate ratios while avoiding division by zero
vel_contract_granted_closed_df['vel_contract_closed_amt_3on12'] = vel_contract_granted_closed_df.apply(
    lambda row: safe_divide(row['avg_closed_amt_granted_3M'], row['avg_closed_amt_granted_12M']),
    axis=1
)

vel_contract_granted_closed_df['vel_contract_closed_amt_6on12'] = vel_contract_granted_closed_df.apply(
    lambda row: safe_divide(row['avg_closed_amt_granted_6M'], row['avg_closed_amt_granted_12M']),
    axis=1
)

vel_contract_granted_closed_df['vel_contract_closed_amt_12on24'] = vel_contract_granted_closed_df.apply(
    lambda row: safe_divide(row['avg_closed_amt_granted_12M'], row['avg_closed_amt_granted_24M']),
    axis=1
)

# Print the updated DataFrame
print(vel_contract_granted_closed_df)


                       digitalLoanAccountId avg_closed_amt_granted_3M  \
0      0000181e-f0bc-414d-bbc1-69fd841f4bc1                       NaN   
1      00025b22-70c8-4d93-ad59-4a58f5c11d56                   50000.0   
2      00029466-5f50-433b-859d-23aa3cb6d04d                       NaN   
3      000a6b2d-c9db-4d5a-9ef2-0f37987301f3                       NaN   
4      000b9c72-3e95-4c0c-8e4c-b5c57a0bd0ff                       NaN   
...                                     ...                       ...   
26208  fff93cc7-530c-4641-ab71-15e7e7ac729c                       NaN   
26209  fffa3d10-6829-4026-8d7b-13b76203f679                       NaN   
26210  fffa9475-c041-49f2-9a1d-5bd8c5667cbf                       NaN   
26211  fffaf81e-7d5d-4019-9c6d-696c86d8eef1                       NaN   
26212  fffd7178-799c-40d3-8990-c43e9e54285d                       NaN   

       tot_closed_cnt_granted_3M avg_closed_amt_granted_6M  \
0                            NaN                   30000.0   

In [148]:
vel_contract_granted_closed_df = vel_contract_granted_closed_df[['digitalLoanAccountId','vel_contract_closed_amt_3on12',
                                                                'vel_contract_closed_amt_6on12','vel_contract_closed_amt_12on24']]
vel_contract_granted_closed_df.head()

,digitalLoanAccountId,vel_contract_closed_amt_3on12,vel_contract_closed_amt_6on12,vel_contract_closed_amt_12on24
0,0000181e-f0bc-414d-bbc1-69fd841f4bc1,NaN,1.00000,1.000000
1,00025b22-70c8-4d93-ad59-4a58f5c11d56,1.41623,1.41623,1.789271
2,00029466-5f50-433b-859d-23aa3cb6d04d,NaN,NaN,NaN
3,000a6b2d-c9db-4d5a-9ef2-0f37987301f3,NaN,NaN,1.000000
4,000b9c72-3e95-4c0c-8e4c-b5c57a0bd0ff,NaN,NaN,1.000000


In [149]:
duplicateincolumn(vel_contract_granted_closed_df, 'digitalLoanAccountId')

No duplicates found in the digitalLoanAccountId column


### Creditlimit in last 12M

In [150]:
Creditlimit_12m = granted_loans_365days[granted_loans_365days['CreditLimit'].isna() == False]

In [151]:
# Convert the 'CreditLimit' column to numeric, coercing errors to NaN using .loc
Creditlimit_12m.loc[:, 'CreditLimit'] = pd.to_numeric(Creditlimit_12m['CreditLimit'], errors='coerce')

# Optionally, drop rows with NaN values if they are not needed
# Creditlimit_12m = Creditlimit_12m.dropna(subset=['CreditLimit'])

# Group by and aggregate after conversion
credit_limit_12mon_df = Creditlimit_12m.groupby('digitalLoanAccountId').agg(
    avg_credit_limit_12M=('CreditLimit', 'mean'),
    min_credit_limit_12M=('CreditLimit', 'min'),
    max_credit_limit_12M=('CreditLimit', 'max')
).reset_index()


In [152]:
credit_limit_12mon_df.head()

,digitalLoanAccountId,avg_credit_limit_12M,min_credit_limit_12M,max_credit_limit_12M
0,002c7453-5836-4ee5-a590-6b037fa423ff,21000.0,21000,21000
1,00414a63-9755-431e-a95b-51dd466a7027,140000.0,140000,140000
2,0049b50f-a05e-48da-b8f2-42477d6301ae,21000.0,21000,21000
3,007a6c6f-f038-4547-96c6-2bf778a4cc75,20000.0,20000,20000
4,00a54bc1-db26-49e7-948c-fa445a0a03e2,55000.0,55000,55000


### FinancedAmount in last 12M

In [153]:
FinancedAmount_12m = granted_loans_365days[granted_loans_365days['FinancedAmount'].isna() == False]

In [154]:
# Convert 'FinancedAmount' to numeric, forcing errors to NaN
FinancedAmount_12m.loc[:, 'FinancedAmount'] = pd.to_numeric(FinancedAmount_12m['FinancedAmount'], errors='coerce')

# Drop rows where 'FinancedAmount' is NaN
FinancedAmount_12m = FinancedAmount_12m.dropna(subset=['FinancedAmount']).copy()

# Group by 'digitalLoanAccountId' and aggregate
FinancedAmt_12mon_df = FinancedAmount_12m.groupby('digitalLoanAccountId').agg(
    avg_financed_amt_12M=('FinancedAmount','mean'),
    min_financed_amt_12M=('FinancedAmount','min'),
    max_financed_amt_12M=('FinancedAmount','max')
).reset_index()

In [155]:
FinancedAmt_12mon_df.head()

,digitalLoanAccountId,avg_financed_amt_12M,min_financed_amt_12M,max_financed_amt_12M
0,0000181e-f0bc-414d-bbc1-69fd841f4bc1,30000.0,30000,30000
1,00025b22-70c8-4d93-ad59-4a58f5c11d56,50000.0,50000,50000
2,000f89ba-1e76-4259-b4d7-76737689180d,30000.0,30000,30000
3,00105a59-1559-4dad-a281-6408eecc96c1,7999.0,7999,7999
4,00163bb9-b1a6-40f8-9f75-64a3e7313ee1,16000.0,16000,16000


In [156]:
duplicateincolumn(FinancedAmt_12mon_df, 'digitalLoanAccountId')

No duplicates found in the digitalLoanAccountId column


### Ratio closed loans cnt/ granted loans cnt

In [157]:
total_closed_loans_24M= granted_closed_loans_730days.groupby(['digitalLoanAccountId']).agg(
                                        tot_closed_amt_granted_24M=('SanctionedAmount', 'sum'),
                                        tot_closed_cnt_granted_24M =('CBContractCode','count')).reset_index()

total_closed_loans_12M= granted_closed_loans_365days.groupby(['digitalLoanAccountId']).agg(
                                        tot_closed_amt_granted_12M=('SanctionedAmount', 'sum'),
                                        tot_closed_cnt_granted_12M =('CBContractCode','count')).reset_index()

total_closed_loans_6M= granted_closed_loans_180days.groupby(['digitalLoanAccountId']).agg(
                                        tot_closed_amt_granted_6M=('SanctionedAmount', 'sum'),
                                        tot_closed_cnt_granted_6M =('CBContractCode','count')).reset_index()


In [158]:
total_closed_loans_df = total_closed_loans_6M.merge(total_closed_loans_12M, on=['digitalLoanAccountId'],how='outer')
total_closed_loans_df = total_closed_loans_df.merge(total_closed_loans_24M, on=['digitalLoanAccountId'],how='outer')
total_closed_loans_df.head()

,digitalLoanAccountId,tot_closed_amt_granted_6M,tot_closed_cnt_granted_6M,tot_closed_amt_granted_12M,tot_closed_cnt_granted_12M,tot_closed_amt_granted_24M,tot_closed_cnt_granted_24M
0,0000181e-f0bc-414d-bbc1-69fd841f4bc1,30000,1.0,60000,2.0,120000,4
1,00025b22-70c8-4d93-ad59-4a58f5c11d56,50000,1.0,70610,2.0,157852,8
2,00029466-5f50-433b-859d-23aa3cb6d04d,NaN,NaN,NaN,NaN,12000,1
3,000a6b2d-c9db-4d5a-9ef2-0f37987301f3,NaN,NaN,10000,2.0,20000,4
4,000b9c72-3e95-4c0c-8e4c-b5c57a0bd0ff,NaN,NaN,5000,1.0,10000,2


In [159]:
duplicateincolumn(total_closed_loans_df, 'digitalLoanAccountId')

No duplicates found in the digitalLoanAccountId column


In [160]:
total_granted_loans_12M= granted_loans_365days.groupby(['digitalLoanAccountId']).agg(
                                        total_cnt_granted_12M=('CBContractCode', 'count'),
                                        total_amt_granted_12M=('SanctionedAmount', 'sum') ).reset_index()

total_granted_loans_6M= granted_loans_180days.groupby(['digitalLoanAccountId']).agg(
                                        total_cnt_granted_6M=('CBContractCode', 'count'),
                                        total_amt_granted_6M=('SanctionedAmount', 'sum') ).reset_index()

total_granted_loans_24M= granted_loans_730days.groupby(['digitalLoanAccountId']).agg(
                                        total_cnt_granted_24M=('CBContractCode', 'count'),
                                        total_amt_granted_24M=('SanctionedAmount', 'sum') ).reset_index()

In [161]:
total_granted_loans_df = total_granted_loans_6M.merge(total_granted_loans_12M, on=['digitalLoanAccountId'],how='outer')
total_granted_loans_df = total_granted_loans_df.merge(total_granted_loans_24M, on=['digitalLoanAccountId'],how='outer')
total_granted_loans_df.head()

,digitalLoanAccountId,total_cnt_granted_6M,total_amt_granted_6M,total_cnt_granted_12M,total_amt_granted_12M,total_cnt_granted_24M,total_amt_granted_24M
0,0000181e-f0bc-414d-bbc1-69fd841f4bc1,NaN,NaN,1.0,30000,3,90000
1,00025b22-70c8-4d93-ad59-4a58f5c11d56,NaN,NaN,1.0,50000,7,145342
2,000a3bee-c57d-4647-beaa-f8d3293b4101,NaN,NaN,NaN,NaN,1,67400
3,000a6b2d-c9db-4d5a-9ef2-0f37987301f3,NaN,NaN,NaN,NaN,4,20000
4,000b9c72-3e95-4c0c-8e4c-b5c57a0bd0ff,NaN,NaN,NaN,NaN,2,10000


In [162]:
duplicateincolumn(total_granted_loans_df, 'digitalLoanAccountId')

No duplicates found in the digitalLoanAccountId column


In [163]:
ratio_btw_closed_granted_loans= total_closed_loans_df.merge(total_granted_loans_df,on=['digitalLoanAccountId'], how='inner')

In [164]:
ratio_btw_closed_granted_loans['ratio_closed_over_new_granted_cnt_6M'] = ratio_btw_closed_granted_loans['tot_closed_cnt_granted_6M']/ratio_btw_closed_granted_loans['total_cnt_granted_6M']
ratio_btw_closed_granted_loans['ratio_closed_over_new_granted_cnt_12M'] = ratio_btw_closed_granted_loans['tot_closed_cnt_granted_12M']/ratio_btw_closed_granted_loans['total_cnt_granted_12M']
ratio_btw_closed_granted_loans['ratio_closed_over_new_granted_cnt_24M'] = ratio_btw_closed_granted_loans['tot_closed_cnt_granted_24M']/ratio_btw_closed_granted_loans['total_cnt_granted_24M']


In [165]:

def safe_divide(numerator, denominator):
    """Return the result of division if denominator is not zero; otherwise, return NaN."""
    return numerator / denominator if denominator != 0 else np.nan

# Calculate ratios while avoiding division by zero
ratio_btw_closed_granted_loans['ratio_closed_over_new_granted_amt_6M'] = ratio_btw_closed_granted_loans.apply(
    lambda row: safe_divide(row['total_amt_granted_6M'], row['tot_closed_amt_granted_6M']),
    axis=1
)

ratio_btw_closed_granted_loans['ratio_closed_over_new_granted_amt_12M'] = ratio_btw_closed_granted_loans.apply(
    lambda row: safe_divide(row['total_amt_granted_12M'], row['tot_closed_amt_granted_12M']),
    axis=1
)

ratio_btw_closed_granted_loans['ratio_closed_over_new_granted_amt_24M'] = ratio_btw_closed_granted_loans.apply(
    lambda row: safe_divide(row['total_amt_granted_24M'], row['tot_closed_amt_granted_24M']),
    axis=1
)

# Print the updated DataFrame
ratio_btw_closed_granted_loans


,digitalLoanAccountId,tot_closed_amt_granted_6M,tot_closed_cnt_granted_6M,tot_closed_amt_granted_12M,tot_closed_cnt_granted_12M,tot_closed_amt_granted_24M,tot_closed_cnt_granted_24M,total_cnt_granted_6M,total_amt_granted_6M,total_cnt_granted_12M,total_amt_granted_12M,total_cnt_granted_24M,total_amt_granted_24M,ratio_closed_over_new_granted_cnt_6M,ratio_closed_over_new_granted_cnt_12M,ratio_closed_over_new_granted_cnt_24M,ratio_closed_over_new_granted_amt_6M,ratio_closed_over_new_granted_amt_12M,ratio_closed_over_new_granted_amt_24M
0,0000181e-f0bc-414d-bbc1-69fd841f4bc1,30000,1.0,60000,2.0,120000,4,NaN,NaN,1.0,30000,3,90000,NaN,2.0,1.333333,NaN,0.500000,0.750000
1,00025b22-70c8-4d93-ad59-4a58f5c11d56,50000,1.0,70610,2.0,157852,8,NaN,NaN,1.0,50000,7,145342,NaN,2.0,1.142857,NaN,0.708115,0.920749
2,000a6b2d-c9db-4d5a-9ef2-0f37987301f3,NaN,NaN,10000,2.0,20000,4,NaN,NaN,NaN,NaN,4,20000,NaN,NaN,1.000000,NaN,NaN,1.000000
3,000b9c72-3e95-4c0c-8e4c-b5c57a0bd0ff,NaN,NaN,5000,1.0,10000,2,NaN,NaN,NaN,NaN,2,10000,NaN,NaN,1.000000,NaN,NaN,1.000000
4,000d27e2-705e-47e2-886d-542a43c0889b,NaN,NaN,40000,1.0,63500,3,NaN,NaN,NaN,NaN,2,23500,NaN,NaN,1.500000,NaN,NaN,0.370079
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22031,fff93cc7-530c-4641-ab71-15e7e7ac729c,NaN,NaN,8000,2.0,8000,2,NaN,NaN,2.0,8000,2,8000,NaN,1.0,1.000000,NaN,1.000000,1.000000
22032,fffa3d10-6829-4026-8d7b-13b76203f679,NaN,NaN,NaN,NaN,26908,1,NaN,NaN,NaN,NaN,1,40000,NaN,NaN,1.000000,NaN,NaN,1.486547
22033,fffa9475-c041-49f2-9a1d-5bd8c5667cbf,14508,1.0,14508,1.0,14508,1,NaN,NaN,NaN,NaN,1,14508,NaN,NaN,1.000000,NaN,NaN,1.000000
22034,fffaf81e-7d5d-4019-9c6d-696c86d8eef1,NaN,NaN,2000,1.0,2000,1,1.0,12000.0,2.0,14000,2,14000,NaN,0.5,0.500000,NaN,7.000000,7.000000


In [166]:
ratio_btw_closed_granted_loans.columns

Index(['digitalLoanAccountId', 'tot_closed_amt_granted_6M',
       'tot_closed_cnt_granted_6M', 'tot_closed_amt_granted_12M',
       'tot_closed_cnt_granted_12M', 'tot_closed_amt_granted_24M',
       'tot_closed_cnt_granted_24M', 'total_cnt_granted_6M',
       'total_amt_granted_6M', 'total_cnt_granted_12M',
       'total_amt_granted_12M', 'total_cnt_granted_24M',
       'total_amt_granted_24M', 'ratio_closed_over_new_granted_cnt_6M',
       'ratio_closed_over_new_granted_cnt_12M',
       'ratio_closed_over_new_granted_cnt_24M',
       'ratio_closed_over_new_granted_amt_6M',
       'ratio_closed_over_new_granted_amt_12M',
       'ratio_closed_over_new_granted_amt_24M'],
      dtype='object')

In [167]:
duplicateincolumn(ratio_btw_closed_granted_loans, 'digitalLoanAccountId')

No duplicates found in the digitalLoanAccountId column


In [168]:
ratio_btw_closed_granted_loans = ratio_btw_closed_granted_loans[['digitalLoanAccountId','ratio_closed_over_new_granted_cnt_6M',
                                                                                   'ratio_closed_over_new_granted_cnt_12M',
                                                                                   'ratio_closed_over_new_granted_cnt_24M',
                                                                                   'ratio_closed_over_new_granted_amt_6M',
                                                                                   'ratio_closed_over_new_granted_amt_12M',
                                                                                   'ratio_closed_over_new_granted_amt_24M']]

In [169]:
ratio_btw_closed_granted_loans.head()

,digitalLoanAccountId,ratio_closed_over_new_granted_cnt_6M,ratio_closed_over_new_granted_cnt_12M,ratio_closed_over_new_granted_cnt_24M,ratio_closed_over_new_granted_amt_6M,ratio_closed_over_new_granted_amt_12M,ratio_closed_over_new_granted_amt_24M
0,0000181e-f0bc-414d-bbc1-69fd841f4bc1,NaN,2.0,1.333333,NaN,0.500000,0.750000
1,00025b22-70c8-4d93-ad59-4a58f5c11d56,NaN,2.0,1.142857,NaN,0.708115,0.920749
2,000a6b2d-c9db-4d5a-9ef2-0f37987301f3,NaN,NaN,1.000000,NaN,NaN,1.000000
3,000b9c72-3e95-4c0c-8e4c-b5c57a0bd0ff,NaN,NaN,1.000000,NaN,NaN,1.000000
4,000d27e2-705e-47e2-886d-542a43c0889b,NaN,NaN,1.500000,NaN,NaN,0.370079


In [170]:
duplicateincolumn(ratio_btw_closed_granted_loans, 'digitalLoanAccountId')

No duplicates found in the digitalLoanAccountId column


### Ratio of Chargedamount/SanctionedAmount

In [171]:
import pandas as pd

# Convert columns to numeric, coercing errors to NaN
granted_loans_365days['ChargedAmount'] = pd.to_numeric(granted_loans_365days['ChargedAmount'], errors='coerce')
granted_loans_365days['SanctionedAmount'] = pd.to_numeric(granted_loans_365days['SanctionedAmount'], errors='coerce')

granted_loans_180days['ChargedAmount'] = pd.to_numeric(granted_loans_180days['ChargedAmount'], errors='coerce')
granted_loans_180days['SanctionedAmount'] = pd.to_numeric(granted_loans_180days['SanctionedAmount'], errors='coerce')

granted_loans_730days['ChargedAmount'] = pd.to_numeric(granted_loans_730days['ChargedAmount'], errors='coerce')
granted_loans_730days['SanctionedAmount'] = pd.to_numeric(granted_loans_730days['SanctionedAmount'], errors='coerce')

# Calculate the ratio
granted_loans_365days['Ratio_of_ChargedAmout'] = (granted_loans_365days['ChargedAmount'] / granted_loans_365days['SanctionedAmount']).round(4)
granted_loans_180days['Ratio_of_ChargedAmout'] = (granted_loans_180days['ChargedAmount'] / granted_loans_180days['SanctionedAmount']).round(4)
granted_loans_730days['Ratio_of_ChargedAmout'] = (granted_loans_730days['ChargedAmount'] / granted_loans_730days['SanctionedAmount']).round(4)

# Optional: Drop rows where the ratio could not be calculated (i.e., where SanctionedAmount was NaN or 0)
granted_loans_365days = granted_loans_365days.dropna(subset=['Ratio_of_ChargedAmout'])
granted_loans_180days = granted_loans_180days.dropna(subset=['Ratio_of_ChargedAmout'])
granted_loans_730days = granted_loans_730days.dropna(subset=['Ratio_of_ChargedAmout'])


In [172]:
ratio_charged_on_sanctioned_12M = granted_loans_365days.groupby(['digitalLoanAccountId']).agg(
                                        min_ratio_charged_on_sanctioned_12M=('Ratio_of_ChargedAmout', 'min'),
                                        max_ratio_charged_on_sanctioned_12M=('Ratio_of_ChargedAmout', 'max'),
                                        avg_ratio_charged_on_sanctioned_12M=('Ratio_of_ChargedAmout', 'mean')).reset_index()

ratio_charged_on_sanctioned_12M.head()

,digitalLoanAccountId,min_ratio_charged_on_sanctioned_12M,max_ratio_charged_on_sanctioned_12M,avg_ratio_charged_on_sanctioned_12M
0,002c7453-5836-4ee5-a590-6b037fa423ff,0.1627,0.1627,0.1627
1,0049b50f-a05e-48da-b8f2-42477d6301ae,0.1627,0.1627,0.1627
2,00bf6bfa-3c51-4173-b34b-08cd8a3a234a,0.0000,0.0000,0.0000
3,00f5147b-5591-4602-af90-cbd639ad4a1e,0.0000,0.0000,0.0000
4,0145a46e-3a58-47de-9eff-4c7bf32fb51e,0.1992,0.1992,0.1992


In [173]:
ratio_charged_on_sanctioned_6M = granted_loans_180days.groupby(['digitalLoanAccountId']).agg(
                                        min_ratio_charged_on_sanctioned_6M=('Ratio_of_ChargedAmout', 'min'),
                                        max_ratio_charged_on_sanctioned_6M=('Ratio_of_ChargedAmout', 'max'),
                                        avg_ratio_charged_on_sanctioned_6M=('Ratio_of_ChargedAmout', 'mean')).reset_index()

ratio_charged_on_sanctioned_6M.head()

,digitalLoanAccountId,min_ratio_charged_on_sanctioned_6M,max_ratio_charged_on_sanctioned_6M,avg_ratio_charged_on_sanctioned_6M
0,002c7453-5836-4ee5-a590-6b037fa423ff,0.1627,0.1627,0.1627
1,0049b50f-a05e-48da-b8f2-42477d6301ae,0.1627,0.1627,0.1627
2,02112690-e1fa-427a-9972-12755a15ce9e,0.0175,0.0175,0.0175
3,0826815b-af39-41c9-8ae1-9b193c014963,0.0000,0.0000,0.0000
4,093ea2d9-f209-4a9d-8a14-363f9b6cb33a,0.0000,0.0000,0.0000


In [174]:
ratio_charged_on_sanctioned_24M = granted_loans_730days.groupby(['digitalLoanAccountId']).agg(
                                        min_ratio_charged_on_sanctioned_24M=('Ratio_of_ChargedAmout', 'min'),
                                        max_ratio_charged_on_sanctioned_24M=('Ratio_of_ChargedAmout', 'max'),
                                        avg_ratio_charged_on_sanctioned_24M=('Ratio_of_ChargedAmout', 'mean')).reset_index()

ratio_charged_on_sanctioned_24M.head()

,digitalLoanAccountId,min_ratio_charged_on_sanctioned_24M,max_ratio_charged_on_sanctioned_24M,avg_ratio_charged_on_sanctioned_24M
0,002c7453-5836-4ee5-a590-6b037fa423ff,0.1627,0.1627,0.1627
1,0047123b-b775-4881-a42d-b886d126608d,2.3628,2.3628,2.3628
2,0049b50f-a05e-48da-b8f2-42477d6301ae,0.1627,0.1627,0.1627
3,008fb1d5-e577-4535-98a9-df3bbe5f19b0,0.0024,0.0330,0.0177
4,00bf6bfa-3c51-4173-b34b-08cd8a3a234a,0.0000,0.0000,0.0000


### Avg, min, max installments

In [175]:
installments_days_6M = granted_loans_180days.groupby(['digitalLoanAccountId']).agg(
                                        min_no_of_installments_6M=('InstallmentsNumber', 'min'),
                                        max_no_of_installments_6M=('InstallmentsNumber', 'max'),
                                        avg_no_of_installments_6M=('InstallmentsNumber', 'mean')).reset_index()

In [176]:
installments_days_12M = granted_loans_365days.groupby(['digitalLoanAccountId']).agg(
                                        min_no_of_installments_12M=('InstallmentsNumber', 'min'),
                                        max_no_of_installments_12M=('InstallmentsNumber', 'max'),
                                        avg_no_of_installments_12M=('InstallmentsNumber', 'mean')).reset_index()

In [177]:
installments_days_24M = granted_loans_730days.groupby(['digitalLoanAccountId']).agg(
                                        min_no_of_installments_24M=('InstallmentsNumber', 'min'),
                                        max_no_of_installments_24M=('InstallmentsNumber', 'max'),
                                        avg_no_of_installments_24M=('InstallmentsNumber', 'mean')).reset_index()

In [178]:
Historical_feat_granted = granted_loan_cnt_by_segment.merge(granted_loan_cnt_df, on ='digitalLoanAccountId', how ='outer')
Historical_feat_granted = Historical_feat_granted.merge(granted_loan_amt_by_segment, on ='digitalLoanAccountId', how ='outer')
Historical_feat_granted = Historical_feat_granted.merge(granted_loan_amt_df, on ='digitalLoanAccountId', how ='outer')
Historical_feat_granted = Historical_feat_granted.merge(grannted_sanctioned_amt_12M, on=['digitalLoanAccountId'], how='outer')
Historical_feat_granted = Historical_feat_granted.merge(grannted_sanctioned_amt_24M, on=['digitalLoanAccountId'], how='outer')
Historical_feat_granted = Historical_feat_granted.merge(grannted_sanctioned_amt_6M, on=['digitalLoanAccountId'], how='outer')
Historical_feat_granted = Historical_feat_granted.merge(granted_loan_datediff_12M_df, on=['digitalLoanAccountId'], how='outer')
Historical_feat_granted = Historical_feat_granted.merge(foreclosure_cnt_6M, on=['digitalLoanAccountId'], how='outer')
Historical_feat_granted = Historical_feat_granted.merge(foreclosure_cnt_12M, on=['digitalLoanAccountId'], how='outer')
Historical_feat_granted = Historical_feat_granted.merge(foreclosure_cnt_24M, on=['digitalLoanAccountId'], how='outer')
Historical_feat_granted = Historical_feat_granted.merge(cnt_writtenoff_ever, on=['digitalLoanAccountId'], how='outer')
Historical_feat_granted = Historical_feat_granted.merge(vel_contract_granted_df, on=['digitalLoanAccountId'], how='outer')
Historical_feat_granted = Historical_feat_granted.merge(vel_contract_granted_closed_df, on=['digitalLoanAccountId'], how='outer')
Historical_feat_granted = Historical_feat_granted.merge(credit_limit_12mon_df, on=['digitalLoanAccountId'], how='outer')
Historical_feat_granted = Historical_feat_granted.merge(FinancedAmt_12mon_df, on=['digitalLoanAccountId'], how='outer')
Historical_feat_granted = Historical_feat_granted.merge(ratio_btw_closed_granted_loans, on=['digitalLoanAccountId'], how='outer')
Historical_feat_granted = Historical_feat_granted.merge(ratio_charged_on_sanctioned_6M, on=['digitalLoanAccountId'], how='outer')
Historical_feat_granted = Historical_feat_granted.merge(ratio_charged_on_sanctioned_12M, on=['digitalLoanAccountId'], how='outer')
Historical_feat_granted = Historical_feat_granted.merge(ratio_charged_on_sanctioned_24M, on=['digitalLoanAccountId'], how='outer')
Historical_feat_granted = Historical_feat_granted.merge(installments_days_6M, on=['digitalLoanAccountId'], how='outer')
Historical_feat_granted = Historical_feat_granted.merge(installments_days_12M, on=['digitalLoanAccountId'], how='outer')
Historical_feat_granted = Historical_feat_granted.merge(installments_days_24M, on=['digitalLoanAccountId'], how='outer')


In [179]:
Historical_feat_granted.columns.values

array(['digitalLoanAccountId', 'Credit Cards_granted_contracts_cnt_6M',
       'Credit Lines_granted_contracts_cnt_6M',
       'Home Equity Loans_granted_contracts_cnt_6M',
       'Other Loans_granted_contracts_cnt_6M',
       'Personal Loans_granted_contracts_cnt_6M',
       'Real Estate Loans_granted_contracts_cnt_6M',
       'Short and Term Loans_granted_contracts_cnt_6M',
       'Time Loans_granted_contracts_cnt_6M',
       'Trust Loans_granted_contracts_cnt_6M',
       'Credit Cards_granted_contracts_cnt_12M',
       'Credit Lines_granted_contracts_cnt_12M',
       'Home Equity Loans_granted_contracts_cnt_12M',
       'Other Loans_granted_contracts_cnt_12M',
       'Personal Loans_granted_contracts_cnt_12M',
       'Real Estate Loans_granted_contracts_cnt_12M',
       'Short and Term Loans_granted_contracts_cnt_12M',
       'Time Loans_granted_contracts_cnt_12M',
       'Trust Loans_granted_contracts_cnt_12M',
       'Credit Cards_granted_contracts_cnt_24M',
       'Credit Lines_g

In [180]:
duplicateincolumn(Historical_feat_granted, 'digitalLoanAccountId')

No duplicates found in the digitalLoanAccountId column


## Historical Features for Non Granted 

### Non granted loans cnt and amt

In [181]:
data['ContractRequestDate'] = pd.to_datetime(data['ContractRequestDate']).dt.date

non_granted_loans = data[data['SOURCE']=='nongranted']

nongranted_loans_90days = non_granted_loans[(non_granted_loans['ContractRequestDate'] >= non_granted_loans['threshold_90days']) & (non_granted_loans['ContractRequestDate'] <= non_granted_loans['run_date'])]
nongranted_loans_180days = non_granted_loans[(non_granted_loans['ContractRequestDate'] >= non_granted_loans['threshold_180days']) & (non_granted_loans['ContractRequestDate'] <= non_granted_loans['run_date'])]
nongranted_loans_365days = non_granted_loans[(non_granted_loans['ContractRequestDate'] >= non_granted_loans['threshold_365days']) & (non_granted_loans['ContractRequestDate'] <= non_granted_loans['run_date'])]
nongranted_loans_730days = non_granted_loans[(non_granted_loans['ContractRequestDate'] >= non_granted_loans['threshold_730days']) & (non_granted_loans['ContractRequestDate'] <= non_granted_loans['run_date'])]




In [182]:
nongranted_loans_3M =nongranted_loans_90days.groupby(['digitalLoanAccountId']).agg(
                                         cnt_nongranted_contracts_3M=('CBContractCode', 'nunique'),
                                        amt_nongranted_contracts_3M=('SanctionedAmount', 'sum')).reset_index()

nongranted_loans_3M.head(3)

,digitalLoanAccountId,cnt_nongranted_contracts_3M,amt_nongranted_contracts_3M
0,0003edd4-5a8c-4e00-bfeb-3fffc587d38b,1,0
1,00087e49-c94a-482d-831a-8fd762f28ca5,1,12
2,000cf231-b6da-4bff-a27e-0880c86c7bbd,1,12


In [183]:
nongranted_loans_6M =nongranted_loans_180days.groupby(['digitalLoanAccountId']).agg(
                                         cnt_nongranted_contracts_6M=('CBContractCode', 'nunique'),
                                        amt_nongranted_contracts_6M=('SanctionedAmount', 'sum')).reset_index()

nongranted_loans_6M.head(3)

,digitalLoanAccountId,cnt_nongranted_contracts_6M,amt_nongranted_contracts_6M
0,00025b22-70c8-4d93-ad59-4a58f5c11d56,1,0
1,0003edd4-5a8c-4e00-bfeb-3fffc587d38b,1,0
2,00087e49-c94a-482d-831a-8fd762f28ca5,1,12


In [184]:
nongranted_loans_12M =nongranted_loans_365days.groupby(['digitalLoanAccountId']).agg(
                                         cnt_nongranted_contracts_12M=('CBContractCode', 'nunique'),
                                        amt_nongranted_contracts_12M=('SanctionedAmount', 'sum')).reset_index()
nongranted_loans_12M.head()

,digitalLoanAccountId,cnt_nongranted_contracts_12M,amt_nongranted_contracts_12M
0,00025b22-70c8-4d93-ad59-4a58f5c11d56,1,0
1,0003edd4-5a8c-4e00-bfeb-3fffc587d38b,1,0
2,00087e49-c94a-482d-831a-8fd762f28ca5,1,12
3,000a3bee-c57d-4647-beaa-f8d3293b4101,1,0
4,000cf231-b6da-4bff-a27e-0880c86c7bbd,1,12


In [185]:
nongranted_loans_24M =nongranted_loans_730days.groupby(['digitalLoanAccountId']).agg(
                                         cnt_nongranted_contracts_24M=('CBContractCode', 'nunique'),
                                        amt_nongranted_contracts_24M=('SanctionedAmount', 'sum')).reset_index()
nongranted_loans_24M.head()

,digitalLoanAccountId,cnt_nongranted_contracts_24M,amt_nongranted_contracts_24M
0,00025b22-70c8-4d93-ad59-4a58f5c11d56,1,0
1,0003edd4-5a8c-4e00-bfeb-3fffc587d38b,1,0
2,00087e49-c94a-482d-831a-8fd762f28ca5,1,12
3,000a3bee-c57d-4647-beaa-f8d3293b4101,1,0
4,000cf231-b6da-4bff-a27e-0880c86c7bbd,1,12


### Avg and Med time difference between nongranted loans

In [186]:
nongranted_loan_datediff_12M= nongranted_loans_365days[['digitalLoanAccountId','ContractRequestDate','CBContractCode']]
nongranted_loan_datediff_12M = nongranted_loan_datediff_12M.sort_values(by=['digitalLoanAccountId','ContractRequestDate','CBContractCode'])


nongranted_loan_datediff_12M['ContractRequestDate'] = pd.to_datetime(nongranted_loan_datediff_12M['ContractRequestDate'])

nongranted_loan_datediff_12M['day_diff'] = nongranted_loan_datediff_12M.groupby(['digitalLoanAccountId'])['ContractRequestDate'].diff().dt.days


nongranted_loan_datediff_12M_sorted = nongranted_loan_datediff_12M.dropna(subset=['day_diff'])

# Sum the differences for each mobile number
nongranted_loan_datediff_12M_df = nongranted_loan_datediff_12M_sorted.groupby(['digitalLoanAccountId']).agg(
                                                        no_of_difference=('ContractRequestDate', 'count'),
                                                        Tot_days_bw_contracts_12M=('day_diff', 'sum'),
                                                        med_days_bw_contracts_12M=('day_diff', 'median')                                
                                                                ).reset_index()
    
    

# Repayment_cashin.columns = ['mobile_num','no_of_difference', 'Total_cashin_datediff_180days','Median_cashin_datediff_180days','Min_cashin_datediff_180days','Max_cashin_datediff_180days']
nongranted_loan_datediff_12M_df['avg_days_bw_contracts_12M'] =( nongranted_loan_datediff_12M_df['Tot_days_bw_contracts_12M']/nongranted_loan_datediff_12M_df['no_of_difference']).round(2)

nongranted_loan_datediff_12M_df= nongranted_loan_datediff_12M_df[['digitalLoanAccountId','med_days_bw_contracts_12M','avg_days_bw_contracts_12M','Tot_days_bw_contracts_12M']]

nongranted_loan_datediff_12M_df.head()

,digitalLoanAccountId,med_days_bw_contracts_12M,avg_days_bw_contracts_12M,Tot_days_bw_contracts_12M
0,000d27e2-705e-47e2-886d-542a43c0889b,3.0,3.0,3.0
1,000ef884-bb3d-4e34-80d7-b4f7f1648d23,205.0,205.0,205.0
2,001c77ba-72f9-4c44-a08d-4de9db9f0539,106.0,106.0,106.0
3,003972ae-7dce-4d8c-9ab7-2ee6f0d01adc,0.0,102.0,306.0
4,004503c8-1892-490c-9838-621d41f61ec2,163.0,163.0,163.0


### Velocity loans nongranted contracts

In [187]:
# Convert 'SanctionedAmount' to numeric, coercing errors to NaN using .loc
nongranted_loans_730days.loc[:, 'SanctionedAmount'] = pd.to_numeric(nongranted_loans_730days['SanctionedAmount'], errors='coerce')
nongranted_loans_365days.loc[:, 'SanctionedAmount'] = pd.to_numeric(nongranted_loans_365days['SanctionedAmount'], errors='coerce')
nongranted_loans_180days.loc[:, 'SanctionedAmount'] = pd.to_numeric(nongranted_loans_180days['SanctionedAmount'], errors='coerce')
nongranted_loans_90days.loc[:, 'SanctionedAmount'] = pd.to_numeric(nongranted_loans_90days['SanctionedAmount'], errors='coerce')

# Now perform your groupby operations
nongranted_amt_dfvelocity_24M = nongranted_loans_730days.groupby(['digitalLoanAccountId']).agg(
    avg_amt_nongranted_24M=('SanctionedAmount', 'mean'),
    avg_cnt_nongranted_24M=('CBContractCode', 'nunique')
).reset_index()

nongranted_amt_dfvelocity_12M = nongranted_loans_365days.groupby(['digitalLoanAccountId']).agg(
    avg_amt_nongranted_12M=('SanctionedAmount', 'mean'),
    avg_cnt_nongranted_12M=('CBContractCode', 'nunique')
).reset_index()

nongranted_amt_dfvelocity_6M = nongranted_loans_180days.groupby(['digitalLoanAccountId']).agg(
    avg_amt_nongranted_6M=('SanctionedAmount', 'mean'),
    avg_cnt_nongranted_6M=('CBContractCode', 'nunique')
).reset_index()

nongranted_amt_dfvelocity_3M = nongranted_loans_90days.groupby(['digitalLoanAccountId']).agg(
    avg_amt_nongranted_3M=('SanctionedAmount', 'mean'),
    avg_cnt_nongranted_3M=('CBContractCode', 'nunique')
).reset_index()


In [188]:
nongranted_amt_dfvelocity_24M['avg_cnt_nongranted_24M'] =(nongranted_amt_dfvelocity_24M['avg_cnt_nongranted_24M']/24).round(3)
nongranted_amt_dfvelocity_12M['avg_cnt_nongranted_12M'] =(nongranted_amt_dfvelocity_12M['avg_cnt_nongranted_12M']/12).round(3)
nongranted_amt_dfvelocity_6M['avg_cnt_nongranted_6M'] =(nongranted_amt_dfvelocity_6M['avg_cnt_nongranted_6M']/6).round(3)
nongranted_amt_dfvelocity_3M['avg_cnt_nongranted_3M'] =(nongranted_amt_dfvelocity_3M['avg_cnt_nongranted_3M']/3).round(3)

nongranted_amt_dfvelocity_24M.head()

,digitalLoanAccountId,avg_amt_nongranted_24M,avg_cnt_nongranted_24M
0,00025b22-70c8-4d93-ad59-4a58f5c11d56,NaN,0.042
1,0003edd4-5a8c-4e00-bfeb-3fffc587d38b,NaN,0.042
2,00087e49-c94a-482d-831a-8fd762f28ca5,12.0,0.042
3,000a3bee-c57d-4647-beaa-f8d3293b4101,NaN,0.042
4,000cf231-b6da-4bff-a27e-0880c86c7bbd,12.0,0.042


In [189]:
vel_contract_nongranted_df = pd.merge(nongranted_amt_dfvelocity_3M, nongranted_amt_dfvelocity_6M, on='digitalLoanAccountId', how='outer')
vel_contract_nongranted_df = pd.merge(vel_contract_nongranted_df, nongranted_amt_dfvelocity_12M, on='digitalLoanAccountId', how='outer')
vel_contract_nongranted_df = pd.merge(vel_contract_nongranted_df, nongranted_amt_dfvelocity_24M, on='digitalLoanAccountId', how='outer')
vel_contract_nongranted_df.head(3)

,digitalLoanAccountId,avg_amt_nongranted_3M,avg_cnt_nongranted_3M,avg_amt_nongranted_6M,avg_cnt_nongranted_6M,avg_amt_nongranted_12M,avg_cnt_nongranted_12M,avg_amt_nongranted_24M,avg_cnt_nongranted_24M
0,00025b22-70c8-4d93-ad59-4a58f5c11d56,NaN,NaN,NaN,0.167,NaN,0.083,NaN,0.042
1,0003edd4-5a8c-4e00-bfeb-3fffc587d38b,NaN,0.333,NaN,0.167,NaN,0.083,NaN,0.042
2,00087e49-c94a-482d-831a-8fd762f28ca5,12.0,0.333,12.0,0.167,12.0,0.083,12.0,0.042


In [190]:
vel_contract_nongranted_df['vel_contract_nongranted_cnt_3on12'] = vel_contract_nongranted_df['avg_cnt_nongranted_3M']/vel_contract_nongranted_df['avg_cnt_nongranted_12M']
vel_contract_nongranted_df['vel_contract_nongranted_cnt_6on12'] = vel_contract_nongranted_df['avg_cnt_nongranted_6M']/vel_contract_nongranted_df['avg_cnt_nongranted_12M']
vel_contract_nongranted_df['vel_contract_nongranted_cnt_12on24'] = vel_contract_nongranted_df['avg_cnt_nongranted_12M']/vel_contract_nongranted_df['avg_cnt_nongranted_24M']

### Velocity loan amount nongranted contracts

In [191]:
def safe_divide(numerator, denominator):
    """Return the result of division if denominator is not zero; otherwise, return NaN."""
    return numerator / denominator if denominator != 0 else np.nan

# Calculate ratios while avoiding division by zero
vel_contract_nongranted_df['vel_contract_nongranted_amt_3on12'] = vel_contract_nongranted_df.apply(
    lambda row: safe_divide(row['avg_amt_nongranted_3M'], row['avg_amt_nongranted_12M']),
    axis=1
)

vel_contract_nongranted_df['vel_contract_nongranted_amt_6on12'] = vel_contract_nongranted_df.apply(
    lambda row: safe_divide(row['avg_amt_nongranted_6M'], row['avg_amt_nongranted_12M']),
    axis=1
)

vel_contract_nongranted_df['vel_contract_nongranted_amt_12on24'] = vel_contract_nongranted_df.apply(
    lambda row: safe_divide(row['avg_amt_nongranted_12M'], row['avg_amt_nongranted_24M']),
    axis=1
)

# Print the updated DataFrame
print(vel_contract_nongranted_df)


                       digitalLoanAccountId avg_amt_nongranted_3M  \
0      00025b22-70c8-4d93-ad59-4a58f5c11d56                   NaN   
1      0003edd4-5a8c-4e00-bfeb-3fffc587d38b                   NaN   
2      00087e49-c94a-482d-831a-8fd762f28ca5                  12.0   
3      000a3bee-c57d-4647-beaa-f8d3293b4101                   NaN   
4      000cf231-b6da-4bff-a27e-0880c86c7bbd                  12.0   
...                                     ...                   ...   
22990  ffe83ac1-20d7-42e1-a0b3-1ab38591d46d                1000.0   
22991  ffe862e2-8bdc-44f3-a9b2-7de5a3e904b4                   6.0   
22992  ffea20a6-bb45-4f73-b846-6d439870be70                4700.0   
22993  fff93cc7-530c-4641-ab71-15e7e7ac729c              103000.0   
22994  fffe95d2-1a9b-4b5e-85a4-356edd0c677b                   NaN   

       avg_cnt_nongranted_3M avg_amt_nongranted_6M  avg_cnt_nongranted_6M  \
0                        NaN                   NaN                  0.167   
1                

In [192]:
vel_contract_nongranted_df.columns

Index(['digitalLoanAccountId', 'avg_amt_nongranted_3M',
       'avg_cnt_nongranted_3M', 'avg_amt_nongranted_6M',
       'avg_cnt_nongranted_6M', 'avg_amt_nongranted_12M',
       'avg_cnt_nongranted_12M', 'avg_amt_nongranted_24M',
       'avg_cnt_nongranted_24M', 'vel_contract_nongranted_cnt_3on12',
       'vel_contract_nongranted_cnt_6on12',
       'vel_contract_nongranted_cnt_12on24',
       'vel_contract_nongranted_amt_3on12',
       'vel_contract_nongranted_amt_6on12',
       'vel_contract_nongranted_amt_12on24'],
      dtype='object')

In [193]:
duplicateincolumn(vel_contract_nongranted_df, 'digitalLoanAccountId')

No duplicates found in the digitalLoanAccountId column


In [194]:
vel_contract_nongranted_df = vel_contract_nongranted_df[['digitalLoanAccountId','vel_contract_nongranted_cnt_3on12',
                                                   'vel_contract_nongranted_cnt_6on12',
                                                   'vel_contract_nongranted_cnt_12on24',
                                                   'vel_contract_nongranted_amt_3on12',
                                                   'vel_contract_nongranted_amt_6on12',
                                                   'vel_contract_nongranted_amt_12on24']]

vel_contract_nongranted_df.head()

,digitalLoanAccountId,vel_contract_nongranted_cnt_3on12,vel_contract_nongranted_cnt_6on12,vel_contract_nongranted_cnt_12on24,vel_contract_nongranted_amt_3on12,vel_contract_nongranted_amt_6on12,vel_contract_nongranted_amt_12on24
0,00025b22-70c8-4d93-ad59-4a58f5c11d56,NaN,2.012048,1.97619,NaN,NaN,NaN
1,0003edd4-5a8c-4e00-bfeb-3fffc587d38b,4.012048,2.012048,1.97619,NaN,NaN,NaN
2,00087e49-c94a-482d-831a-8fd762f28ca5,4.012048,2.012048,1.97619,1.0,1.0,1.0
3,000a3bee-c57d-4647-beaa-f8d3293b4101,NaN,NaN,1.97619,NaN,NaN,NaN
4,000cf231-b6da-4bff-a27e-0880c86c7bbd,4.012048,2.012048,1.97619,1.0,1.0,1.0


In [195]:
Historical_feat_nongranted = nongranted_loans_3M.merge(nongranted_loans_6M, on=['digitalLoanAccountId'], how ='outer')
Historical_feat_nongranted = nongranted_loans_3M.merge(nongranted_loans_12M, on=['digitalLoanAccountId'], how ='outer')
Historical_feat_nongranted = nongranted_loans_3M.merge(nongranted_loans_24M, on=['digitalLoanAccountId'], how ='outer')
Historical_feat_nongranted = nongranted_loans_3M.merge(nongranted_loan_datediff_12M_df, on=['digitalLoanAccountId'], how ='outer')
Historical_feat_nongranted = nongranted_loans_3M.merge(vel_contract_nongranted_df, on=['digitalLoanAccountId'], how ='outer')


In [196]:
duplicateincolumn(Historical_feat_nongranted, 'digitalLoanAccountId')

No duplicates found in the digitalLoanAccountId column


## Historical features for Granted and Non Granted Combined

## Ratio of Co-Borrow cnt and Amt

In [197]:

coborrow_df_24M = granted_loans_730days[granted_loans_730days['RoleDesc'] =='Co-Borrower']

coborrow_ratio_f_24M = coborrow_df_24M.groupby(['digitalLoanAccountId']).agg(
                                        cnt_contract_coborrower_24M =('CBContractCode', 'count'),
                                        amt_contract_coborrower_24M=('SanctionedAmount', 'sum'),
                                            ).reset_index()



coborrow_ratio_f_24M.head(5)


,digitalLoanAccountId,cnt_contract_coborrower_24M,amt_contract_coborrower_24M


In [198]:

grantedloans_ratio_f_24M = granted_loans_730days.groupby(['digitalLoanAccountId']).agg(
                                        cnt_granted_loans_24M =('CBContractCode', 'count'),
                                        amt_granted_loans_24M=('SanctionedAmount', 'sum'),
                                            ).reset_index()



grantedloans_ratio_f_24M.head(5)


,digitalLoanAccountId,cnt_granted_loans_24M,amt_granted_loans_24M
0,002c7453-5836-4ee5-a590-6b037fa423ff,1,21000
1,0047123b-b775-4881-a42d-b886d126608d,1,45000
2,0049b50f-a05e-48da-b8f2-42477d6301ae,1,21000
3,008fb1d5-e577-4535-98a9-df3bbe5f19b0,2,410000
4,00bf6bfa-3c51-4173-b34b-08cd8a3a234a,1,41000


In [199]:
ratio_coborrow_vs_grantedloans = coborrow_ratio_f_24M.merge(grantedloans_ratio_f_24M, on='digitalLoanAccountId')
ratio_coborrow_vs_grantedloans['ratio_coborrow_contracts_cnt_24M'] =  ratio_coborrow_vs_grantedloans['cnt_contract_coborrower_24M']/ratio_coborrow_vs_grantedloans['cnt_granted_loans_24M']
ratio_coborrow_vs_grantedloans['ratio_coborrow_contracts_amt_24M'] =  ratio_coborrow_vs_grantedloans['amt_contract_coborrower_24M']/ratio_coborrow_vs_grantedloans['amt_granted_loans_24M']

ratio_coborrow_vs_grantedloans = ratio_coborrow_vs_grantedloans[['digitalLoanAccountId','ratio_coborrow_contracts_cnt_24M','ratio_coborrow_contracts_amt_24M']]

### applicant_name_match_score

In [200]:
from fuzzywuzzy import fuzz
# Handle missing values by filling them with empty strings
data['Firstname'] = data['Firstname'].fillna('')
data['middleName'] = data['middleName'].fillna('')
data['LastName'] = data['LastName'].fillna('')
data['LinkedSubject_Name'] = data['LinkedSubject_Name'].fillna('')
 
# Create the FullName column by concatenating Firstname, middleName, and LastName
data['FullName'] = data['Firstname'] + ' ' + data['middleName'] + ' ' + data['LastName']
 
# Ensure all values are strings
data['FullName'] = data['FullName'].astype(str)
data['LinkedSubject_Name'] = data['LinkedSubject_Name'].astype(str)
 
# Function to calculate fuzzy match score with set matching
def calculate_match_score(name1, name2):
    # If either name is empty, return -9
    if not name1.strip() or not name2.strip():
        return -9
    return fuzz.token_set_ratio(name1, name2)
 
# Apply the function to calculate match scores for FullName vs LinkedSubject_Name
data['applicant_name_match_score'] = data.apply(lambda row: calculate_match_score(row['FullName'], row['LinkedSubject_Name']), axis=1)

c:\Users\DwaipayanChakroborti\Myenv\reporting\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [201]:
applicant_name_match_score = data[['digitalLoanAccountId', 'applicant_name_match_score']].drop_duplicates(keep='first')
applicant_name_match_score.head()

,digitalLoanAccountId,applicant_name_match_score
0,008a74eb-d9e9-4150-94ce-39c7d1333dfd,-9
21,008fb1d5-e577-4535-98a9-df3bbe5f19b0,-9
29,00efd0eb-e027-4e25-b0a3-74c6e5de690b,-9
31,00f21238-a8a4-4b29-9980-52f7f6866c7b,-9
34,014aac2b-39d9-42f3-9b70-1919db85e7c6,-9


In [202]:
duplicateincolumn(applicant_name_match_score, 'digitalLoanAccountId')

There are duplicates in the digitalLoanAccountId column


### applicant_employment_match_score

In [203]:
from fuzzywuzzy import fuzz
# Handle missing values by filling them with empty strings
data['natureofwork'] = data['natureofwork'].fillna('')
data['PSICDesc'] = data['PSICDesc'].fillna('')
 
# Ensure all values are strings
data['natureofwork'] = data['natureofwork'].astype(str)
data['PSICDesc'] = data['PSICDesc'].astype(str)
 
# Function to calculate fuzzy match score with set matching
def calculate_match_score(name1, name2):
    # If either name is empty, return -9
    if not name1.strip() or not name2.strip():
        return -9
    return fuzz.token_set_ratio(name1, name2)
 
# Apply the function to calculate match scores
data['applicant_employment_match_score'] = data.apply(lambda row: calculate_match_score(row['natureofwork'], row['PSICDesc']), axis=1)
 
applicant_employment_match_score = data[['digitalLoanAccountId','applicant_employment_match_score']].drop_duplicates(keep='first')

In [204]:
applicant_employment_match_score.head()

,digitalLoanAccountId,applicant_employment_match_score
0,008a74eb-d9e9-4150-94ce-39c7d1333dfd,-9
21,008fb1d5-e577-4535-98a9-df3bbe5f19b0,-9
29,00efd0eb-e027-4e25-b0a3-74c6e5de690b,-9
31,00f21238-a8a4-4b29-9980-52f7f6866c7b,-9
34,014aac2b-39d9-42f3-9b70-1919db85e7c6,-9


In [205]:
duplicateincolumn(applicant_employment_match_score, 'digitalLoanAccountId')

No duplicates found in the digitalLoanAccountId column


In [206]:
import re
# Fill NAs and concatenate fields
data['subIndustryDescription'] = data['subIndustryDescription'].fillna(' ').astype(str)
data['industryDescription'] = data['industryDescription'].fillna(' ').astype(str)
data['OccupationDesc'] = data['OccupationDesc'].fillna(' ').astype(str)
data['OccupationStatusDesc'] = data['OccupationStatusDesc'].fillna(' ').astype(str)
 
data['industryDerived'] = data['subIndustryDescription'] + ' ' + data['industryDescription']
data['occupationDerived'] = data['OccupationDesc'] + ' ' + data['OccupationStatusDesc']
 
# Remove numbers from strings
def remove_numbers(s):
    return re.sub(r'\d+', '', s)
 
data['industryDerived'] = data['industryDerived'].apply(remove_numbers)
data['occupationDerived'] = data['occupationDerived'].apply(remove_numbers)
 
# Function to calculate fuzzy match score
def calculate_match_score(name1, name2):
    if not name1.strip() or not name2.strip():
        return -9
    return fuzz.token_set_ratio(name1, name2)
 
# Apply function to calculate match scores
data['industry_employment_match_score'] = data.apply(
    lambda row: calculate_match_score(row['industryDerived'], row['occupationDerived']),
    axis=1
)
 
# Merge the results with the existing DataFrame
applicant_employment_match_score = pd.merge(
    applicant_employment_match_score,
    data[['digitalLoanAccountId', 'industry_employment_match_score']],
    on='digitalLoanAccountId',
    how='left'
)
applicant_employment_match_score = applicant_employment_match_score.drop_duplicates()
 
# Display the updated DataFrame
applicant_employment_match_score.head()

,digitalLoanAccountId,applicant_employment_match_score,industry_employment_match_score
0,008a74eb-d9e9-4150-94ce-39c7d1333dfd,-9,-9
21,008fb1d5-e577-4535-98a9-df3bbe5f19b0,-9,-9
29,00efd0eb-e027-4e25-b0a3-74c6e5de690b,-9,-9
31,00f21238-a8a4-4b29-9980-52f7f6866c7b,-9,-9
34,014aac2b-39d9-42f3-9b70-1919db85e7c6,-9,44


In [207]:
Historical_feat_combined = ratio_coborrow_vs_grantedloans.merge(applicant_employment_match_score, on=['digitalLoanAccountId'], how ='outer')
Historical_feat_combined = Historical_feat_combined.merge(applicant_name_match_score, on=['digitalLoanAccountId'], how ='outer')

In [208]:
duplicateincolumn(Historical_feat_combined, 'digitalLoanAccountId')

There are duplicates in the digitalLoanAccountId column


In [209]:
counts = Historical_feat_combined['digitalLoanAccountId'].value_counts()
counts[counts > 1]

digitalLoanAccountId
321034e0-9767-4be5-8de7-8c8d52bccb8a    6
d50fd072-a2a2-48ab-9a53-700ec8eafc41    6
6e5c0abe-9eed-4124-b0b2-c712b9345456    5
2e4fa057-8c7d-4ece-ba3b-fff549c65732    5
207af9b2-a48c-47ae-a59e-cd45de99d01a    4
                                       ..
fe943fc8-b083-43c8-9a65-ae3cbf263f7b    2
fea87676-1597-4a94-8d89-77d1d809188b    2
fece28f2-c046-4de3-a4f1-8cc879be68b2    2
ff386b22-ce50-4399-afeb-c8186991c074    2
ffacbe9c-4e09-46db-bfad-15a4c1eddb41    2
Name: count, Length: 1452, dtype: int64

In [210]:
Historical_feat_combined[Historical_feat_combined['digitalLoanAccountId']== '321034e0-9767-4be5-8de7-8c8d52bccb8a']

,digitalLoanAccountId,ratio_coborrow_contracts_cnt_24M,ratio_coborrow_contracts_amt_24M,applicant_employment_match_score,industry_employment_match_score,applicant_name_match_score
10937,321034e0-9767-4be5-8de7-8c8d52bccb8a,NaN,NaN,-9,-9,18
10938,321034e0-9767-4be5-8de7-8c8d52bccb8a,NaN,NaN,-9,-9,43
10939,321034e0-9767-4be5-8de7-8c8d52bccb8a,NaN,NaN,-9,-9,41
10940,321034e0-9767-4be5-8de7-8c8d52bccb8a,NaN,NaN,-9,-9,37
10941,321034e0-9767-4be5-8de7-8c8d52bccb8a,NaN,NaN,-9,-9,-9
10942,321034e0-9767-4be5-8de7-8c8d52bccb8a,NaN,NaN,-9,-9,24


In [211]:
Historical_feat_combined = Historical_feat_combined.groupby('digitalLoanAccountId')[[
    'ratio_coborrow_contracts_cnt_24M',
    'ratio_coborrow_contracts_amt_24M',
    'applicant_employment_match_score',
    'industry_employment_match_score',
    'applicant_name_match_score'
]].max().reset_index()

## Scorecard Features Binning

In [212]:
df_CreditMaxOverdueDays = data.groupby('digitalLoanAccountId')['CreditMaxOverdueDays'].max().reset_index()


df_CreditMaxOverdueDays['CreditMaxOverdueDays_0123'] = np.where(df_CreditMaxOverdueDays['CreditMaxOverdueDays'].isin([0, 1, 2, 3]), 1, 0)
df_CreditMaxOverdueDays['CreditMaxOverdueDays_456'] = np.where(df_CreditMaxOverdueDays['CreditMaxOverdueDays'].isin([4,5,6]), 1, 0)
df_CreditMaxOverdueDays['CreditMaxOverdueDays_other'] = np.where(~df_CreditMaxOverdueDays['CreditMaxOverdueDays'].isin([0,1,2,3,4,5,6]), 1, 0)


df_CreditMaxOverdueDays = df_CreditMaxOverdueDays[['digitalLoanAccountId','CreditMaxOverdueDays_0123','CreditMaxOverdueDays_456','CreditMaxOverdueDays_other']]
df_CreditMaxOverdueDays.head()

,digitalLoanAccountId,CreditMaxOverdueDays_0123,CreditMaxOverdueDays_456,CreditMaxOverdueDays_other
0,0000181e-f0bc-414d-bbc1-69fd841f4bc1,0,0,1
1,00025b22-70c8-4d93-ad59-4a58f5c11d56,0,0,1
2,00029466-5f50-433b-859d-23aa3cb6d04d,0,0,1
3,0003edd4-5a8c-4e00-bfeb-3fffc587d38b,0,0,1
4,00051867-ce3d-4514-a9f3-b1c2f85ae4ed,0,0,1


In [213]:
duplicateincolumn(df_CreditMaxOverdueDays, 'digitalLoanAccountId')

No duplicates found in the digitalLoanAccountId column


In [214]:
# Group by 'digitalLoanAccountId' and get the max of 'CreditAvgCreditLimit'
df_CreditAvgCreditLimit = data.groupby('digitalLoanAccountId')['CreditAvgCreditLimit'].max().reset_index()

# Convert 'CreditAvgCreditLimit' to numeric, coercing errors to NaN
df_CreditAvgCreditLimit['CreditAvgCreditLimit'] = pd.to_numeric(df_CreditAvgCreditLimit['CreditAvgCreditLimit'], errors='coerce')

# Create binary columns based on the ranges provided
df_CreditAvgCreditLimit['CreditAvgCreditLimit_<2k'] = np.where(df_CreditAvgCreditLimit['CreditAvgCreditLimit'] < 2000, 1, 0)
df_CreditAvgCreditLimit['CreditAvgCreditLimit_2k_to_42k'] = np.where((df_CreditAvgCreditLimit['CreditAvgCreditLimit'] >= 2000) & (df_CreditAvgCreditLimit['CreditAvgCreditLimit'] < 42000), 1, 0)
df_CreditAvgCreditLimit['CreditAvgCreditLimit_42k_to_106k'] = np.where((df_CreditAvgCreditLimit['CreditAvgCreditLimit'] >= 42000) & (df_CreditAvgCreditLimit['CreditAvgCreditLimit'] < 106000), 1, 0)
df_CreditAvgCreditLimit['CreditAvgCreditLimit_>=106k'] = np.where(df_CreditAvgCreditLimit['CreditAvgCreditLimit'] >= 106000, 1, 0)
df_CreditAvgCreditLimit['CreditAvgCreditLimit_other'] = np.where(df_CreditAvgCreditLimit['CreditAvgCreditLimit'].isna(), 1, 0)

# Select only the required columns
df_CreditAvgCreditLimit = df_CreditAvgCreditLimit[['digitalLoanAccountId',
                                                   'CreditAvgCreditLimit_<2k', 'CreditAvgCreditLimit_2k_to_42k',
                                                   'CreditAvgCreditLimit_42k_to_106k', 'CreditAvgCreditLimit_>=106k',
                                                   'CreditAvgCreditLimit_other']]

# Display the first few rows
df_CreditAvgCreditLimit.head()


,digitalLoanAccountId,CreditAvgCreditLimit_<2k,CreditAvgCreditLimit_2k_to_42k,CreditAvgCreditLimit_42k_to_106k,CreditAvgCreditLimit_>=106k,CreditAvgCreditLimit_other
0,0000181e-f0bc-414d-bbc1-69fd841f4bc1,0,1,0,0,0
1,00025b22-70c8-4d93-ad59-4a58f5c11d56,1,0,0,0,0
2,00029466-5f50-433b-859d-23aa3cb6d04d,0,0,1,0,0
3,0003edd4-5a8c-4e00-bfeb-3fffc587d38b,1,0,0,0,0
4,00051867-ce3d-4514-a9f3-b1c2f85ae4ed,1,0,0,0,0


In [215]:
duplicateincolumn(df_CreditAvgCreditLimit, 'digitalLoanAccountId')

No duplicates found in the digitalLoanAccountId column


In [216]:
# Create the new DataFrame with specified columns and filter
df_accountsOverduePct = data[data['ContractPhaseDesc'] == 'Active'][[
    'digitalLoanAccountId',
    'customerId',
    'ContractPhaseDesc',
    'ContractStartDate',
    'ContractEndDate',
    'OverdueDaysDesc',
    'CBContractCode'
]]
 
# Define the categories of overdue days to count
overdue_categories = [
    '91-180 days delay / More than 3 Cycles late',
    '181-365 days delay',
    'More than 1 year delay'
]

df_filtered_overdue = df_accountsOverduePct[df_accountsOverduePct['OverdueDaysDesc'].isin(overdue_categories)]

#Group by 'digitalLoanAccountId' and count occurrences
df_count_90days = df_filtered_overdue.groupby('digitalLoanAccountId').size().reset_index(name='accounts90DaysCount')

 
# # Create the 'accounts90DaysCount' column that counts the number of rows per 'digitalLoanAccountId' for the specified overdue categories
# df_accountsOverduePct['accounts90DaysCount'] = df_accountsOverduePct[df_accountsOverduePct['OverdueDaysDesc'].isin(overdue_categories)].groupby('digitalLoanAccountId')['OverdueDaysDesc'].transform('count')
 

# # Create the 'total_active' column that counts the number of rows per 'digitalLoanAccountId'
df_accountsOverduePct_active = df_accountsOverduePct.groupby('digitalLoanAccountId').agg(
                                        total_active =('CBContractCode', 'count')).reset_index()
 
#Merge the counts back into the original DataFrame
df_accountsOverduePct = df_accountsOverduePct_active.merge(df_count_90days, on='digitalLoanAccountId', how='left')
    
# # Calculate 'accountsOverduePct'
df_accountsOverduePct['accountsOverduePct'] = df_accountsOverduePct['accounts90DaysCount'] / df_accountsOverduePct['total_active'] * 100
 
df_accountsOverduePct_ = df_accountsOverduePct

In [217]:
# # Create new columns based on 'accountsOverduePct'
df_accountsOverduePct['NumberOverduePct<17'] = np.where(df_accountsOverduePct['accountsOverduePct'] < 17, 1, 0)
df_accountsOverduePct['NumberOverduePct<41'] = np.where((df_accountsOverduePct['accountsOverduePct'] >= 17) & (df_accountsOverduePct['accountsOverduePct'] < 41), 1, 0)
df_accountsOverduePct['NumberOverduePct>=41'] = np.where(df_accountsOverduePct['accountsOverduePct'] >= 41, 1, 0)
df_accountsOverduePct['NumberOverduePct_other'] = np.where(df_accountsOverduePct['accountsOverduePct'].isna(), 1, 0)
 
    
df_accountsOverduePct = df_accountsOverduePct[['digitalLoanAccountId', 'NumberOverduePct<17', 'NumberOverduePct<41',
                       'NumberOverduePct>=41', 'NumberOverduePct_other']]

# # Display the new DataFrame
df_accountsOverduePct.head()

,digitalLoanAccountId,NumberOverduePct<17,NumberOverduePct<41,NumberOverduePct>=41,NumberOverduePct_other
0,0000181e-f0bc-414d-bbc1-69fd841f4bc1,0,0,0,1
1,00029466-5f50-433b-859d-23aa3cb6d04d,0,0,0,1
2,000a3bee-c57d-4647-beaa-f8d3293b4101,0,0,0,1
3,000d27e2-705e-47e2-886d-542a43c0889b,0,0,0,1
4,000d5b99-ffd3-45ad-b650-32fe43a95dc7,0,0,0,1


In [218]:
duplicateincolumn(df_accountsOverduePct, 'digitalLoanAccountId')

No duplicates found in the digitalLoanAccountId column


In [219]:
# Group by 'digitalLoanAccountId' and get the maximum 'NonInstAvgCreditLimit'
df_NonInstAvgCreditLimit = data.groupby('digitalLoanAccountId')['NonInstAvgCreditLimit'].max().reset_index()

# Convert 'NonInstAvgCreditLimit' to numeric, coercing errors to NaN
df_NonInstAvgCreditLimit['NonInstAvgCreditLimit'] = pd.to_numeric(df_NonInstAvgCreditLimit['NonInstAvgCreditLimit'], errors='coerce')

# Apply conditions to create new columns based on 'NonInstAvgCreditLimit'
df_NonInstAvgCreditLimit['NonInstAvgCreditLimit<15k'] = np.where(df_NonInstAvgCreditLimit['NonInstAvgCreditLimit'] < 15000, 1, 0)
df_NonInstAvgCreditLimit['NonInstAvgCreditLimit<35k'] = np.where((df_NonInstAvgCreditLimit['NonInstAvgCreditLimit'] >= 15000) & (df_NonInstAvgCreditLimit['NonInstAvgCreditLimit'] < 35000), 1, 0)
df_NonInstAvgCreditLimit['NonInstAvgCreditLimit<65k'] = np.where((df_NonInstAvgCreditLimit['NonInstAvgCreditLimit'] >= 35000) & (df_NonInstAvgCreditLimit['NonInstAvgCreditLimit'] < 65000), 1, 0)
df_NonInstAvgCreditLimit['NonInstAvgCreditLimit>=65k'] = np.where((df_NonInstAvgCreditLimit['NonInstAvgCreditLimit'] >= 65000) & (df_NonInstAvgCreditLimit['NonInstAvgCreditLimit'] < 106000), 1, 0)
df_NonInstAvgCreditLimit['NonInstAvgCreditLimit>=106k'] = np.where(df_NonInstAvgCreditLimit['NonInstAvgCreditLimit'] >= 106000, 1, 0)

# Handle NaN values in 'NonInstAvgCreditLimit'
df_NonInstAvgCreditLimit['NonInstAvgCreditLimit_others'] = np.where(df_NonInstAvgCreditLimit['NonInstAvgCreditLimit'].isna(), 1, 0)

# Select only the required columns
df_NonInstAvgCreditLimit = df_NonInstAvgCreditLimit[['digitalLoanAccountId', 'NonInstAvgCreditLimit<15k', 
                                                     'NonInstAvgCreditLimit<35k', 'NonInstAvgCreditLimit<65k', 
                                                     'NonInstAvgCreditLimit>=65k', 'NonInstAvgCreditLimit>=106k', 
                                                     'NonInstAvgCreditLimit_others']]

# Display the first few rows
df_NonInstAvgCreditLimit.head()


,digitalLoanAccountId,NonInstAvgCreditLimit<15k,NonInstAvgCreditLimit<35k,NonInstAvgCreditLimit<65k,NonInstAvgCreditLimit>=65k,NonInstAvgCreditLimit>=106k,NonInstAvgCreditLimit_others
0,0000181e-f0bc-414d-bbc1-69fd841f4bc1,1,0,0,0,0,0
1,00025b22-70c8-4d93-ad59-4a58f5c11d56,1,0,0,0,0,0
2,00029466-5f50-433b-859d-23aa3cb6d04d,1,0,0,0,0,0
3,0003edd4-5a8c-4e00-bfeb-3fffc587d38b,1,0,0,0,0,0
4,00051867-ce3d-4514-a9f3-b1c2f85ae4ed,1,0,0,0,0,0


In [220]:
duplicateincolumn(df_NonInstAvgCreditLimit, 'digitalLoanAccountId')

No duplicates found in the digitalLoanAccountId column


In [221]:

# Sample data creation (Assuming you have already imported your data)
# data = pd.read_csv('your_data_file.csv')

# Ensure the 'ScoreRange' column is of string type
data['ScoreRange'] = data['ScoreRange'].astype(str)

# Apply groupby and max aggregation
df_ScoreRange = data.groupby('digitalLoanAccountId')['ScoreRange'].max().reset_index()

# Create new columns based on conditions
df_ScoreRange['ScoreRange_HiJiDiEiFiGiIi'] = np.where(df_ScoreRange['ScoreRange'].isin(['Hi','Ji','Di','Ei','Fi','Gi','Ii']), 1, 0)
df_ScoreRange['ScoreRange_Ci'] = np.where(df_ScoreRange['ScoreRange'].isin(['Ci']), 1, 0)
df_ScoreRange['ScoreRange_Bi'] = np.where(df_ScoreRange['ScoreRange'].isin(['Bi']), 1, 0)
df_ScoreRange['ScoreRange_Other'] = np.where(~df_ScoreRange['ScoreRange'].isin(['Hi','Ji','Di','Ei','Fi','Gi','Ii','Ci','Bi']), 1, 0)

# Select only the required columns
df_ScoreRange = df_ScoreRange[['digitalLoanAccountId','ScoreRange_HiJiDiEiFiGiIi','ScoreRange_Ci','ScoreRange_Bi','ScoreRange_Other']]

# Display the first few rows of the resulting DataFrame
df_ScoreRange.head()


,digitalLoanAccountId,ScoreRange_HiJiDiEiFiGiIi,ScoreRange_Ci,ScoreRange_Bi,ScoreRange_Other
0,0000181e-f0bc-414d-bbc1-69fd841f4bc1,0,1,0,0
1,00025b22-70c8-4d93-ad59-4a58f5c11d56,1,0,0,0
2,00029466-5f50-433b-859d-23aa3cb6d04d,0,0,0,1
3,0003edd4-5a8c-4e00-bfeb-3fffc587d38b,0,0,0,1
4,00051867-ce3d-4514-a9f3-b1c2f85ae4ed,0,0,0,1


In [222]:
duplicateincolumn(df_ScoreRange, 'digitalLoanAccountId')

No duplicates found in the digitalLoanAccountId column


In [223]:
ScoreCard_Features = df_CreditMaxOverdueDays.merge(df_CreditAvgCreditLimit, on='digitalLoanAccountId',how ='outer')
ScoreCard_Features = ScoreCard_Features.merge(df_accountsOverduePct_, on='digitalLoanAccountId',how ='outer')
ScoreCard_Features = ScoreCard_Features.merge(df_NonInstAvgCreditLimit, on='digitalLoanAccountId',how ='outer')
ScoreCard_Features = ScoreCard_Features.merge(df_ScoreRange, on='digitalLoanAccountId',how ='outer')

In [224]:
duplicateincolumn(ScoreCard_Features, 'digitalLoanAccountId')

No duplicates found in the digitalLoanAccountId column


## Scorecard Feat for Master Table

In [225]:
CreditMaxOverdueDays = data[['digitalLoanAccountId','CreditMaxOverdueDays']].drop_duplicates(keep='first')
CreditAvgCreditLimit = data[['digitalLoanAccountId','CreditAvgCreditLimit']].drop_duplicates(keep='first')
ScoreRange = data[['digitalLoanAccountId','ScoreRange']].drop_duplicates(keep='first')
NonInstAvgCreditLimit = data[['digitalLoanAccountId','NonInstAvgCreditLimit']].drop_duplicates(keep='first')
accountsOverduePct = df_accountsOverduePct_[['digitalLoanAccountId','accountsOverduePct']].drop_duplicates(keep='first')

In [226]:
ScoreCard_Features_Master = CreditMaxOverdueDays.merge(CreditAvgCreditLimit, on='digitalLoanAccountId',how ='outer')
ScoreCard_Features_Master = ScoreCard_Features_Master.merge(accountsOverduePct, on='digitalLoanAccountId',how ='outer')
ScoreCard_Features_Master = ScoreCard_Features_Master.merge(NonInstAvgCreditLimit, on='digitalLoanAccountId',how ='outer')
ScoreCard_Features_Master = ScoreCard_Features_Master.merge(ScoreRange, on='digitalLoanAccountId',how ='outer')

In [227]:
duplicateincolumn(ScoreCard_Features_Master, 'digitalLoanAccountId')

There are duplicates in the digitalLoanAccountId column


In [228]:
counts = ScoreCard_Features_Master['digitalLoanAccountId'].value_counts()
counts[counts > 1]

digitalLoanAccountId
02ab686d-73f9-4594-af64-6648f04f5a11    8
Name: count, dtype: int64

In [229]:
ScoreCard_Features_Master[ScoreCard_Features_Master['digitalLoanAccountId']=='02ab686d-73f9-4594-af64-6648f04f5a11']

,digitalLoanAccountId,CreditMaxOverdueDays,CreditAvgCreditLimit,accountsOverduePct,NonInstAvgCreditLimit,ScoreRange
559,02ab686d-73f9-4594-af64-6648f04f5a11,,0,NaN,0,None
560,02ab686d-73f9-4594-af64-6648f04f5a11,,0,NaN,0,Gi
561,02ab686d-73f9-4594-af64-6648f04f5a11,,557000,NaN,0,None
562,02ab686d-73f9-4594-af64-6648f04f5a11,,557000,NaN,0,Gi
563,02ab686d-73f9-4594-af64-6648f04f5a11,0,0,NaN,0,None
564,02ab686d-73f9-4594-af64-6648f04f5a11,0,0,NaN,0,Gi
565,02ab686d-73f9-4594-af64-6648f04f5a11,0,557000,NaN,0,None
566,02ab686d-73f9-4594-af64-6648f04f5a11,0,557000,NaN,0,Gi


In [230]:
ScoreCard_Features_Master = ScoreCard_Features_Master.groupby('digitalLoanAccountId')[['CreditMaxOverdueDays',  'CreditAvgCreditLimit',   'accountsOverduePct',   'NonInstAvgCreditLimit',   'ScoreRange']].max().reset_index()

In [231]:
ScoreCard_Features_Master[ScoreCard_Features_Master['digitalLoanAccountId']=='02ab686d-73f9-4594-af64-6648f04f5a11']

,digitalLoanAccountId,CreditMaxOverdueDays,CreditAvgCreditLimit,accountsOverduePct,NonInstAvgCreditLimit,ScoreRange
559,02ab686d-73f9-4594-af64-6648f04f5a11,0,557000,NaN,0,None


## Master table

In [232]:
Master_table = df.merge(snapshot_feature_df,on=['digitalLoanAccountId'],how='left')
Master_table = Master_table.merge(Historical_feat_granted,on=['digitalLoanAccountId'],how='left')
Master_table = Master_table.merge(Historical_feat_nongranted,on=['digitalLoanAccountId'],how='left')
Master_table = Master_table.merge(Historical_feat_combined,on=['digitalLoanAccountId'],how='left')
Master_table = Master_table.merge(ScoreCard_Features_Master,on=['digitalLoanAccountId'],how='left')

In [233]:
print(f"The shape of the Master_table is:\t {Master_table.shape}")

The shape of the Master_table is:	 (1302893, 223)


In [234]:
Master_table.head()

,customerId,digitalLoanAccountId,tinNumber,dateOfBirth,email,mobileNo,firstName,middleName,lastName,custLoanJourneyZipCode,custLoanJourneyCity,tsa_onboarding_time,new_loan_type,natureofwork,age,Education,industryDescription,custLoanJourneyProvince,monthlyIncome,AnnualIncome,incomeperSalaryScale,incomeperSalaryScale_Annual,gender,credolabScore,maritalStatus,dependentsCount,Region_Name,deviceType,brand,osversion,appVersion,defFPD15,obsFPD15,obs_fpd30_vol,def_fpd30_vol,obs_fpd30_cnt,def_fpd30_cnt,obs_fspd30_vol,def_fspd30_vol,obs_fspd30_cnt,def_fspd30_cnt,obs_fstpd30_vol,def_fstpd30_vol,obs_fstpd30_cnt,def_fstpd30_cnt,OccupationDesc,OccupationStatusDesc,PSICDesc,TradeName,AnnualMonthlyIndicator,GrossIncome,AnnualIncomecalculated,LastUpdateDate,cnt_active_contracts,cnt_active_contracts_Credit Cards,cnt_active_contracts_Credit Lines,cnt_active_contracts_Home Equity Loans,cnt_active_contracts_Other Loans,cnt_active_contracts_Personal Loans,cnt_active_contracts_Real Estate Loans,cnt_active_contracts_Short and Term Loans,cnt_active_contracts_Time Loans,cnt_active_contracts_Trust Loans,max_amt_active_contracts,tot_active_contracts_os_amt,tot_active_contracts_util,Credit Cards_active_contracts_util,Credit Lines_active_contracts_util,Home Equity Loans_active_contracts_util,Other Loans_active_contracts_util,Personal Loans_active_contracts_util,Real Estate Loans_active_contracts_util,Short and Term Loans_active_contracts_util,Time Loans_active_contracts_util,Trust Loans_active_contracts_util,last_repay_amt_30d,highest_latest_repay_method_30d,last_repay_amt_60d,highest_latest_repay_method_60d,last_repay_amt_90d,highest_latest_repay_method_90d,last_repay_amt,highest_latest_repay_method,max_age_active_contracts_snapshot,max_age_all_contracts_snapshot,days_since_last_inquiry,days_since_last_grant,days_since_last_closed,days_since_last_repay,days_since_last_reject,days_since_last_renounce,Credit Cards_granted_contracts_cnt_6M,Credit Lines_granted_contracts_cnt_6M,Home Equity Loans_granted_contracts_cnt_6M,Other Loans_granted_contracts_cnt_6M,Personal Loans_granted_contracts_cnt_6M,Real Estate Loans_granted_contracts_cnt_6M,Short and Term Loans_granted_contracts_cnt_6M,Time Loans_granted_contracts_cnt_6M,Trust Loans_granted_contracts_cnt_6M,Credit Cards_granted_contracts_cnt_12M,Credit Lines_granted_contracts_cnt_12M,Home Equity Loans_granted_contracts_cnt_12M,Other Loans_granted_contracts_cnt_12M,Personal Loans_granted_contracts_cnt_12M,Real Estate Loans_granted_contracts_cnt_12M,Short and Term Loans_granted_contracts_cnt_12M,Time Loans_granted_contracts_cnt_12M,Trust Loans_granted_contracts_cnt_12M,Credit Cards_granted_contracts_cnt_24M,Credit Lines_granted_contracts_cnt_24M,Home Equity Loans_granted_contracts_cnt_24M,Other Loans_granted_contracts_cnt_24M,Personal Loans_granted_contracts_cnt_24M,Real Estate Loans_granted_contracts_cnt_24M,Short and Term Loans_granted_contracts_cnt_24M,Time Loans_granted_contracts_cnt_24M,Trust Loans_granted_contracts_cnt_24M,granted_contracts_cnt_6M,granted_contracts_cnt_12M,granted_contracts_cnt_24M,Credit Cards_granted_contracts_amt_6M,Credit Lines_granted_contracts_amt_6M,Home Equity Loans_granted_contracts_amt_6M,Other Loans_granted_contracts_amt_6M,Personal Loans_granted_contracts_amt_6M,Real Estate Loans_granted_contracts_amt_6M,Short and Term Loans_granted_contracts_amt_6M,Time Loans_granted_contracts_amt_6M,Trust Loans_granted_contracts_amt_6M,Credit Cards_granted_contracts_amt_12M,Credit Lines_granted_contracts_amt_12M,Home Equity Loans_granted_contracts_amt_12M,Other Loans_granted_contracts_amt_12M,Personal Loans_granted_contracts_amt_12M,Real Estate Loans_granted_contracts_amt_12M,Short and Term Loans_granted_contracts_amt_12M,Time Loans_granted_contracts_amt_12M,Trust Loans_granted_contracts_amt_12M,Credit Cards_granted_contracts_amt_24M,Credit Lines_granted_contracts_amt_24M,Home Equity Loans_granted_contracts_amt_24M,Other Loans_granted_contracts_amt_24M,Personal Loans_granted_contracts_amt_24M,R

In [235]:
duplicateincolumn(Master_table, 'digitalLoanAccountId')

No duplicates found in the digitalLoanAccountId column


In [236]:
def replace_spaces_in_columns(df):
    # Replace spaces with underscores in column names
    df.columns = df.columns.str.replace(' ', '_')
    return df

In [237]:
Master_table = replace_spaces_in_columns(Master_table)

In [238]:
Master_table.to_csv("Master_table.csv", index = False)

# Credolab

In [239]:
sq = """with app as 
(Select  DISTINCT t1.digitalLoanAccountId, t1.customerId,t3.crodolabDeviceId, ca.package_name
FROM
`prj-prod-dataplatform.risk_credit_mis.loan_master_table` t1
LEFT JOIN
`prj-prod-dataplatform.dl_loans_db_raw.tdbk_digital_loan_application` t2
ON t1.digitalLoanAccountId = t2.digitalLoanAccountId
LEFT JOIN
`prj-prod-dataplatform.dl_loans_db_raw.tdbk_credolab_track` t3
ON t2.credolabRefNumber = t3.refno
LEFT JOIN
`prj-prod-dataplatform.credolab_raw.android_credolab_datasets_struct_columns` t4
ON t3.refno = t4.deviceId
inner join
`prj-prod-dataplatform.core_raw.loan_accounts` loan
on loan.CUSTOMERID = t1.customerId
 INNER JOIN
(select deviceId, af.package_name as package_name, af.first_install_time as first_install_time from `prj-prod-dataplatform.credolab_raw.android_credolab_Application`  ,
unnest(Application) as af) ca
ON ca.deviceId = t3.refno
)
Select * from app
;"""

df_risk_table_2  = client.query(sq).to_dataframe(progress_bar_type = 'tqdm')
print(f"The rows and columns in the credolab dataframe are:\t {df_risk_table_2.shape}")

Job ID 6bcd9f7f-94c5-4039-8e7e-ba64f2109ce0 successfully executed: |          |
Downloading: 100%|██████████|
The rows and columns in the credolab dataframe are:	 (43248597, 4)


In [240]:
df_risk_table_2.head()

,digitalLoanAccountId,customerId,crodolabDeviceId,package_name
0,798e86ab-b544-42d1-99b2-fa8d3abdbfa9,1420631,e1bdea4b10d011cb79f99d56d301ff03,com.coloring.halloween
1,9a77baa9-ea9e-4438-8584-c230fa2d59ab,2499057,4143276319aac217af5809e4b7b744df,com.zhiliaoapp.musically.go
2,8940f845-881a-47c4-bc22-97e3e1893bf4,2511127,77108b9a4301ab9e850399a22befc441,com.android.proxyhandler
3,82cf7be9-4007-4626-ac41-295b13ac75d1,1494251,743990b8df56cf53da465638e5a9e018,com.byimplication.sakay
4,f8516306-8c90-4151-95f1-96449690406f,1062597,4676d7661e14aa2dc822efa4f45f24a4,com.google.android.music


In [241]:
dfapp = pd.read_csv(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Biswa\IncomeEstimation\Data\PH_TonikBank_applications_20240807.csv")

In [242]:
dfapp.head()

,Package Name,# of datasets,# of datasets where app is considered as user installed,Category,Rating,Rating Description,Is Paid
0,com.tonik.mobile,685556,685553,finance,rated for 3+,undefined,free
1,android,684929,0,NaN,NaN,NaN,NaN
2,com.android.providers.settings,684922,0,NaN,NaN,NaN,NaN
3,com.android.externalstorage,684920,0,NaN,NaN,NaN,NaN
4,com.android.settings,684920,0,NaN,NaN,NaN,NaN


In [243]:

# For df_risk_table_2
df_risk_table_2['package'] = df_risk_table_2['package_name'].str.replace(r'[ ._]', '', regex=True)

# For dfapp
dfapp['package'] = dfapp['Package Name'].str.replace(r'[ ._]', '', regex=True)


In [244]:
df_risk_table_2.head()

,digitalLoanAccountId,customerId,crodolabDeviceId,package_name,package
0,798e86ab-b544-42d1-99b2-fa8d3abdbfa9,1420631,e1bdea4b10d011cb79f99d56d301ff03,com.coloring.halloween,comcoloringhalloween
1,9a77baa9-ea9e-4438-8584-c230fa2d59ab,2499057,4143276319aac217af5809e4b7b744df,com.zhiliaoapp.musically.go,comzhiliaoappmusicallygo
2,8940f845-881a-47c4-bc22-97e3e1893bf4,2511127,77108b9a4301ab9e850399a22befc441,com.android.proxyhandler,comandroidproxyhandler
3,82cf7be9-4007-4626-ac41-295b13ac75d1,1494251,743990b8df56cf53da465638e5a9e018,com.byimplication.sakay,combyimplicationsakay
4,f8516306-8c90-4151-95f1-96449690406f,1062597,4676d7661e14aa2dc822efa4f45f24a4,com.google.android.music,comgoogleandroidmusic


In [245]:
dfapp.head()

,Package Name,# of datasets,# of datasets where app is considered as user installed,Category,Rating,Rating Description,Is Paid,package
0,com.tonik.mobile,685556,685553,finance,rated for 3+,undefined,free,comtonikmobile
1,android,684929,0,NaN,NaN,NaN,NaN,android
2,com.android.providers.settings,684922,0,NaN,NaN,NaN,NaN,comandroidproviderssettings
3,com.android.externalstorage,684920,0,NaN,NaN,NaN,NaN,comandroidexternalstorage
4,com.android.settings,684920,0,NaN,NaN,NaN,NaN,comandroidsettings


In [246]:
df_risk_table_2 = df_risk_table_2.merge(dfapp, on='package', how = 'outer')

In [247]:
df_risk_table_2.head()

,digitalLoanAccountId,customerId,crodolabDeviceId,package_name,package,Package Name,# of datasets,# of datasets where app is considered as user installed,Category,Rating,Rating Description,Is Paid
0,69e0b7b8-616f-4a7c-83ff-3788155b054b,2335890,cae2f5da66606b6b1459d3655b4d92e5,ACEN.APK,ACENAPK,ACEN.APK,1.0,1.0,NaN,NaN,NaN,NaN
1,NaN,<NA>,NaN,NaN,AO3ArchiveofOurOwnhwawi,AO3.Archive_of_Our_Own_hwawi,1.0,1.0,NaN,NaN,NaN,NaN
2,6adc679b-de22-40e4-91d7-52d5ebbedf48,1697855,ae25fd647672f8e723cc2c17dfdb1d5c,ARRuler.BirdmanStudio.net,ARRulerBirdmanStudionet,ARRuler.BirdmanStudio.net,3.0,3.0,NaN,NaN,NaN,NaN
3,23134646-6b75-4ef8-bfac-2050eae33335,1697855,ae25fd647672f8e723cc2c17dfdb1d5c,ARRuler.BirdmanStudio.net,ARRulerBirdmanStudionet,ARRuler.BirdmanStudio.net,3.0,3.0,NaN,NaN,NaN,NaN
4,ebe8c52c-ccfc-4fd3-89c2-29ae443e5265,1697855,ae25fd647672f8e723cc2c17dfdb1d5c,ARRuler.BirdmanStudio.net,ARRulerBirdmanStudionet,ARRuler.BirdmanStudio.net,3.0,3.0,NaN,NaN,NaN,NaN


In [248]:
df_risk_table_2.groupby('Category')['digitalLoanAccountId'].nunique().sort_values(ascending = False)

Category
finance                    144892
tools                      144891
communication              144757
video players & editors    144612
business                   144567
travel & local             144427
social                     143792
productivity               143455
photography                140162
music & audio              138794
shopping                   135131
entertainment              131363
auto & vehicles            127093
lifestyle                   83205
personalization             78369
maps & navigation           75522
health & fitness            72037
action                      69181
casual                      66988
news & magazines            66874
food & drink                61864
education                   47288
books & reference           43982
puzzle                      43322
arcade                      34196
adventure                   31657
art & design                28957
strategy                    27122
simulation                  25853
sport

In [249]:
df_risk_table_2.groupby('Rating Description')['digitalLoanAccountId'].nunique().sort_values(ascending = False)

Rating Description
undefined                                                   144892
parental guidance recommended                               144842
moderate violence                                            64331
simulated gambling                                           44061
mild violence                                                41588
                                                             ...  
sex, use of alcohol/tobacco, strong language                     0
sexual violence, strong language                                 0
strong violence, gambling                                        0
strong violence, use of alcohol/tobacco, strong language         0
violence, sex, mild swearing                                     0
Name: digitalLoanAccountId, Length: 151, dtype: int64

In [250]:

df_risk_table_2.groupby('Rating Description')['digitalLoanAccountId'].nunique().sort_values(ascending = False)

Rating Description
undefined                                                   144892
parental guidance recommended                               144842
moderate violence                                            64331
simulated gambling                                           44061
mild violence                                                41588
                                                             ...  
sex, use of alcohol/tobacco, strong language                     0
sexual violence, strong language                                 0
strong violence, gambling                                        0
strong violence, use of alcohol/tobacco, strong language         0
violence, sex, mild swearing                                     0
Name: digitalLoanAccountId, Length: 151, dtype: int64

In [251]:
df_risk_table_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43539825 entries, 0 to 43539824
Data columns (total 12 columns):
 #   Column                                                   Dtype  
---  ------                                                   -----  
 0   digitalLoanAccountId                                     object 
 1   customerId                                               Int64  
 2   crodolabDeviceId                                         object 
 3   package_name                                             object 
 4   package                                                  object 
 5   Package Name                                             object 
 6   # of datasets                                            float64
 7   # of datasets where app is considered as user installed  float64
 8   Category                                                 object 
 9   Rating                                                   object 
 10  Rating Description                      

In [252]:
df_risk_table_2.columns

Index(['digitalLoanAccountId', 'customerId', 'crodolabDeviceId',
       'package_name', 'package', 'Package Name', '# of datasets',
       '# of datasets where app is considered as user installed', 'Category',
       'Rating', 'Rating Description', 'Is Paid'],
      dtype='object')

In [253]:
df_risk_table_2 = df_risk_table_2[['digitalLoanAccountId', 'customerId', 'crodolabDeviceId', 'package_name', 'package', 'Category', 'Rating', 'Rating Description', 'Is Paid']].copy()

In [254]:
df_risk_table_2['Category'] = df_risk_table_2['Category'].fillna('Others')

In [255]:
df_risk_table_2 = df_risk_table_2.dropna(subset=['digitalLoanAccountId'])

In [256]:
df_risk_table_2 = df_risk_table_2.dropna(subset=['Category'])

In [257]:
# Find all rows with duplicates (including the first occurrence)
all_duplicate_rows = df_risk_table_2[df_risk_table_2.duplicated(keep=False)]

# Display all duplicate rows
print(all_duplicate_rows.head())

                        digitalLoanAccountId  customerId  \
622656  ff5e0928-5130-4a56-86a3-471d2abdcc81     1855783   
622657  ff5e0928-5130-4a56-86a3-471d2abdcc81     1855783   
622658  4d46c8ad-8bb9-41bd-95d7-9dd18c338501     1418366   
622659  4d46c8ad-8bb9-41bd-95d7-9dd18c338501     1418366   
622660  0b039588-1712-422f-a96d-9402442d1a2a     1209975   

                        crodolabDeviceId        package_name  \
622656  624671bcf7c62b138e485e277449d0ed  cn.oneplus.oemtcma   
622657  624671bcf7c62b138e485e277449d0ed  cn.oneplus.oemtcma   
622658  da417c71afe0c435ac5b941a00b3dd74  cn.oneplus.oemtcma   
622659  da417c71afe0c435ac5b941a00b3dd74  cn.oneplus.oemtcma   
622660  80db0802857d719294ad611b6c3a6826  cn.oneplus.oemtcma   

                 package Category Rating Rating Description Is Paid  
622656  cnoneplusoemtcma   Others    NaN                NaN     NaN  
622657  cnoneplusoemtcma   Others    NaN                NaN     NaN  
622658  cnoneplusoemtcma   Others    NaN    

In [258]:
df_risk_table_2[df_risk_table_2['digitalLoanAccountId']=='3cc22ed0-73a3-4917-93b3-905547a620ab']

,digitalLoanAccountId,customerId,crodolabDeviceId,package_name,package,Category,Rating,Rating Description,Is Paid
72163,3cc22ed0-73a3-4917-93b3-905547a620ab,2679408,04d3eb7885369112e6f19a22c995537a,android,android,Others,NaN,NaN,NaN
412557,3cc22ed0-73a3-4917-93b3-905547a620ab,2679408,04d3eb7885369112e6f19a22c995537a,android.overlay.common,androidoverlaycommon,Others,NaN,NaN,NaN
466906,3cc22ed0-73a3-4917-93b3-905547a620ab,2679408,04d3eb7885369112e6f19a22c995537a,android.overlay.target,androidoverlaytarget,Others,NaN,NaN,NaN
530675,3cc22ed0-73a3-4917-93b3-905547a620ab,2679408,04d3eb7885369112e6f19a22c995537a,android.qvaoverlay.common,androidqvaoverlaycommon,Others,NaN,NaN,NaN
552076,3cc22ed0-73a3-4917-93b3-905547a620ab,2679408,04d3eb7885369112e6f19a22c995537a,appinventor.ai_progetto2003.SCAN,appinventoraiprogetto2003SCAN,tools,rated for 3+,undefined,free
...,...,...,...,...,...,...,...,...,...
43402155,3cc22ed0-73a3-4917-93b3-905547a620ab,2679408,04d3eb7885369112e6f19a22c995537a,vendor.qti.hardware.cacert.server,vendorqtihardwarecacertserver,Others,NaN,NaN,NaN
43443808,3cc22ed0-73a3-4917-93b3-905547a620ab,2679408,04d3eb7885369112e6f19a22c995537a,vendor.qti.iwlan,vendorqtiiwlan,Others,NaN,NaN,NaN
43503342,3cc22ed0-73a3-4917-93b3-905547a620ab,2679408,04d3eb7885369112e6f19a22c995537a,weatherradar.livemaps.free,weatherradarlivemapsfree,weather,rated for 3+,undefined,free
43517274,3cc22ed0-73a3-4917-93b3-905547a620ab,2679408,04d3eb7885369112e6f19a22c995537a,www.mobile.bdo.com.ph,wwwmobilebdocomph,finance,rated for 3+,undefined,free


In [259]:
len(df_risk_table_2['Category'].unique())

50

In [260]:
dfapp1 = df_risk_table_2.pivot_table(index = 'digitalLoanAccountId', columns='Category', values='package', aggfunc='nunique').reset_index()

In [261]:
dfapp1.head()

Category,digitalLoanAccountId,Others,action,adventure,arcade,art & design,auto & vehicles,beauty,board,books & reference,business,card,casino,casual,comics,communication,dating,education,educational,entertainment,events,finance,food & drink,health & fitness,house & home,libraries & demo,lifestyle,maps & navigation,medical,music,music & audio,news & magazines,parenting,personalization,photography,productivity,puzzle,racing,role playing,shopping,simulation,social,sports,strategy,tools,travel & local,trivia,undefined,video players & editors,weather,word
0,00000ad2-697b-49a4-9f16-b177eb3b5f39,165.0,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,7.0,NaN,NaN,NaN,1.0,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,1.0,2.0,5.0,NaN,NaN,NaN,2.0,NaN,5.0,NaN,NaN,16.0,1.0,NaN,NaN,4.0,NaN,NaN
1,000043be-c9df-4622-8689-11ffde9060df,219.0,1.0,NaN,1.0,NaN,1.0,NaN,NaN,NaN,5.0,NaN,1.0,2.0,NaN,9.0,NaN,1.0,NaN,7.0,NaN,26.0,6.0,NaN,NaN,NaN,1.0,3.0,1.0,NaN,4.0,2.0,NaN,2.0,1.0,13.0,3.0,NaN,NaN,8.0,NaN,9.0,NaN,1.0,25.0,4.0,NaN,NaN,3.0,2.0,NaN
2,000093c0-1c86-4efb-b648-5b34580ee28e,204.0,NaN,1.0,NaN,NaN,1.0,NaN,1.0,NaN,2.0,1.0,1.0,NaN,NaN,6.0,NaN,NaN,NaN,2.0,NaN,5.0,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,3.0,NaN,NaN,NaN,1.0,7.0,NaN,NaN,NaN,2.0,NaN,1.0,NaN,NaN,16.0,2.0,NaN,NaN,3.0,NaN,NaN
3,00009619-2ace-4fcf-bb9d-3f5bbf7423ce,171.0,NaN,1.0,NaN,NaN,1.0,1.0,NaN,2.0,5.0,NaN,NaN,1.0,NaN,8.0,NaN,NaN,NaN,5.0,NaN,12.0,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,2.0,10.0,NaN,NaN,NaN,4.0,1.0,3.0,NaN,NaN,14.0,2.0,NaN,NaN,7.0,NaN,NaN
4,0000a6c8-2d70-466a-99a4-2be1fe5d5e0b,238.0,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,9.0,NaN,NaN,NaN,3.0,NaN,21.0,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,3.0,NaN,NaN,NaN,3.0,9.0,NaN,NaN,NaN,4.0,NaN,3.0,1.0,1.0,22.0,3.0,NaN,NaN,4.0,NaN,NaN


In [262]:
dfapp1.columns

Index(['digitalLoanAccountId', 'Others', 'action', 'adventure', 'arcade',
       'art & design', 'auto & vehicles', 'beauty', 'board',
       'books & reference', 'business', 'card', 'casino', 'casual', 'comics',
       'communication', 'dating', 'education', 'educational', 'entertainment',
       'events', 'finance', 'food & drink', 'health & fitness', 'house & home',
       'libraries & demo', 'lifestyle', 'maps & navigation', 'medical',
       'music', 'music & audio', 'news & magazines', 'parenting',
       'personalization', 'photography', 'productivity', 'puzzle', 'racing',
       'role playing', 'shopping', 'simulation', 'social', 'sports',
       'strategy', 'tools', 'travel & local', 'trivia', 'undefined',
       'video players & editors', 'weather', 'word'],
      dtype='object', name='Category')

In [263]:
dfapp1 = dfapp1[['digitalLoanAccountId', 'Others', 'action', 'adventure', 'arcade', 'art & design',
       'auto & vehicles', 'beauty', 'board', 'books & reference', 'business',
       'card', 'casino', 'casual', 'comics', 'communication', 'dating',
       'education', 'educational', 'entertainment', 'events', 'finance',
       'food & drink', 'health & fitness', 'house & home', 'libraries & demo',
       'lifestyle', 'maps & navigation', 'medical', 'music', 'music & audio',
       'news & magazines', 'parenting', 'personalization', 'photography',
       'productivity', 'puzzle', 'racing', 'role playing', 'shopping',
       'simulation', 'social', 'sports', 'strategy', 'tools', 'travel & local',
       'trivia', 'undefined', 'video players & editors', 'weather', 'word']].copy()

In [264]:
Master_table.columns

Index(['customerId', 'digitalLoanAccountId', 'tinNumber', 'dateOfBirth',
       'email', 'mobileNo', 'firstName', 'middleName', 'lastName',
       'custLoanJourneyZipCode',
       ...
       'ratio_coborrow_contracts_cnt_24M', 'ratio_coborrow_contracts_amt_24M',
       'applicant_employment_match_score', 'industry_employment_match_score',
       'applicant_name_match_score', 'CreditMaxOverdueDays',
       'CreditAvgCreditLimit', 'accountsOverduePct', 'NonInstAvgCreditLimit',
       'ScoreRange'],
      dtype='object', length=223)

In [265]:
Master_table = Master_table.merge(dfapp1, on ='digitalLoanAccountId', how = 'left')

In [266]:
print(f"The shape of Master_table is:\t {Master_table.shape}")

The shape of Master_table is:	 (1302893, 273)


In [267]:
Master_table.head()

,customerId,digitalLoanAccountId,tinNumber,dateOfBirth,email,mobileNo,firstName,middleName,lastName,custLoanJourneyZipCode,custLoanJourneyCity,tsa_onboarding_time,new_loan_type,natureofwork,age,Education,industryDescription,custLoanJourneyProvince,monthlyIncome,AnnualIncome,incomeperSalaryScale,incomeperSalaryScale_Annual,gender,credolabScore,maritalStatus,dependentsCount,Region_Name,deviceType,brand,osversion,appVersion,defFPD15,obsFPD15,obs_fpd30_vol,def_fpd30_vol,obs_fpd30_cnt,def_fpd30_cnt,obs_fspd30_vol,def_fspd30_vol,obs_fspd30_cnt,def_fspd30_cnt,obs_fstpd30_vol,def_fstpd30_vol,obs_fstpd30_cnt,def_fstpd30_cnt,OccupationDesc,OccupationStatusDesc,PSICDesc,TradeName,AnnualMonthlyIndicator,GrossIncome,AnnualIncomecalculated,LastUpdateDate,cnt_active_contracts,cnt_active_contracts_Credit_Cards,cnt_active_contracts_Credit_Lines,cnt_active_contracts_Home_Equity_Loans,cnt_active_contracts_Other_Loans,cnt_active_contracts_Personal_Loans,cnt_active_contracts_Real_Estate_Loans,cnt_active_contracts_Short_and_Term_Loans,cnt_active_contracts_Time_Loans,cnt_active_contracts_Trust_Loans,max_amt_active_contracts,tot_active_contracts_os_amt,tot_active_contracts_util,Credit_Cards_active_contracts_util,Credit_Lines_active_contracts_util,Home_Equity_Loans_active_contracts_util,Other_Loans_active_contracts_util,Personal_Loans_active_contracts_util,Real_Estate_Loans_active_contracts_util,Short_and_Term_Loans_active_contracts_util,Time_Loans_active_contracts_util,Trust_Loans_active_contracts_util,last_repay_amt_30d,highest_latest_repay_method_30d,last_repay_amt_60d,highest_latest_repay_method_60d,last_repay_amt_90d,highest_latest_repay_method_90d,last_repay_amt,highest_latest_repay_method,max_age_active_contracts_snapshot,max_age_all_contracts_snapshot,days_since_last_inquiry,days_since_last_grant,days_since_last_closed,days_since_last_repay,days_since_last_reject,days_since_last_renounce,Credit_Cards_granted_contracts_cnt_6M,Credit_Lines_granted_contracts_cnt_6M,Home_Equity_Loans_granted_contracts_cnt_6M,Other_Loans_granted_contracts_cnt_6M,Personal_Loans_granted_contracts_cnt_6M,Real_Estate_Loans_granted_contracts_cnt_6M,Short_and_Term_Loans_granted_contracts_cnt_6M,Time_Loans_granted_contracts_cnt_6M,Trust_Loans_granted_contracts_cnt_6M,Credit_Cards_granted_contracts_cnt_12M,Credit_Lines_granted_contracts_cnt_12M,Home_Equity_Loans_granted_contracts_cnt_12M,Other_Loans_granted_contracts_cnt_12M,Personal_Loans_granted_contracts_cnt_12M,Real_Estate_Loans_granted_contracts_cnt_12M,Short_and_Term_Loans_granted_contracts_cnt_12M,Time_Loans_granted_contracts_cnt_12M,Trust_Loans_granted_contracts_cnt_12M,Credit_Cards_granted_contracts_cnt_24M,Credit_Lines_granted_contracts_cnt_24M,Home_Equity_Loans_granted_contracts_cnt_24M,Other_Loans_granted_contracts_cnt_24M,Personal_Loans_granted_contracts_cnt_24M,Real_Estate_Loans_granted_contracts_cnt_24M,Short_and_Term_Loans_granted_contracts_cnt_24M,Time_Loans_granted_contracts_cnt_24M,Trust_Loans_granted_contracts_cnt_24M,granted_contracts_cnt_6M,granted_contracts_cnt_12M,granted_contracts_cnt_24M,Credit_Cards_granted_contracts_amt_6M,Credit_Lines_granted_contracts_amt_6M,Home_Equity_Loans_granted_contracts_amt_6M,Other_Loans_granted_contracts_amt_6M,Personal_Loans_granted_contracts_amt_6M,Real_Estate_Loans_granted_contracts_amt_6M,Short_and_Term_Loans_granted_contracts_amt_6M,Time_Loans_granted_contracts_amt_6M,Trust_Loans_granted_contracts_amt_6M,Credit_Cards_granted_contracts_amt_12M,Credit_Lines_granted_contracts_amt_12M,Home_Equity_Loans_granted_contracts_amt_12M,Other_Loans_granted_contracts_amt_12M,Personal_Loans_granted_contracts_amt_12M,Real_Estate_Loans_granted_contracts_amt_12M,Short_and_Term_Loans_granted_contracts_amt_12M,Time_Loans_granted_contracts_amt_12M,Trust_Loans_granted_contracts_amt_12M,Credit_Cards_granted_contracts_amt_24M,Credit_Lines_granted_contracts_amt_24M,Home_Equity_Loans_granted_contracts_amt_24M,Other_Loans_granted_contracts_amt_24M,Personal_Loans_granted_contracts_amt_24M,R

In [268]:
# Find all rows with duplicates (including the first occurrence)
all_duplicate_rows = Master_table[Master_table.duplicated(keep=False)]

# Display all duplicate rows
print(all_duplicate_rows.head())

Empty DataFrame
Columns: [customerId, digitalLoanAccountId, tinNumber, dateOfBirth, email, mobileNo, firstName, middleName, lastName, custLoanJourneyZipCode, custLoanJourneyCity, tsa_onboarding_time, new_loan_type, natureofwork, age, Education, industryDescription, custLoanJourneyProvince, monthlyIncome, AnnualIncome, incomeperSalaryScale, incomeperSalaryScale_Annual, gender, credolabScore, maritalStatus, dependentsCount, Region_Name, deviceType, brand, osversion, appVersion, defFPD15, obsFPD15, obs_fpd30_vol, def_fpd30_vol, obs_fpd30_cnt, def_fpd30_cnt, obs_fspd30_vol, def_fspd30_vol, obs_fspd30_cnt, def_fspd30_cnt, obs_fstpd30_vol, def_fstpd30_vol, obs_fstpd30_cnt, def_fstpd30_cnt, OccupationDesc, OccupationStatusDesc, PSICDesc, TradeName, AnnualMonthlyIndicator, GrossIncome, AnnualIncomecalculated, LastUpdateDate, cnt_active_contracts, cnt_active_contracts_Credit_Cards, cnt_active_contracts_Credit_Lines, cnt_active_contracts_Home_Equity_Loans, cnt_active_contracts_Other_Loans, cnt_a

In [269]:
duplicateincolumn(Master_table, 'digitalLoanAccountId')

No duplicates found in the digitalLoanAccountId column


In [270]:
Master_table.to_csv("Master_table_final_20240823.csv", index = False)

In [271]:
Master_table.columns.values

array(['customerId', 'digitalLoanAccountId', 'tinNumber', 'dateOfBirth',
       'email', 'mobileNo', 'firstName', 'middleName', 'lastName',
       'custLoanJourneyZipCode', 'custLoanJourneyCity',
       'tsa_onboarding_time', 'new_loan_type', 'natureofwork', 'age',
       'Education', 'industryDescription', 'custLoanJourneyProvince',
       'monthlyIncome', 'AnnualIncome', 'incomeperSalaryScale',
       'incomeperSalaryScale_Annual', 'gender', 'credolabScore',
       'maritalStatus', 'dependentsCount', 'Region_Name', 'deviceType',
       'brand', 'osversion', 'appVersion', 'defFPD15', 'obsFPD15',
       'obs_fpd30_vol', 'def_fpd30_vol', 'obs_fpd30_cnt', 'def_fpd30_cnt',
       'obs_fspd30_vol', 'def_fspd30_vol', 'obs_fspd30_cnt',
       'def_fspd30_cnt', 'obs_fstpd30_vol', 'def_fstpd30_vol',
       'obs_fstpd30_cnt', 'def_fstpd30_cnt', 'OccupationDesc',
       'OccupationStatusDesc', 'PSICDesc', 'TradeName',
       'AnnualMonthlyIndicator', 'GrossIncome', 'AnnualIncomecalculated',
    

In [272]:
Master_table.sample(100).to_csv("MasterSample.csv", index = False)

In [275]:
dfd = pd.read_csv(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Biswa\IncomeEstimation\Data\Master_table_final_20240821.csv")
dfd.shape

C:\Users\DwaipayanChakroborti\AppData\Local\Temp\ipykernel_24932\3381811760.py:1: DtypeWarning: Columns (2,64,75,76,77,78,79,80,81,82,206) have mixed types. Specify dtype option on import or set low_memory=False.
  dfd = pd.read_csv(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Biswa\IncomeEstimation\Data\Master_table_final_20240821.csv")


(1276974, 273)

In [276]:
# Find all rows with duplicates (including the first occurrence)
all_duplicate_rows = dfd[dfd.duplicated(keep=False)]

# Display all duplicate rows
print(all_duplicate_rows.head())

Empty DataFrame
Columns: [customerId, digitalLoanAccountId, tinNumber, dateOfBirth, email, mobileNo, firstName, middleName, lastName, custLoanJourneyZipCode, custLoanJourneyCity, tsa_onboarding_time, new_loan_type, natureofwork, age, Education, industryDescription, custLoanJourneyProvince, monthlyIncome, AnnualIncome, incomeperSalaryScale, incomeperSalaryScale_Annual, gender, credolabScore, maritalStatus, dependentsCount, Region_Name, deviceType, brand, osversion, appVersion, defFPD15, obsFPD15, obs_fpd30_vol, def_fpd30_vol, obs_fpd30_cnt, def_fpd30_cnt, obs_fspd30_vol, def_fspd30_vol, obs_fspd30_cnt, def_fspd30_cnt, obs_fstpd30_vol, def_fstpd30_vol, obs_fstpd30_cnt, def_fstpd30_cnt, OccupationDesc, OccupationStatusDesc, PSICDesc, TradeName, AnnualMonthlyIndicator, GrossIncome, AnnualIncomecalculated, LastUpdateDate, cnt_active_contracts, cnt_active_contracts_Credit_Cards, cnt_active_contracts_Credit_Lines, cnt_active_contracts_Home_Equity_Loans, cnt_active_contracts_Other_Loans, cnt_a

In [277]:
dfd[dfd['digitalLoanAccountId'] =='70f7ac1f-fa19-40ec-a038-3be19104d062']

,customerId,digitalLoanAccountId,tinNumber,dateOfBirth,email,mobileNo,firstName,middleName,lastName,custLoanJourneyZipCode,custLoanJourneyCity,tsa_onboarding_time,new_loan_type,natureofwork,age,Education,industryDescription,custLoanJourneyProvince,monthlyIncome,AnnualIncome,incomeperSalaryScale,incomeperSalaryScale_Annual,gender,credolabScore,maritalStatus,dependentsCount,Region_Name,deviceType,brand,osversion,appVersion,defFPD15,obsFPD15,obs_fpd30_vol,def_fpd30_vol,obs_fpd30_cnt,def_fpd30_cnt,obs_fspd30_vol,def_fspd30_vol,obs_fspd30_cnt,def_fspd30_cnt,obs_fstpd30_vol,def_fstpd30_vol,obs_fstpd30_cnt,def_fstpd30_cnt,OccupationDesc,OccupationStatusDesc,PSICDesc,TradeName,AnnualMonthlyIndicator,GrossIncome,AnnualIncomecalculated,LastUpdateDate,cnt_active_contracts,cnt_active_contracts_Credit_Cards,cnt_active_contracts_Credit_Lines,cnt_active_contracts_Home_Equity_Loans,cnt_active_contracts_Other_Loans,cnt_active_contracts_Personal_Loans,cnt_active_contracts_Real_Estate_Loans,cnt_active_contracts_Short_and_Term_Loans,cnt_active_contracts_Time_Loans,cnt_active_contracts_Trust_Loans,max_amt_active_contracts,tot_active_contracts_os_amt,tot_active_contracts_util,Credit_Cards_active_contracts_util,Credit_Lines_active_contracts_util,Home_Equity_Loans_active_contracts_util,Other_Loans_active_contracts_util,Personal_Loans_active_contracts_util,Real_Estate_Loans_active_contracts_util,Short_and_Term_Loans_active_contracts_util,Time_Loans_active_contracts_util,Trust_Loans_active_contracts_util,last_repay_amt_30d,highest_latest_repay_method_30d,last_repay_amt_60d,highest_latest_repay_method_60d,last_repay_amt_90d,highest_latest_repay_method_90d,last_repay_amt,highest_latest_repay_method,max_age_active_contracts_snapshot,max_age_all_contracts_snapshot,days_since_last_inquiry,days_since_last_grant,days_since_last_closed,days_since_last_repay,days_since_last_reject,days_since_last_renounce,Credit_Cards_granted_contracts_cnt_6M,Credit_Lines_granted_contracts_cnt_6M,Home_Equity_Loans_granted_contracts_cnt_6M,Other_Loans_granted_contracts_cnt_6M,Personal_Loans_granted_contracts_cnt_6M,Real_Estate_Loans_granted_contracts_cnt_6M,Short_and_Term_Loans_granted_contracts_cnt_6M,Time_Loans_granted_contracts_cnt_6M,Trust_Loans_granted_contracts_cnt_6M,Credit_Cards_granted_contracts_cnt_12M,Credit_Lines_granted_contracts_cnt_12M,Home_Equity_Loans_granted_contracts_cnt_12M,Other_Loans_granted_contracts_cnt_12M,Personal_Loans_granted_contracts_cnt_12M,Real_Estate_Loans_granted_contracts_cnt_12M,Short_and_Term_Loans_granted_contracts_cnt_12M,Time_Loans_granted_contracts_cnt_12M,Trust_Loans_granted_contracts_cnt_12M,Credit_Cards_granted_contracts_cnt_24M,Credit_Lines_granted_contracts_cnt_24M,Home_Equity_Loans_granted_contracts_cnt_24M,Other_Loans_granted_contracts_cnt_24M,Personal_Loans_granted_contracts_cnt_24M,Real_Estate_Loans_granted_contracts_cnt_24M,Short_and_Term_Loans_granted_contracts_cnt_24M,Time_Loans_granted_contracts_cnt_24M,Trust_Loans_granted_contracts_cnt_24M,granted_contracts_cnt_6M,granted_contracts_cnt_12M,granted_contracts_cnt_24M,Credit_Cards_granted_contracts_amt_6M,Credit_Lines_granted_contracts_amt_6M,Home_Equity_Loans_granted_contracts_amt_6M,Other_Loans_granted_contracts_amt_6M,Personal_Loans_granted_contracts_amt_6M,Real_Estate_Loans_granted_contracts_amt_6M,Short_and_Term_Loans_granted_contracts_amt_6M,Time_Loans_granted_contracts_amt_6M,Trust_Loans_granted_contracts_amt_6M,Credit_Cards_granted_contracts_amt_12M,Credit_Lines_granted_contracts_amt_12M,Home_Equity_Loans_granted_contracts_amt_12M,Other_Loans_granted_contracts_amt_12M,Personal_Loans_granted_contracts_amt_12M,Real_Estate_Loans_granted_contracts_amt_12M,Short_and_Term_Loans_granted_contracts_amt_12M,Time_Loans_granted_contracts_amt_12M,Trust_Loans_granted_contracts_amt_12M,Credit_Cards_granted_contracts_amt_24M,Credit_Lines_granted_contracts_amt_24M,Home_Equity_Loans_granted_contracts_amt_24M,Other_Loans_granted_contracts_amt_24M,Personal_Loans_granted_contracts_amt_24M,R

In [278]:
dfd = dfd.drop_duplicates(subset=['digitalLoanAccountId'])

In [279]:
dfd.to_csv("Master_table_final_20240830.csv", index = False)

In [280]:
dfd['digitalLoanAccountId'].nunique()

1276974

In [294]:
pd.set_option("Display.max_rows", None)
column_types = dfd.dtypes

print(column_types)


customerId                                        float64
digitalLoanAccountId                               object
tinNumber                                          object
dateOfBirth                                        object
email                                              object
mobileNo                                          float64
firstName                                          object
middleName                                         object
lastName                                           object
custLoanJourneyZipCode                            float64
custLoanJourneyCity                                object
tsa_onboarding_time                                object
new_loan_type                                      object
natureofwork                                       object
age                                               float64
Education                                          object
industryDescription                                object
custLoanJourne